In [6]:
import warnings
warnings.filterwarnings(action='ignore')

# 0. Data 합치기

### 1) data: (서울시 열린데이터 광장) 상권 매출 + (서울시 열린데이터 광장) 상권 영역

In [8]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

data = pd.read_csv(path+'train_test.csv', encoding = 'euc_kr')
data.head()

,Unnamed: 0,기준_년_코드,기준_분기_코드,시군구_코드,행정동_코드,당월_매출_금액
0,0,2013,1,11140,11140520,2.235364e+11
1,1,2013,1,11140,11140540,1.111424e+10
2,2,2013,1,11140,11140550,1.046387e+11
3,3,2013,1,11140,11140570,2.865646e+10
4,4,2013,1,11140,11140580,9.500578e+09


In [9]:
#데이터 변수명 바꾸기
data.columns 
data = data.rename(columns = {'Unnamed: 0':'index','기준_년_코드':'year','기준_분기_코드':'quarter','시군구_코드':'sigungu','행정동_코드':'dong', '당월_매출_금액':'amt' })

In [10]:
#데이터 타입 바꾸기
data['sigungu'] = data['sigungu'].astype('object')
data['dong'] = data['dong'].astype('object')
data['index'] = data['index'].astype('object')
data.dtypes

index       object
year         int64
quarter      int64
sigungu     object
dong        object
amt        float64
dtype: object

In [11]:
#2020년 데이터 test데이터 저장
data20 = data[data['year']==2020]
data20.head()

,index,year,quarter,sigungu,dong,amt
679,679,2020,1,11140,11140520,1.748050e+11
680,680,2020,1,11140,11140540,9.133832e+09
681,681,2020,1,11140,11140550,6.296640e+10
682,682,2020,1,11140,11140570,2.408009e+10
683,683,2020,1,11140,11140580,8.447369e+09


In [12]:
#20년도: test, 나머지: train
data1319 = data[data['year']!=2020]
data1319.head()

,index,year,quarter,sigungu,dong,amt
0,0,2013,1,11140,11140520,2.235364e+11
1,1,2013,1,11140,11140540,1.111424e+10
2,2,2013,1,11140,11140550,1.046387e+11
3,3,2013,1,11140,11140570,2.865646e+10
4,4,2013,1,11140,11140580,9.500578e+09


### 2)  상권영역+상주인구+소득소비+아파트+직장인구+집객시설+추정유동인구
### (서울시 열린데이터 광장)

각 데이터가 모든 연도가 있지 않았기 때문에 확인을 통해 공통 년도로 데이터를 합쳤다

In [13]:
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\raw데이터\\'
area = pd.read_csv(path+'상권영역.csv', encoding = 'euc_kr') #상권영역
area['기준_년월_코드'].unique()

array([201810], dtype=int64)

In [14]:
resident = pd.read_csv(path+'상주인구.csv', encoding = 'euc_kr') #상주인구
resident['기준_년_코드'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [15]:
money = pd.read_csv(path+'소득소비.csv', encoding = 'euc_kr') #소득소비
money['기준 년 코드'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [16]:
apt = pd.read_csv(path+'아파트.csv', encoding = 'euc_kr') #아파트
apt['기준_년_코드'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [17]:
worker = pd.read_csv(path+'직장인구.csv', encoding = 'euc_kr') #직장인구
worker['기준_년_코드'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [18]:
facility = pd.read_csv(path+'집객시설.csv', encoding = 'euc_kr') #집객시설
facility = facility.fillna(0)
facility['기준_년_코드'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015], dtype=int64)

In [19]:
fluid = pd.read_csv(path+'추정유동인구.csv', encoding = 'euc_kr') #추정유동인구
fluid['기준 년코드'].unique()

array([2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

데이터 merge

In [53]:
#앞선 data를 통해 행정구 코드 확인
data.head()
data['sigungu'].unique() #11140:중구, 11350: 노원구

array([11140, 11350], dtype=object)

In [54]:
#area데이터에서 노원구, 중구만 추출해서 저장
area = area[['시군구_코드','행정동_코드','상권_구분_코드','상권_코드']]
area1 = area[area['시군구_코드']==11140]
area2 = area[area['시군구_코드']==11350]
# area2.head()
area = pd.concat([area1, area2])
area['시군구_코드'].unique()
area.head()

,시군구_코드,행정동_코드,상권_구분_코드,상권_코드
69,11140,11140520,U,1001492
70,11140,11140590,U,1001493
82,11140,11140550,A,1000039
83,11140,11140605,A,1000040
409,11140,11140540,A,1000044


In [55]:
resident.shape
resident = resident[resident.columns[0:25]] #필요한 칼럼만 저장
resident.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권 코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,남성연령대 20 상주인구 수,남성연령대 30 상주인구 수,남성연령대 40 상주인구 수,남성연령대 50 상주인구 수,남성연령대 60 이상 상주인구 수,여성연령대 10 상주인구 수,여성연령대 20 상주인구 수,여성연령대 30 상주인구 수,여성연령대 40 상주인구 수,여성연령대 50 상주인구 수
0,2020,1,U,관광특구,1001496,24,16,8,6,2,...,1,1,1,5,4,2,1,1,2,0
1,2020,1,U,관광특구,1001495,8221,3708,4513,822,1679,...,639,903,511,500,730,397,1040,1160,628,497
2,2020,1,U,관광특구,1001494,1924,1274,650,49,195,...,135,135,145,320,511,21,60,65,64,130
3,2020,1,U,관광특구,1001493,2427,1146,1281,169,505,...,193,273,183,191,234,97,312,290,179,178
4,2020,1,U,관광특구,1001492,1104,401,703,15,51,...,25,59,61,90,163,12,26,54,120,149


data2 = 상주인구+상권영역

In [61]:
data2 = resident.copy()
data2 = data2.rename(columns = {'상권 코드':'상권_코드'})
data2 = pd.merge(data2, area, left_on = '상권_코드', right_on = '상권_코드', how = 'left')

,기준_년_코드,기준_분기_코드,상권_구분_코드_x,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,남성연령대 50 상주인구 수,남성연령대 60 이상 상주인구 수,여성연령대 10 상주인구 수,여성연령대 20 상주인구 수,여성연령대 30 상주인구 수,여성연령대 40 상주인구 수,여성연령대 50 상주인구 수,시군구_코드,행정동_코드,상권_구분_코드_y
0,2020,1,U,관광특구,1001496,24,16,8,6,2,...,5,4,2,1,1,2,0,NaN,NaN,NaN
1,2020,1,U,관광특구,1001495,8221,3708,4513,822,1679,...,500,730,397,1040,1160,628,497,NaN,NaN,NaN
2,2020,1,U,관광특구,1001494,1924,1274,650,49,195,...,320,511,21,60,65,64,130,NaN,NaN,NaN
3,2020,1,U,관광특구,1001493,2427,1146,1281,169,505,...,191,234,97,312,290,179,178,11140.0,11140590.0,U
4,2020,1,U,관광특구,1001492,1104,401,703,15,51,...,90,163,12,26,54,120,149,11140.0,11140520.0,U
5,2020,1,U,관광특구,1001491,5176,2462,2714,366,699,...,404,626,174,345,431,405,439,NaN,NaN,NaN
6,2020,1,R,전통시장,1001490,3097,1540,1557,440,373,...,221,387,194,170,228,226,270,NaN,NaN,NaN
7,2020,1,R,전통시장,1001489,595,304,291,81,102,...,68,62,39,41,29,49,58,NaN,NaN,NaN
8,2020,1,R,전통시장,1001488,1023,527,496,115,140,...,89,135,66,68,75,76,78,NaN,NaN,NaN
9,2020,1,R,전통시장,1001487,90,52,38,1,7,...,18,17,1,2,6,7,7,NaN,NaN,NaN


In [65]:
data2 = data2.dropna(axis=0)
data2 = data2[data2.columns[0:27]]
data2 = data2.rename(columns = {'상권_구분_코드_x':'상권_구분_코드'})
data2.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,남성연령대 40 상주인구 수,남성연령대 50 상주인구 수,남성연령대 60 이상 상주인구 수,여성연령대 10 상주인구 수,여성연령대 20 상주인구 수,여성연령대 30 상주인구 수,여성연령대 40 상주인구 수,여성연령대 50 상주인구 수,시군구_코드,행정동_코드
3,2020,1,U,관광특구,1001493,2427,1146,1281,169,505,...,183,191,234,97,312,290,179,178,11140.0,11140590.0
4,2020,1,U,관광특구,1001492,1104,401,703,15,51,...,61,90,163,12,26,54,120,149,11140.0,11140520.0
125,2020,1,R,전통시장,1001367,1833,911,922,207,298,...,152,172,232,99,146,72,138,183,11350.0,11350670.0
126,2020,1,R,전통시장,1001365,1498,764,734,151,324,...,97,115,183,80,170,130,79,112,11350.0,11350595.0
204,2020,1,R,전통시장,1001285,90,71,19,4,6,...,11,21,26,1,1,1,6,2,11140.0,11140680.0


In [37]:
money = money[money.columns[0:25]] #필요한 칼럼만 가져감
money.head()

,기준 년 코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,2020,1,A,골목상권,1001010,풍성로37가길,2850735,6,12120296344,3292343150,1466629217,892634747,1607315609,1972205430,444384439,377360177,1550457530,516966045
1,2020,1,A,골목상권,1001009,천호옛길,3146942,7,8976116500,2526811041,1073907318,659101424,1225903534,1395888645,327727539,283116450,1109489033,374171516
2,2020,1,A,골목상권,1001008,천호옛14길,2774214,6,6175425196,1846295541,733624837,464028324,882017929,906576672,219745655,205715662,662629513,254791063
3,2020,1,A,골목상권,1001007,천호대로219길,2561943,6,1609692543,452889712,190742939,108296299,224205264,256333051,52791935,47847994,208823957,67761392
4,2020,1,A,골목상권,1001006,천호대로197길,3001401,7,10437230074,2691268507,1267558670,747339442,1326410044,1722932932,403612797,304016442,1535009509,439081731


In [67]:
money = money.rename(columns = {'기준 년 코드':'기준_년_코드'}) #data2와 money 칼럼명 맞춤

data3 = 상권영역 + 상주인구 + 소득소비

In [75]:
data3 = pd.merge(data2, money, left_on = ['상권_코드','기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명'], right_on = ['상권_코드','기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명'])
data3.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,2020,1,A,골목상권,1000388,3728,1845,1883,430,476,...,9815978102,2862656385,1145054097,670211660,1414871448,1464698013,319290234,300080747,1235605938,403509580
1,2020,1,A,골목상권,1000387,2311,1172,1139,227,339,...,10185630041,2919741079,1194240233,696405952,1445475658,1526016629,352403988,308588125,1332900290,409858087
2,2020,1,A,골목상권,1000386,1832,946,886,314,259,...,7955571980,2197447668,945562580,536645263,1078064038,1203613703,290233581,232887804,1148332204,322785139
3,2020,1,A,골목상권,1000385,2011,1031,980,241,264,...,5986196032,1696112801,715778366,417588915,831047874,924451856,211925887,184537362,756197854,248555117
4,2020,1,A,골목상권,1000384,2421,1244,1177,252,337,...,10246196062,2854237177,1218498045,722060927,1418558663,1571414895,380655748,308970857,1356497466,415302284


data4 = 상권영역 + 상주인구 + 소득소비 + 아파트

In [94]:
data4 = pd.merge(data3, apt, left_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'], right_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'])
data4.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,2020,1,A,골목상권,1000388,3728,1845,1883,430,476,...,0,968,1753,1780,176,0,0,0,50,116940598
1,2020,1,A,골목상권,1000387,2311,1172,1139,227,339,...,8,746,2120,1873,320,67,8,0,51,121036725
2,2020,1,A,골목상권,1000386,1832,946,886,314,259,...,0,630,1754,2086,1163,136,96,0,62,191498464
3,2020,1,A,골목상권,1000385,2011,1031,980,241,264,...,0,327,725,1424,1239,149,0,0,57,163091743
4,2020,1,A,골목상권,1000384,2421,1244,1177,252,337,...,8,947,1655,1365,498,67,8,0,51,123547229


In [92]:
worker.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수', '연령대_10_직장_인구_수',
       '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수',
       '연령대_60_이상_직장_인구_수', '남성연령대_10_직장_인구_수', '남성연령대_20_직장_인구_수',
       '남성연령대_30_직장_인구_수', '남성연령대_40_직장_인구_수', '남성연령대_50_직장_인구_수',
       '남성연령대_60_이상_직장_인구_수', '여성연령대_10_직장_인구_수', '여성연령대_20_직장_인구_수',
       '여성연령대_30_직장_인구_수', '여성연령대_40_직장_인구_수', '여성연령대_50_직장_인구_수',
       '여성연령대_60_이상_직장_인구_수'],
      dtype='object')

data5 = 상권영역 + 상주인구 + 소득소비 + 아파트 + 직장인구

In [121]:
data5 = pd.merge(data4, worker, left_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'], right_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'])
data5.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,2020,1,A,골목상권,1000388,3728,1845,1883,430,476,...,84,104,182,383,1,56,74,151,234,298
1,2020,1,A,골목상권,1000387,2311,1172,1139,227,339,...,105,152,179,156,4,93,117,175,205,128
2,2020,1,A,골목상권,1000386,1832,946,886,314,259,...,195,259,208,202,5,139,198,246,284,136
3,2020,1,A,골목상권,1000385,2011,1031,980,241,264,...,112,183,225,224,2,115,161,239,252,118
4,2020,1,A,골목상권,1000384,2421,1244,1177,252,337,...,161,196,203,138,2,143,148,275,291,153


data6 = 상권영역 + 상주인구 + 소득소비 + 아파트 + 직장인구 + 집객시설

In [124]:
data6 = pd.merge(data5, facility, left_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'], right_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'])
data6.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,2020,1,A,골목상권,1000388,3728,1845,1883,430,476,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2020,1,A,골목상권,1000387,2311,1172,1139,227,339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,2020,1,A,골목상권,1000386,1832,946,886,314,259,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0
3,2020,1,A,골목상권,1000385,2011,1031,980,241,264,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0
4,2020,1,A,골목상권,1000384,2421,1244,1177,252,337,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [109]:
# fluid 데이터 칼럼명 맞추기
fluid = fluid.rename(columns = {'기준 년코드':'기준_년_코드', ' 상권_구분_코드':'상권_구분_코드',' 상권_구분_코드_명':'상권_구분_코드_명'})

data7 = 상권영역 + 상주인구 + 소득소비 + 아파트 + 점포 + 직장인구 + 집객시설 + 추정유동인구

In [126]:
data7 = pd.merge(data6, fluid, left_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'], right_on = ['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드','상권_코드_명'])
data7.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
0,2019,2,A,골목상권,1000388,3932,1969,1963,500,524,...,659,685,786,557,1045,976,731,614,790,491
1,2019,2,A,골목상권,1000387,2465,1255,1210,273,353,...,742,719,967,548,899,1028,1089,817,882,491
2,2019,2,A,골목상권,1000386,1815,926,889,327,251,...,1033,894,1470,813,1157,934,972,872,1371,691
3,2019,2,A,골목상권,1000385,2084,1078,1006,244,259,...,475,409,610,322,446,441,439,363,549,255
4,2019,2,A,골목상권,1000384,2597,1341,1256,295,362,...,1535,1568,1853,927,1556,1786,1537,1340,1582,885


In [127]:
data7.to_excel('totaldata.xlsx')

In [ ]:
rawdata = data7

최종 데이터를 rawdata로 저장

In [31]:
rawdata = pd.merge(data, rawdata, left_on = ['year','quarter','dong'], right_on = ['기준_년_코드', '기준_분기_코드','행정동_코드'])
rawdata=rawdata.drop(['index'],axis = 'columns')
rawdata.head()
rawdata.to_excel('finaldata.xlsx')

In [15]:
rawdata.head()

,Unnamed: 0,year,quarter,sigungu,dong,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
0,0,2015,1,11140,11140540,11152009251,657,376,281,35,...,294,251,112,3,0,17,85,101,53,4
1,1,2015,1,11140,11140550,99170342338,964,353,611,0,...,42,36,14,0,0,5,27,22,6,0
2,2,2015,1,11140,11140580,9889692667,2881,1575,1306,123,...,704,625,551,33,0,107,508,485,496,67
3,3,2015,1,11140,11140605,26217489487,6,0,6,0,...,143,131,89,7,0,59,183,152,62,3
4,4,2015,1,11140,11140615,8372678394,1750,1085,665,34,...,772,582,476,29,0,116,340,365,471,178


In [5]:
rawdata[['기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','시군구_코드','행정동_코드','상권_코드_명','소득_구간_코드']] = rawdata[['기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','시군구_코드','행정동_코드','상권_코드_명','소득_구간_코드']].astype('object')
rawdata[['기준_년_코드','기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','시군구_코드','행정동_코드','상권_코드_명','소득_구간_코드']].dtypes

기준_년_코드       object
기준_분기_코드      object
상권_구분_코드      object
상권_구분_코드_명    object
상권_코드         object
시군구_코드        object
행정동_코드        object
상권_코드_명       object
소득_구간_코드      object
dtype: object

In [11]:
rawdata = rawdata.groupby(['기준_년_코드','기준_분기_코드','행정동_코드']).sum()
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)
rawdata

총 상주인구 수  남성 상주인구 수  여성 상주인구 수  연령대 10 상주인구 수  \
기준_년_코드 기준_분기_코드 행정동_코드                                                    
2015    1        11140540       657        376        281             35   
                 11140550       964        353        611              0   
                 11140580      2881       1575       1306            123   
                 11140605         6          0          6              0   
                 11140615      1750       1085        665             34   
...                             ...        ...        ...            ...   
2019    2        11350611      1523        759        764            258   
                 11350624      1815        926        889            327   
                 11350630      2506       1237       1269            315   
                 11350640      7366       3723       3643            923   
                 11350670      6397       3224       3173            773   

                           연령대 20 상주인구 수  연령대 30 상주인구 수  연령대 40 상주인구 수  \
기준_년_코드 기준_분기_코드 행정동_코드                                                  
2015    1        11140540             55             80            119   
                 11140550             55            171            244   
                 11140580            664            615            600   
                 11140605              0              0              0   
                 11140615            219            224            610   
...                                  ...            ...            ...   
2019    2        11350611            330            161            227   
                 11350624            251            193            304   
                 11350630            413            306            318   
                 11350640           1173            910           1134   
                 11350670            877            733            981   

                           연령대 50 상주인구 수  연령대 60 이상 상주인구 수  남성연령대 10 상주인구 수  \
기준_년_코드 기준_분기_코드 행정동_코드                                                       
2015    1        11140540            194               174               16   
                 11140550            317               177                0   
                 11140580            611               268               79   
                 11140605              6                 0                0   
                 11140615            606                57               14   
...                                  ...               ...              ...   
2019    2        11350611            264               283              145   
                 11350624            328               412              164   
                 11350630            456               698              169   
                 11350640           1330              1896              491   
                 11350670           1122              1911              390   

                           ...  여성연령대_60_이상_토요일시간대_3_유동인구_수  \
기준_년_코드 기준_분기_코드 행정동_코드    ...                                
2015    1        11140540  ...                          294   
                 11140550  ...                           42   
                 11140580  ...                          704   
                 11140605  ...                          143   
                 11140615  ...                          772   
...                        ...                          ...   
2019    2        11350611  ...                          159   
                 11350624  ...                         1033   
                 11350630  ...                          719   
                 11350640  ...                         2651   
                 11350670  ...                         1401   

                           여성연령대_60_이상_토요일시간대_4_유동인구_수  \
기준_년_코드 기준_분기_코드 행정동_코드                                  
2015    1        11140540                          251   
                 11140550                           3

# 1. Regression

### 1) Lasso Regression - year, quarter 모두 object

##### #전지역

In [20]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

rawdata = pd.read_excel(path+'finaldata.xlsx', encoding = 'euc_kr')
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)

데이터에 존재하는 연도

In [126]:
rawdata['year'].unique()

array([2015, 2016, 2017, 2018, 2019], dtype=int64)

In [16]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

MAE 계산 함수

In [21]:
#MAE
def mae(y,yhat):
    return np.mean(abs(y - yhat))

In [ ]:
# data = data.rename(columns = {'Unnamed: 0':'index','기준_년_코드':'year','기준_분기_코드':'quarter','시군구_코드':'sigungu','행정동_코드':'dong', '당월_매출_금액':'amt' })

In [23]:
rawdata[['year','quarter']] = rawdata[['year','quarter']].astype('object')
rawdata.dtypes

year                           object
quarter                        object
sigungu                         int64
dong                            int64
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

rawdata1 : year, quarter를 object로 생각해 dummy variable로 만든 데이터

In [24]:
rawdata1 = pd.get_dummies(rawdata, columns=['year','quarter','sigungu','dong'], prefix=['year','quarter','sigungu','dong'])

매출 변수인 amt의 변수가 숫자가 크기때문에 log값을 취해 새로운 변수 logamt로 만들어준다

In [34]:
import numpy as np
rawdata1["Logamt"] = np.log1p(rawdata1["amt"])
rawdata1 = rawdata1.drop(['amt'],axis=1)
rawdata1.head()
rawdata1.to_excel('realfinaldata.xlsx')

In [35]:
#Regression function
def predict_columns(data_frame, x_cols, y_cols):   #x_cols: 설명변수, y_cols: 반응변수
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]
    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        print('{} : {},  {},  {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse))

In [37]:
columnname = rawdata1.columns
columnname = pd.DataFrame(columnname)
columnname.shape

(643, 1)

칼럼명을 저장하는 array 생성

In [38]:
column_array = []
for i in range(643):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()

In [39]:
column_array

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_

In [99]:
# column_array.remove('amt')

In [40]:
rawdata1.columns

Index(['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수',
       '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수', '연령대 60 이상 상주인구 수',
       '남성연령대 10 상주인구 수',
       ...
       'dong_11140665', 'dong_11140670', 'dong_11350560', 'dong_11350595',
       'dong_11350611', 'dong_11350624', 'dong_11350630', 'dong_11350640',
       'dong_11350670', 'Logamt'],
      dtype='object', length=643)

rawdata1을 이용해 Lasso Regression으로 Logamt에 영향을 끼치는 변수만 추린다

In [42]:
def predict_columns(data_frame, x_cols, y_cols):   #x_cols: 설명변수, y_cols: 반응변수
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]
    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

variable : 변수 모음 

In [43]:
variable = ['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_10_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '여성연령대_60_이상_직장_인구_수',
 '집객시설_수',
 '관공서_수',
 '은행_수',
 '종합병원_수',
 '일반_병원_수',
 '약국_수',
 '유치원_수',
 '초등학교_수',
 '중학교_수',
 '고등학교_수',
 '대학교_수',
 '백화점_수',
 '슈퍼마켓_수',
 '극장_수',
 '숙박_시설_수',
 '공항_수',
 '철도_역_수',
 '버스_터미널_수',
 '지하철_역_수',
 '버스_정거장_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대_6_유동인구_수',
 '월요일_유동인구_수',
 '화요일_유동인구_수',
 '수요일_유동인구_수',
 '목요일_유동인구_수',
 '금요일_유동인구_수',
 '토요일_유동인구_수',
 '일요일_유동인구_수',
 '남성연령대_10_월요일시간대_1_유동인구_수',
 '남성연령대_10_월요일시간대_2_유동인구_수',
 '남성연령대_10_월요일시간대_3_유동인구_수',
 '남성연령대_10_월요일시간대_4_유동인구_수',
 '남성연령대_10_월요일시간대_5_유동인구_수',
 '남성연령대_10_월요일시간대_6_유동인구_수',
 '남성연령대_10_화요일시간대_1_유동인구_수',
 '남성연령대_10_화요일시간대_2_유동인구_수',
 '남성연령대_10_화요일시간대_3_유동인구_수',
 '남성연령대_10_화요일시간대_4_유동인구_수',
 '남성연령대_10_화요일시간대_5_유동인구_수',
 '남성연령대_10_화요일시간대_6_유동인구_수',
 '남성연령대_10_수요일시간대_1_유동인구_수',
 '남성연령대_10_수요일시간대_2_유동인구_수',
 '남성연령대_10_수요일시간대_3_유동인구_수',
 '남성연령대_10_수요일시간대_4_유동인구_수',
 '남성연령대_10_수요일시간대_5_유동인구_수',
 '남성연령대_10_수요일시간대_6_유동인구_수',
 '남성연령대_10_목요일시간대_1_유동인구_수',
 '남성연령대_10_목요일시간대_2_유동인구_수',
 '남성연령대_10_목요일시간대_3_유동인구_수',
 '남성연령대_10_목요일시간대_4_유동인구_수',
 '남성연령대_10_목요일시간대_5_유동인구_수',
 '남성연령대_10_목요일시간대_6_유동인구_수',
 '남성연령대_10_금요일시간대_1_유동인구_수',
 '남성연령대_10_금요일시간대_2_유동인구_수',
 '남성연령대_10_금요일시간대_3_유동인구_수',
 '남성연령대_10_금요일시간대_4_유동인구_수',
 '남성연령대_10_금요일시간대_5_유동인구_수',
 '남성연령대_10_금요일시간대_6_유동인구_수',
 '남성연령대_10_토요일시간대_1_유동인구_수',
 '남성연령대_10_토요일시간대_2_유동인구_수',
 '남성연령대_10_토요일시간대_3_유동인구_수',
 '남성연령대_10_토요일시간대_4_유동인구_수',
 '남성연령대_10_토요일시간대_5_유동인구_수',
 '남성연령대_10_토요일시간대_6_유동인구_수',
 '남성연령대_10_일요일시간대_1_유동인구_수',
 '남성연령대_10_일요일시간대_2_유동인구_수',
 '남성연령대_10_일요일시간대_3_유동인구_수',
 '남성연령대_10_일요일시간대_4_유동인구_수',
 '남성연령대_10_일요일시간대_5_유동인구_수',
 '남성연령대_10_일요일시간대_6_유동인구_수',
 '남성연령대_20_월요일시간대_1_유동인구_수',
 '남성연령대_20_월요일시간대_2_유동인구_수',
 '남성연령대_20_월요일시간대_3_유동인구_수',
 '남성연령대_20_월요일시간대_4_유동인구_수',
 '남성연령대_20_월요일시간대_5_유동인구_수',
 '남성연령대_20_월요일시간대_6_유동인구_수',
 '남성연령대_20_화요일시간대_1_유동인구_수',
 '남성연령대_20_화요일시간대_2_유동인구_수',
 '남성연령대_20_화요일시간대_3_유동인구_수',
 '남성연령대_20_화요일시간대_4_유동인구_수',
 '남성연령대_20_화요일시간대_5_유동인구_수',
 '남성연령대_20_화요일시간대_6_유동인구_수',
 '남성연령대_20_수요일시간대_1_유동인구_수',
 '남성연령대_20_수요일시간대_2_유동인구_수',
 '남성연령대_20_수요일시간대_3_유동인구_수',
 '남성연령대_20_수요일시간대_4_유동인구_수',
 '남성연령대_20_수요일시간대_5_유동인구_수',
 '남성연령대_20_수요일시간대_6_유동인구_수',
 '남성연령대_20_목요일시간대_1_유동인구_수',
 '남성연령대_20_목요일시간대_2_유동인구_수',
 '남성연령대_20_목요일시간대_3_유동인구_수',
 '남성연령대_20_목요일시간대_4_유동인구_수',
 '남성연령대_20_목요일시간대_5_유동인구_수',
 '남성연령대_20_목요일시간대_6_유동인구_수',
 '남성연령대_20_금요일시간대_1_유동인구_수',
 '남성연령대_20_금요일시간대_2_유동인구_수',
 '남성연령대_20_금요일시간대_3_유동인구_수',
 '남성연령대_20_금요일시간대_4_유동인구_수',
 '남성연령대_20_금요일시간대_5_유동인구_수',
 '남성연령대_20_금요일시간대_6_유동인구_수',
 '남성연령대_20_토요일시간대_1_유동인구_수',
 '남성연령대_20_토요일시간대_2_유동인구_수',
 '남성연령대_20_토요일시간대_3_유동인구_수',
 '남성연령대_20_토요일시간대_4_유동인구_수',
 '남성연령대_20_토요일시간대_5_유동인구_수',
 '남성연령대_20_토요일시간대_6_유동인구_수',
 '남성연령대_20_일요일시간대_1_유동인구_수',
 '남성연령대_20_일요일시간대_2_유동인구_수',
 '남성연령대_20_일요일시간대_3_유동인구_수',
 '남성연령대_20_일요일시간대_4_유동인구_수',
 '남성연령대_20_일요일시간대_5_유동인구_수',
 '남성연령대_20_일요일시간대_6_유동인구_수',
 '남성연령대_30_월요일시간대_1_유동인구_수',
 '남성연령대_30_월요일시간대_2_유동인구_수',
 '남성연령대_30_월요일시간대_3_유동인구_수',
 '남성연령대_30_월요일시간대_4_유동인구_수',
 '남성연령대_30_월요일시간대_5_유동인구_수',
 '남성연령대_30_월요일시간대_6_유동인구_수',
 '남성연령대_30_화요일시간대_1_유동인구_수',
 '남성연령대_30_화요일시간대_2_유동인구_수',
 '남성연령대_30_화요일시간대_3_유동인구_수',
 '남성연령대_30_화요일시간대_4_유동인구_수',
 '남성연령대_30_화요일시간대_5_유동인구_수',
 '남성연령대_30_화요일시간대_6_유동인구_수',
 '남성연령대_30_수요일시간대_1_유동인구_수',
 '남성연령대_30_수요일시간대_2_유동인구_수',
 '남성연령대_30_수요일시간대_3_유동인구_수',
 '남성연령대_30_수요일시간대_4_유동인구_수',
 '남성연령대_30_수요일시간대_5_유동인구_수',
 '남성연령대_30_수요일시간대_6_유동인구_수',
 '남성연령대_30_목요일시간대_1_유동인구_수',
 '남성연령대_30_목요일시간대_2_유동인구_수',
 '남성연령대_30_목요일시간대_3_유동인구_수',
 '남성연령대_30_목요일시간대_4_유동인구_수',
 '남성연령대_30_목요일시간대_5_유동인구_수',
 '남성연령대_30_목요일시간대_6_유동인구_수',
 '남성연령대_30_금요일시간대_1_유동인구_수',
 '남성연령대_30_금요일시간대_2_유동인구_수',
 '남성연령대_30_금요일시간대_3_유동인구_수',
 '남성연령대_30_금요일시간대_4_유동인구_수',
 '남성연령대_30_금요일시간대_5_유동인구_수',
 '남성연령대_30_금요일시간대_6_유동인구_수',
 '남성연령대_30_토요일시간대_1_유동인구_수',
 '남성연령대_30_토요일시간대_2_유동인구_수',
 '남성연령대_30_토요일시간대_3_유동인구_수',
 '남성연령대_30_토요일시간대_4_유동인구_수',
 '남성연령대_30_토요일시간대_5_유동인구_수',
 '남성연령대_30_토요일시간대_6_유동인구_수',
 '남성연령대_30_일요일시간대_1_유동인구_수',
 '남성연령대_30_일요일시간대_2_유동인구_수',
 '남성연령대_30_일요일시간대_3_유동인구_수',
 '남성연령대_30_일요일시간대_4_유동인구_수',
 '남성연령대_30_일요일시간대_5_유동인구_수',
 '남성연령대_30_일요일시간대_6_유동인구_수',
 '남성연령대_40_월요일시간대_1_유동인구_수',
 '남성연령대_40_월요일시간대_2_유동인구_수',
 '남성연령대_40_월요일시간대_3_유동인구_수',
 '남성연령대_40_월요일시간대_4_유동인구_수',
 '남성연령대_40_월요일시간대_5_유동인구_수',
 '남성연령대_40_월요일시간대_6_유동인구_수',
 '남성연령대_40_화요일시간대_1_유동인구_수',
 '남성연령대_40_화요일시간대_2_유동인구_수',
 '남성연령대_40_화요일시간대_3_유동인구_수',
 '남성연령대_40_화요일시간대_4_유동인구_수',
 '남성연령대_40_화요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_6_유동인구_수',
 '남성연령대_40_수요일시간대_1_유동인구_수',
 '남성연령대_40_수요일시간대_2_유동인구_수',
 '남성연령대_40_수요일시간대_3_유동인구_수',
 '남성연령대_40_수요일시간대_4_유동인구_수',
 '남성연령대_40_수요일시간대_5_유동인구_수',
 '남성연령대_40_수요일시간대_6_유동인구_수',
 '남성연령대_40_목요일시간대_1_유동인구_수',
 '남성연령대_40_목요일시간대_2_유동인구_수',
 '남성연령대_40_목요일시간대_3_유동인구_수',
 '남성연령대_40_목요일시간대_4_유동인구_수',
 '남성연령대_40_목요일시간대_5_유동인구_수',
 '남성연령대_40_목요일시간대_6_유동인구_수',
 '남성연령대_40_금요일시간대_1_유동인구_수',
 '남성연령대_40_금요일시간대_2_유동인구_수',
 '남성연령대_40_금요일시간대_3_유동인구_수',
 '남성연령대_40_금요일시간대_4_유동인구_수',
 '남성연령대_40_금요일시간대_5_유동인구_수',
 '남성연령대_40_금요일시간대_6_유동인구_수',
 '남성연령대_40_토요일시간대_1_유동인구_수',
 '남성연령대_40_토요일시간대_2_유동인구_수',
 '남성연령대_40_토요일시간대_3_유동인구_수',
 '남성연령대_40_토요일시간대_4_유동인구_수',
 '남성연령대_40_토요일시간대_5_유동인구_수',
 '남성연령대_40_토요일시간대_6_유동인구_수',
 '남성연령대_40_일요일시간대_1_유동인구_수',
 '남성연령대_40_일요일시간대_2_유동인구_수',
 '남성연령대_40_일요일시간대_3_유동인구_수',
 '남성연령대_40_일요일시간대_4_유동인구_수',
 '남성연령대_40_일요일시간대_5_유동인구_수',
 '남성연령대_40_일요일시간대_6_유동인구_수',
 '남성연령대_50_월요일시간대_1_유동인구_수',
 '남성연령대_50_월요일시간대_2_유동인구_수',
 '남성연령대_50_월요일시간대_3_유동인구_수',
 '남성연령대_50_월요일시간대_4_유동인구_수',
 '남성연령대_50_월요일시간대_5_유동인구_수',
 '남성연령대_50_월요일시간대_6_유동인구_수',
 '남성연령대_50_화요일시간대_1_유동인구_수',
 '남성연령대_50_화요일시간대_2_유동인구_수',
 '남성연령대_50_화요일시간대_3_유동인구_수',
 '남성연령대_50_화요일시간대_4_유동인구_수',
 '남성연령대_50_화요일시간대_5_유동인구_수',
 '남성연령대_50_화요일시간대_6_유동인구_수',
 '남성연령대_50_수요일시간대_1_유동인구_수',
 '남성연령대_50_수요일시간대_2_유동인구_수',
 '남성연령대_50_수요일시간대_3_유동인구_수',
 '남성연령대_50_수요일시간대_4_유동인구_수',
 '남성연령대_50_수요일시간대_5_유동인구_수',
 '남성연령대_50_수요일시간대_6_유동인구_수',
 '남성연령대_50_목요일시간대_1_유동인구_수',
 '남성연령대_50_목요일시간대_2_유동인구_수',
 '남성연령대_50_목요일시간대_3_유동인구_수',
 '남성연령대_50_목요일시간대_4_유동인구_수',
 '남성연령대_50_목요일시간대_5_유동인구_수',
 '남성연령대_50_목요일시간대_6_유동인구_수',
 '남성연령대_50_금요일시간대_1_유동인구_수',
 '남성연령대_50_금요일시간대_2_유동인구_수',
 '남성연령대_50_금요일시간대_3_유동인구_수',
 '남성연령대_50_금요일시간대_4_유동인구_수',
 '남성연령대_50_금요일시간대_5_유동인구_수',
 '남성연령대_50_금요일시간대_6_유동인구_수',
 '남성연령대_50_토요일시간대_1_유동인구_수',
 '남성연령대_50_토요일시간대_2_유동인구_수',
 '남성연령대_50_토요일시간대_3_유동인구_수',
 '남성연령대_50_토요일시간대_4_유동인구_수',
 '남성연령대_50_토요일시간대_5_유동인구_수',
 '남성연령대_50_토요일시간대_6_유동인구_수',
 '남성연령대_50_일요일시간대_1_유동인구_수',
 '남성연령대_50_일요일시간대_2_유동인구_수',
 '남성연령대_50_일요일시간대_3_유동인구_수',
 '남성연령대_50_일요일시간대_4_유동인구_수',
 '남성연령대_50_일요일시간대_5_유동인구_수',
 '남성연령대_50_일요일시간대_6_유동인구_수',
 '남성연령대_60_이상_월요일시간대_1_유동인구_수',
 '남성연령대_60_이상_월요일시간대_2_유동인구_수',
 '남성연령대_60_이상_월요일시간대_3_유동인구_수',
 '남성연령대_60_이상_월요일시간대_4_유동인구_수',
 '남성연령대_60_이상_월요일시간대_5_유동인구_수',
 '남성연령대_60_이상_월요일시간대_6_유동인구_수',
 '남성연령대_60_이상_화요일시간대_1_유동인구_수',
 '남성연령대_60_이상_화요일시간대_2_유동인구_수',
 '남성연령대_60_이상_화요일시간대_3_유동인구_수',
 '남성연령대_60_이상_화요일시간대_4_유동인구_수',
 '남성연령대_60_이상_화요일시간대_5_유동인구_수',
 '남성연령대_60_이상_화요일시간대_6_유동인구_수',
 '남성연령대_60_이상_수요일시간대_1_유동인구_수',
 '남성연령대_60_이상_수요일시간대_2_유동인구_수',
 '남성연령대_60_이상_수요일시간대_3_유동인구_수',
 '남성연령대_60_이상_수요일시간대_4_유동인구_수',
 '남성연령대_60_이상_수요일시간대_5_유동인구_수',
 '남성연령대_60_이상_수요일시간대_6_유동인구_수',
 '남성연령대_60_이상_목요일시간대_1_유동인구_수',
 '남성연령대_60_이상_목요일시간대_2_유동인구_수',
 '남성연령대_60_이상_목요일시간대_3_유동인구_수',
 '남성연령대_60_이상_목요일시간대_4_유동인구_수',
 '남성연령대_60_이상_목요일시간대_5_유동인구_수',
 '남성연령대_60_이상_목요일시간대_6_유동인구_수',
 '남성연령대_60_이상_금요일시간대_1_유동인구_수',
 '남성연령대_60_이상_금요일시간대_2_유동인구_수',
 '남성연령대_60_이상_금요일시간대_3_유동인구_수',
 '남성연령대_60_이상_금요일시간대_4_유동인구_수',
 '남성연령대_60_이상_금요일시간대_5_유동인구_수',
 '남성연령대_60_이상_금요일시간대_6_유동인구_수',
 '남성연령대_60_이상_토요일시간대_1_유동인구_수',
 '남성연령대_60_이상_토요일시간대_2_유동인구_수',
 '남성연령대_60_이상_토요일시간대_3_유동인구_수',
 '남성연령대_60_이상_토요일시간대_4_유동인구_수',
 '남성연령대_60_이상_토요일시간대_5_유동인구_수',
 '남성연령대_60_이상_토요일시간대_6_유동인구_수',
 '남성연령대_60_이상_일요일시간대_1_유동인구_수',
 '남성연령대_60_이상_일요일시간대_2_유동인구_수',
 '남성연령대_60_이상_일요일시간대_3_유동인구_수',
 '남성연령대_60_이상_일요일시간대_4_유동인구_수',
 '남성연령대_60_이상_일요일시간대_5_유동인구_수',
 '남성연령대_60_이상_일요일시간대_6_유동인구_수',
 '여성연령대_10_월요일시간대_1_유동인구_수',
 '여성연령대_10_월요일시간대_2_유동인구_수',
 '여성연령대_10_월요일시간대_3_유동인구_수',
 '여성연령대_10_월요일시간대_4_유동인구_수',
 '여성연령대_10_월요일시간대_5_유동인구_수',
 '여성연령대_10_월요일시간대_6_유동인구_수',
 '여성연령대_10_화요일시간대_1_유동인구_수',
 '여성연령대_10_화요일시간대_2_유동인구_수',
 '여성연령대_10_화요일시간대_3_유동인구_수',
 '여성연령대_10_화요일시간대_4_유동인구_수',
 '여성연령대_10_화요일시간대_5_유동인구_수',
 '여성연령대_10_화요일시간대_6_유동인구_수',
 '여성연령대_10_수요일시간대_1_유동인구_수',
 '여성연령대_10_수요일시간대_2_유동인구_수',
 '여성연령대_10_수요일시간대_3_유동인구_수',
 '여성연령대_10_수요일시간대_4_유동인구_수',
 '여성연령대_10_수요일시간대_5_유동인구_수',
 '여성연령대_10_수요일시간대_6_유동인구_수',
 '여성연령대_10_목요일시간대_1_유동인구_수',
 '여성연령대_10_목요일시간대_2_유동인구_수',
 '여성연령대_10_목요일시간대_3_유동인구_수',
 '여성연령대_10_목요일시간대_4_유동인구_수',
 '여성연령대_10_목요일시간대_5_유동인구_수',
 '여성연령대_10_목요일시간대_6_유동인구_수',
 '여성연령대_10_금요일시간대_1_유동인구_수',
 '여성연령대_10_금요일시간대_2_유동인구_수',
 '여성연령대_10_금요일시간대_3_유동인구_수',
 '여성연령대_10_금요일시간대_4_유동인구_수',
 '여성연령대_10_금요일시간대_5_유동인구_수',
 '여성연령대_10_금요일시간대_6_유동인구_수',
 '여성연령대_10_토요일시간대_1_유동인구_수',
 '여성연령대_10_토요일시간대_2_유동인구_수',
 '여성연령대_10_토요일시간대_3_유동인구_수',
 '여성연령대_10_토요일시간대_4_유동인구_수',
 '여성연령대_10_토요일시간대_5_유동인구_수',
 '여성연령대_10_토요일시간대_6_유동인구_수',
 '여성연령대_10_일요일시간대_1_유동인구_수',
 '여성연령대_10_일요일시간대_2_유동인구_수',
 '여성연령대_10_일요일시간대_3_유동인구_수',
 '여성연령대_10_일요일시간대_4_유동인구_수',
 '여성연령대_10_일요일시간대_5_유동인구_수',
 '여성연령대_10_일요일시간대_6_유동인구_수',
 '여성연령대_20_월요일시간대_1_유동인구_수',
 '여성연령대_20_월요일시간대_2_유동인구_수',
 '여성연령대_20_월요일시간대_3_유동인구_수',
 '여성연령대_20_월요일시간대_4_유동인구_수',
 '여성연령대_20_월요일시간대_5_유동인구_수',
 '여성연령대_20_월요일시간대_6_유동인구_수',
 '여성연령대_20_화요일시간대_1_유동인구_수',
 '여성연령대_20_화요일시간대_2_유동인구_수',
 '여성연령대_20_화요일시간대_3_유동인구_수',
 '여성연령대_20_화요일시간대_4_유동인구_수',
 '여성연령대_20_화요일시간대_5_유동인구_수',
 '여성연령대_20_화요일시간대_6_유동인구_수',
 '여성연령대_20_수요일시간대_1_유동인구_수',
 '여성연령대_20_수요일시간대_2_유동인구_수',
 '여성연령대_20_수요일시간대_3_유동인구_수',
 '여성연령대_20_수요일시간대_4_유동인구_수',
 '여성연령대_20_수요일시간대_5_유동인구_수',
 '여성연령대_20_수요일시간대_6_유동인구_수',
 '여성연령대_20_목요일시간대_1_유동인구_수',
 '여성연령대_20_목요일시간대_2_유동인구_수',
 '여성연령대_20_목요일시간대_3_유동인구_수',
 '여성연령대_20_목요일시간대_4_유동인구_수',
 '여성연령대_20_목요일시간대_5_유동인구_수',
 '여성연령대_20_목요일시간대_6_유동인구_수',
 '여성연령대_20_금요일시간대_1_유동인구_수',
 '여성연령대_20_금요일시간대_2_유동인구_수',
 '여성연령대_20_금요일시간대_3_유동인구_수',
 '여성연령대_20_금요일시간대_4_유동인구_수',
 '여성연령대_20_금요일시간대_5_유동인구_수',
 '여성연령대_20_금요일시간대_6_유동인구_수',
 '여성연령대_20_토요일시간대_1_유동인구_수',
 '여성연령대_20_토요일시간대_2_유동인구_수',
 '여성연령대_20_토요일시간대_3_유동인구_수',
 '여성연령대_20_토요일시간대_4_유동인구_수',
 '여성연령대_20_토요일시간대_5_유동인구_수',
 '여성연령대_20_토요일시간대_6_유동인구_수',
 '여성연령대_20_일요일시간대_1_유동인구_수',
 '여성연령대_20_일요일시간대_2_유동인구_수',
 '여성연령대_20_일요일시간대_3_유동인구_수',
 '여성연령대_20_일요일시간대_4_유동인구_수',
 '여성연령대_20_일요일시간대_5_유동인구_수',
 '여성연령대_20_일요일시간대_6_유동인구_수',
 '여성연령대_30_월요일시간대_1_유동인구_수',
 '여성연령대_30_월요일시간대_2_유동인구_수',
 '여성연령대_30_월요일시간대_3_유동인구_수',
 '여성연령대_30_월요일시간대_4_유동인구_수',
 '여성연령대_30_월요일시간대_5_유동인구_수',
 '여성연령대_30_월요일시간대_6_유동인구_수',
 '여성연령대_30_화요일시간대_1_유동인구_수',
 '여성연령대_30_화요일시간대_2_유동인구_수',
 '여성연령대_30_화요일시간대_3_유동인구_수',
 '여성연령대_30_화요일시간대_4_유동인구_수',
 '여성연령대_30_화요일시간대_5_유동인구_수',
 '여성연령대_30_화요일시간대_6_유동인구_수',
 '여성연령대_30_수요일시간대_1_유동인구_수',
 '여성연령대_30_수요일시간대_2_유동인구_수',
 '여성연령대_30_수요일시간대_3_유동인구_수',
 '여성연령대_30_수요일시간대_4_유동인구_수',
 '여성연령대_30_수요일시간대_5_유동인구_수',
 '여성연령대_30_수요일시간대_6_유동인구_수',
 '여성연령대_30_목요일시간대_1_유동인구_수',
 '여성연령대_30_목요일시간대_2_유동인구_수',
 '여성연령대_30_목요일시간대_3_유동인구_수',
 '여성연령대_30_목요일시간대_4_유동인구_수',
 '여성연령대_30_목요일시간대_5_유동인구_수',
 '여성연령대_30_목요일시간대_6_유동인구_수',
 '여성연령대_30_금요일시간대_1_유동인구_수',
 '여성연령대_30_금요일시간대_2_유동인구_수',
 '여성연령대_30_금요일시간대_3_유동인구_수',
 '여성연령대_30_금요일시간대_4_유동인구_수',
 '여성연령대_30_금요일시간대_5_유동인구_수',
 '여성연령대_30_금요일시간대_6_유동인구_수',
 '여성연령대_30_토요일시간대_1_유동인구_수',
 '여성연령대_30_토요일시간대_2_유동인구_수',
 '여성연령대_30_토요일시간대_3_유동인구_수',
 '여성연령대_30_토요일시간대_4_유동인구_수',
 '여성연령대_30_토요일시간대_5_유동인구_수',
 '여성연령대_30_토요일시간대_6_유동인구_수',
 '여성연령대_30_일요일시간대_1_유동인구_수',
 '여성연령대_30_일요일시간대_2_유동인구_수',
 '여성연령대_30_일요일시간대_3_유동인구_수',
 '여성연령대_30_일요일시간대_4_유동인구_수',
 '여성연령대_30_일요일시간대_5_유동인구_수',
 '여성연령대_30_일요일시간대_6_유동인구_수',
 '여성연령대_40_월요일시간대_1_유동인구_수',
 '여성연령대_40_월요일시간대_2_유동인구_수',
 '여성연령대_40_월요일시간대_3_유동인구_수',
 '여성연령대_40_월요일시간대_4_유동인구_수',
 '여성연령대_40_월요일시간대_5_유동인구_수',
 '여성연령대_40_월요일시간대_6_유동인구_수',
 '여성연령대_40_화요일시간대_1_유동인구_수',
 '여성연령대_40_화요일시간대_2_유동인구_수',
 '여성연령대_40_화요일시간대_3_유동인구_수',
 '여성연령대_40_화요일시간대_4_유동인구_수',
 '여성연령대_40_화요일시간대_5_유동인구_수',
 '여성연령대_40_화요일시간대_6_유동인구_수',
 '여성연령대_40_수요일시간대_1_유동인구_수',
 '여성연령대_40_수요일시간대_2_유동인구_수',
 '여성연령대_40_수요일시간대_3_유동인구_수',
 '여성연령대_40_수요일시간대_4_유동인구_수',
 '여성연령대_40_수요일시간대_5_유동인구_수',
 '여성연령대_40_수요일시간대_6_유동인구_수',
 '여성연령대_40_목요일시간대_1_유동인구_수',
 '여성연령대_40_목요일시간대_2_유동인구_수',
 '여성연령대_40_목요일시간대_3_유동인구_수',
 '여성연령대_40_목요일시간대_4_유동인구_수',
 '여성연령대_40_목요일시간대_5_유동인구_수',
 '여성연령대_40_목요일시간대_6_유동인구_수',
 '여성연령대_40_금요일시간대_1_유동인구_수',
 '여성연령대_40_금요일시간대_2_유동인구_수',
 '여성연령대_40_금요일시간대_3_유동인구_수',
 '여성연령대_40_금요일시간대_4_유동인구_수',
 '여성연령대_40_금요일시간대_5_유동인구_수',
 '여성연령대_40_금요일시간대_6_유동인구_수',
 '여성연령대_40_토요일시간대_1_유동인구_수',
 '여성연령대_40_토요일시간대_2_유동인구_수',
 '여성연령대_40_토요일시간대_3_유동인구_수',
 '여성연령대_40_토요일시간대_4_유동인구_수',
 '여성연령대_40_토요일시간대_5_유동인구_수',
 '여성연령대_40_토요일시간대_6_유동인구_수',
 '여성연령대_40_일요일시간대_1_유동인구_수',
 '여성연령대_40_일요일시간대_2_유동인구_수',
 '여성연령대_40_일요일시간대_3_유동인구_수',
 '여성연령대_40_일요일시간대_4_유동인구_수',
 '여성연령대_40_일요일시간대_5_유동인구_수',
 '여성연령대_40_일요일시간대_6_유동인구_수',
 '여성연령대_50_월요일시간대_1_유동인구_수',
 '여성연령대_50_월요일시간대_2_유동인구_수',
 '여성연령대_50_월요일시간대_3_유동인구_수',
 '여성연령대_50_월요일시간대_4_유동인구_수',
 '여성연령대_50_월요일시간대_5_유동인구_수',
 '여성연령대_50_월요일시간대_6_유동인구_수',
 '여성연령대_50_화요일시간대_1_유동인구_수',
 '여성연령대_50_화요일시간대_2_유동인구_수',
 '여성연령대_50_화요일시간대_3_유동인구_수',
 '여성연령대_50_화요일시간대_4_유동인구_수',
 '여성연령대_50_화요일시간대_5_유동인구_수',
 '여성연령대_50_화요일시간대_6_유동인구_수',
 '여성연령대_50_수요일시간대_1_유동인구_수',
 '여성연령대_50_수요일시간대_2_유동인구_수',
 '여성연령대_50_수요일시간대_3_유동인구_수',
 '여성연령대_50_수요일시간대_4_유동인구_수',
 '여성연령대_50_수요일시간대_5_유동인구_수',
 '여성연령대_50_수요일시간대_6_유동인구_수',
 '여성연령대_50_목요일시간대_1_유동인구_수',
 '여성연령대_50_목요일시간대_2_유동인구_수',
 '여성연령대_50_목요일시간대_3_유동인구_수',
 '여성연령대_50_목요일시간대_4_유동인구_수',
 '여성연령대_50_목요일시간대_5_유동인구_수',
 '여성연령대_50_목요일시간대_6_유동인구_수',
 '여성연령대_50_금요일시간대_1_유동인구_수',
 '여성연령대_50_금요일시간대_2_유동인구_수',
 '여성연령대_50_금요일시간대_3_유동인구_수',
 '여성연령대_50_금요일시간대_4_유동인구_수',
 '여성연령대_50_금요일시간대_5_유동인구_수',
 '여성연령대_50_금요일시간대_6_유동인구_수',
 '여성연령대_50_토요일시간대_1_유동인구_수',
 '여성연령대_50_토요일시간대_2_유동인구_수',
 '여성연령대_50_토요일시간대_3_유동인구_수',
 '여성연령대_50_토요일시간대_4_유동인구_수',
 '여성연령대_50_토요일시간대_5_유동인구_수',
 '여성연령대_50_토요일시간대_6_유동인구_수',
 '여성연령대_50_일요일시간대_1_유동인구_수',
 '여성연령대_50_일요일시간대_2_유동인구_수',
 '여성연령대_50_일요일시간대_3_유동인구_수',
 '여성연령대_50_일요일시간대_4_유동인구_수',
 '여성연령대_50_일요일시간대_5_유동인구_수',
 '여성연령대_50_일요일시간대_6_유동인구_수',
 '여성연령대_60_이상_월요일시간대_1_유동인구_수',
 '여성연령대_60_이상_월요일시간대_2_유동인구_수',
 '여성연령대_60_이상_월요일시간대_3_유동인구_수',
 '여성연령대_60_이상_월요일시간대_4_유동인구_수',
 '여성연령대_60_이상_월요일시간대_5_유동인구_수',
 '여성연령대_60_이상_월요일시간대_6_유동인구_수',
 '여성연령대_60_이상_화요일시간대_1_유동인구_수',
 '여성연령대_60_이상_화요일시간대_2_유동인구_수',
 '여성연령대_60_이상_화요일시간대_3_유동인구_수',
 '여성연령대_60_이상_화요일시간대_4_유동인구_수',
 '여성연령대_60_이상_화요일시간대_5_유동인구_수',
 '여성연령대_60_이상_화요일시간대_6_유동인구_수',
 '여성연령대_60_이상_수요일시간대_1_유동인구_수',
 '여성연령대_60_이상_수요일시간대_2_유동인구_수',
 '여성연령대_60_이상_수요일시간대_3_유동인구_수',
 '여성연령대_60_이상_수요일시간대_4_유동인구_수',
 '여성연령대_60_이상_수요일시간대_5_유동인구_수',
 '여성연령대_60_이상_수요일시간대_6_유동인구_수',
 '여성연령대_60_이상_목요일시간대_1_유동인구_수',
 '여성연령대_60_이상_목요일시간대_2_유동인구_수',
 '여성연령대_60_이상_목요일시간대_3_유동인구_수',
 '여성연령대_60_이상_목요일시간대_4_유동인구_수',
 '여성연령대_60_이상_목요일시간대_5_유동인구_수',
 '여성연령대_60_이상_목요일시간대_6_유동인구_수',
 '여성연령대_60_이상_금요일시간대_1_유동인구_수',
 '여성연령대_60_이상_금요일시간대_2_유동인구_수',
 '여성연령대_60_이상_금요일시간대_3_유동인구_수',
 '여성연령대_60_이상_금요일시간대_4_유동인구_수',
 '여성연령대_60_이상_금요일시간대_5_유동인구_수',
 '여성연령대_60_이상_금요일시간대_6_유동인구_수',
 '여성연령대_60_이상_토요일시간대_1_유동인구_수',
 '여성연령대_60_이상_토요일시간대_2_유동인구_수',
 '여성연령대_60_이상_토요일시간대_3_유동인구_수',
 '여성연령대_60_이상_토요일시간대_4_유동인구_수',
 '여성연령대_60_이상_토요일시간대_5_유동인구_수',
 '여성연령대_60_이상_토요일시간대_6_유동인구_수',
 '여성연령대_60_이상_일요일시간대_1_유동인구_수',
 '여성연령대_60_이상_일요일시간대_2_유동인구_수',
 '여성연령대_60_이상_일요일시간대_3_유동인구_수',
 '여성연령대_60_이상_일요일시간대_4_유동인구_수',
 '여성연령대_60_이상_일요일시간대_5_유동인구_수',
 '여성연령대_60_이상_일요일시간대_6_유동인구_수',
 'year_2015',
 'year_2016',
 'year_2017',
 'year_2018',
 'year_2019',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4',
 'sigungu_11140',
 'sigungu_11350',
 'dong_11140540',
 'dong_11140550',
 'dong_11140580',
 'dong_11140605',
 'dong_11140615',
 'dong_11140625',
 'dong_11140635',
 'dong_11140645',
 'dong_11140650',
 'dong_11140665',
 'dong_11140670',
 'dong_11350560',
 'dong_11350595',
 'dong_11350611',
 'dong_11350624',
 'dong_11350630',
 'dong_11350640',
 'dong_11350670']
#variable

Lasso Regression을 통해 mae, R-squared, RMSE를 출력하고 변수들의 계수를 출력하는 과정을 진행

In [44]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = variable
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7537845704249974,  0.6598234634488311,  0.9136377031306748, [-1.40972049e-04  0.00000000e+00 -1.03380073e-05 -1.46043069e-03
 -8.48412964e-04  1.38222514e-03  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  1.54860510e-03  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -1.27499560e-07  2.11003898e-11  9.71387089e-11 -6.34077869e-11
  1.03439904e-09 -1.67175332e-09  2.34117438e-10  1.20878722e-09
  7.45448824e-09 -9.73692853e-10 -7.38814884e-11  0.00000000e+00
 -0.00000000e+00  1.90529700e-05 -1.99330405e-04 -0.00000000e+00
  0.00000000e+00 -1.38810983e-04  4.24316188e-05  1.00998541e-04
  2.18230479e-04 -5.09598313e-04 -3.00541979e-05 -4.51081722e-04
 -0.00000000e+00 -2.65929069e-10 -6.11439005e-05 -6.57281831e-05
  1.55990059e-04  3.35560336e-04 -2.11609469e-04 -9.85426206e-06
  1.06138199e-04 -1.19714931e-04  3.10542971e-04  0.00000000e+00
  1.49141017e-04 -5.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.198818391012406, tolerance: 0.030091418941377436
  positive)


In [46]:
zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])

impovar : 중요변수모음

In [47]:
impovar

['총 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '여성연령대_60_이상_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_

계수가 0인 변수가 없을때까지 이 과정을 반복

In [48]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7537845704249974,  0.659601283456801,  0.923818747605906, [-1.36863531e-04 -2.00829033e-05 -1.46165843e-03 -8.20896453e-04
  1.31129753e-03  1.53618318e-03 -1.15809562e-07  2.09979657e-11
  9.79551388e-11 -8.25492660e-11  1.10209620e-09 -1.65070719e-09
  2.00239865e-10  1.09908282e-09  7.63121354e-09 -1.01514875e-09
  2.44135824e-11  3.51020089e-06 -2.46417096e-04 -1.36352517e-04
  5.15136177e-05  1.03399489e-04  2.66817461e-04 -5.05837889e-04
 -0.00000000e+00 -3.67281402e-04 -4.62358913e-10 -6.23185421e-05
 -6.56889753e-05  1.55881037e-04  4.38019817e-04 -2.14760075e-04
 -1.09048440e-05  1.08460831e-04 -1.31813407e-04  3.66331578e-04
  1.54567664e-04 -5.24282531e-04  2.77346081e-04 -2.32720721e-04
  9.26820651e-04 -6.22443874e-07  6.50383895e-04  2.11940400e-04
  1.58715986e-05  5.00529361e-05  9.36174003e-07 -3.63878280e-06
  7.63139053e-06  3.74747477e-06 -1.29577787e-05  1.08757546e-05
 -1.09001793e-05  4.34177097e-06 -2.47609638e-06 -1.76685458e-06
  1.64370491e-07 -6.11

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.060179537014385, tolerance: 0.030091418941377436
  positive)


In [49]:
zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_1

In [50]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7806398261685638,  0.6620700532190552,  0.9197992233926184, [-3.24563899e-04  0.00000000e+00 -1.77560410e-04 -1.26569003e-03
 -4.61349216e-05  1.27685171e-03 -0.00000000e+00  0.00000000e+00
  1.30103506e-04 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  3.42877322e-03  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -3.09703419e-04 -0.00000000e+00
 -5.65057361e-08 -9.50563461e-12  2.21346837e-11  1.33418496e-10
 -1.40147066e-09  6.91956034e-10 -2.26088888e-10  8.14911481e-09
 -8.18182746e-10 -1.61916256e-10 -0.00000000e+00  5.93370300e-05
 -6.90487472e-05 -2.30626137e-04  0.00000000e+00  0.00000000e+00
 -1.76231043e-04  0.00000000e+00  1.17750316e-04  2.07537702e-04
 -5.13219770e-04  0.00000000e+00 -1.99808669e-04 -0.00000000e+00
 -8.76036817e-10 -7.05771343e-05 -6.76563425e-05  1.60636503e-04
  1.54144877e-04 -2.13418924e-04 -1.98571817e-05  1.49355185e-04
 -1.30667523e-04  2.30138032e-04  0.00000000e+00  1.28196585e-04
 -5.61380243e-04  3.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.698876646099496, tolerance: 0.030091418941377436
  positive)


In [51]:
zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 40 상주인구 수',
 '여성연령대 40 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '남성연령대_10_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '숙박_시설_수',
 '공항_수',
 '철도_역_수',
 '버스_터미널_수',
 '지하철_역_수',
 '버스_정거장_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',

In [52]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7198427774224477,  0.6725379499941473,  0.8635958071555199, [-3.65029808e-04 -3.26970274e-04 -9.24285239e-04 -2.59410834e-05
  1.58176131e-03  0.00000000e+00  5.02637917e-03 -9.57413394e-04
 -6.71977421e-08  1.56121002e-11  1.29077351e-10 -2.20383341e-10
  7.99633525e-10 -1.29243583e-09  1.93806043e-10  6.78191335e-10
  6.01857811e-09 -7.47519117e-10 -8.01071299e-05 -1.18556611e-04
 -3.42899232e-04 -0.00000000e+00  2.23650098e-04  3.94305624e-04
  1.85814230e-04  2.18121035e-05 -0.00000000e+00 -1.13014647e-09
 -8.35239636e-05 -8.37973635e-05  1.99745796e-04  5.54724781e-05
 -2.64174847e-04 -7.78486828e-06  1.25329904e-04 -2.60946657e-04
  0.00000000e+00  5.11328881e-04 -5.80400808e-04  3.38107773e-04
  0.00000000e+00  1.37463902e-03 -1.94614689e-04  7.11909531e-04
  1.82976414e-04  4.71756267e-04  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.64272489e-06 -1.01186155e-07  2.66128095e-07  4.88437098e-05
 -1.35802655e-05  1.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23.000794180175767, tolerance: 0.030091418941377436
  positive)


In [53]:
zero_index = np.where(abs(variables) != 0)  #계수가 거의 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_10_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_

In [54]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7047645640038049,  0.6330264131180692,  0.8896234588347607, [-1.09154487e-04  3.82882794e-05 -5.87296312e-04 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.20345156e-04
 -1.29816521e-03 -1.53787939e-03  2.61889381e-03  6.82224064e-03
  1.65326733e-03  3.28441467e-04  0.00000000e+00  0.00000000e+00
  2.04362240e-03 -4.32296389e-03 -0.00000000e+00  2.19237368e-08
 -1.08635353e-11 -2.59446150e-10  1.14620149e-09 -2.06961065e-09
  1.15998085e-09  9.58462535e-09 -3.19782876e-10 -8.26280385e-10
 -9.65712797e-04  6.93281950e-05 -6.37617209e-05  4.96374029e-04
  0.00000000e+00 -1.62983660e-05 -5.86064712e-05  1.28717010e-04
 -7.00495062e-04 -0.00000000e+00 -8.92056962e-05  0.00000000e+00
 -1.00027476e-09 -3.25496193e-04 -2.42398629e-04 -0.00000000e+00
 -0.00000000e+00 -4.65157651e-04 -4.76039052e-04  6.25855480e-04
 -5.13106599e-04 -3.66752800e-04  1.92413331e-03 -2.15476731e-04
  7.25139356e-04  9.84983649e-04  2.70527644e-04  5.54384143e-04
  0.00000000e+00 -0.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 37.603252561882, tolerance: 0.030091418941377436
  positive)


In [55]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_10_직장_인구_수']

In [56]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.6752707326857336,  0.6405474286299486,  0.7958346589056318, [-4.00664328e-04  1.97669396e-04 -9.19101147e-04  0.00000000e+00
  2.97928405e-04 -2.00035270e-03 -1.33249596e-03  7.59475179e-04
  4.77705400e-03  1.65880409e-03  0.00000000e+00  4.18336661e-03
 -0.00000000e+00  6.36672317e-08  2.77976789e-11  1.89262269e-10
 -5.36847171e-10  2.39147079e-09 -2.46587582e-09  2.67115550e-10
  2.47705713e-10  8.68634107e-09 -3.68154774e-10 -2.17374291e-09
 -4.54965460e-04  6.32218375e-05  9.67654873e-05  0.00000000e+00
  0.00000000e+00  1.35625660e-05  3.26342197e-04 -7.50168502e-04
 -2.17274440e-04 -1.61179079e-09  3.24253175e-05 -1.88945367e-04
  4.60617171e-04  1.36313212e-03 -6.10700460e-04  2.57399124e-05
  1.39200318e-04 -3.77020596e-04  5.99008233e-05 -6.38975670e-04]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40.77831425005152, tolerance: 0.030091418941377436
  positive)


In [57]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수']

In [58]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.5869688772043051,  0.13394097521392212,  0.7665662304733893, [ 7.74402916e-05  0.00000000e+00  0.00000000e+00 -1.67055450e-03
  1.52429255e-03  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -3.77280400e-03  3.27863855e-03 -3.73956818e-03 -0.00000000e+00
  3.32492576e-04 -0.00000000e+00 -0.00000000e+00 -1.44157422e-04
  2.48636430e-04  3.79996027e-08  2.56497489e-10  7.50092316e-10
 -2.16573613e-09  1.79440009e-09 -2.45014007e-09  1.14644681e-09
  3.90594194e-10 -4.75493842e-09  0.00000000e+00  2.02864519e-04
 -4.53791218e-05 -1.37868127e-04  2.27031025e-04  0.00000000e+00
 -3.34805735e-04  8.75554275e-05 -2.14210450e-04  7.03945599e-04
 -1.34704584e-03 -4.33131160e-04  2.20152767e-04]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92.49691339874131, tolerance: 0.030091418941377436
  positive)


In [59]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 40 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수']

In [60]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.6542056697696021,  -0.32410851784285266,  0.8718840218483891, [-0.00000000e+00 -4.66174924e-04 -0.00000000e+00 -8.26544560e-04
 -0.00000000e+00  0.00000000e+00  1.92035357e-03 -0.00000000e+00
 -1.08501501e-03  2.67247541e-03 -5.33208106e-03  2.74980379e-03
  1.84854601e-09  5.21119981e-11  7.18769360e-10 -1.50042229e-09
 -1.51630410e-09 -2.50721437e-09  7.53961995e-09  7.01789849e-09
  3.20386459e-11 -2.65049986e-09  3.57585870e-04 -1.20178398e-04
 -9.51949222e-04  1.82705535e-03 -1.71973948e-05 -4.30149125e-04
 -8.65487518e-06]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105.28677345258183, tolerance: 0.030091418941377436
  positive)


In [61]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['남성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액']

In [62]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.9077181545928438,  -2.3548404575108517,  0.8848032011160871, [ 9.74061755e-04 -6.47938264e-04 -0.00000000e+00 -9.44015055e-04
 -0.00000000e+00 -0.00000000e+00  4.71938229e-03  3.17930216e-03
 -5.97389854e-03  4.25406818e-04 -0.00000000e+00  0.00000000e+00
 -4.65340726e-03 -3.50843292e-03  1.97407849e-03  1.32546450e-07
  1.60658060e-10  1.15619371e-09 -3.46857628e-09 -5.82806174e-09
  2.96913186e-10 -5.97577400e-12  1.16991007e-08 -4.31018726e-09]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 121.39533505913263, tolerance: 0.030091418941377436
  positive)


In [63]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액']

In [64]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.8542252096014714,  -5.9454826016683295,  0.9609511460867673, [-1.40431929e-04  1.20463874e-03 -1.34671501e-03  0.00000000e+00
  0.00000000e+00 -2.36574595e-03 -0.00000000e+00  2.89114350e-03
 -7.48010409e-03  4.96934135e-03 -0.00000000e+00 -0.00000000e+00
 -2.07976344e-03 -2.09094440e-03  3.70391373e-03  1.01015297e-07
 -3.78077549e-11  6.26709947e-10 -1.10209967e-09]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 128.78430803987683, tolerance: 0.030091418941377436
  positive)


In [65]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수']

In [66]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7885043737431584,  -8.113968074797645,  0.9516620983964903, [-0.          0.00057329 -0.0006207   0.00226661  0.00189092 -0.00338847
 -0.00160302  0.00417687 -0.00809507  0.00855416 -0.          0.
 -0.00483682 -0.        ]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.7764371401562755, tolerance: 0.030091418941377436
  positive)


In [67]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 40 상주인구 수']

In [68]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7864417631789751,  -19.44250056768166,  0.959284638115518, [ 2.02568597e-03 -2.59450467e-03 -1.03120290e-04  3.38371923e-05
  1.04939622e-03  0.00000000e+00 -5.83384375e-04  7.02240476e-04
 -0.00000000e+00  5.97729984e-04]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 51.63092006900693, tolerance: 0.030091418941377436
  positive)


In [69]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '남성연령대 10 상주인구 수']

In [70]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7864417631789751,  -19.99361209384312,  0.9641153953684346, [ 0.00084872  0.00173848 -0.00270131 -0.00088999 -0.00020225  0.
 -0.00145194 -0.        ]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.706767754162172, tolerance: 0.030091418941377436
  positive)


In [71]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 40 상주인구 수']

In [72]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.8419973187345307,  -25.175891749674065,  0.9836116826784, [ 0.00052882  0.00153173 -0.00231266 -0.00080851 -0.         -0.00034694]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2883689357648791, tolerance: 0.030091418941377436
  positive)


In [73]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 30 상주인구 수']

In [74]:
data_frame = rawdata1
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = impovar
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.8975528742900861,  -26.437495243283166,  0.9760966950211243, [ 0.00046581  0.00160215 -0.00243765 -0.00078459  0.00021311]


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.283872864473949, tolerance: 0.030091418941377436
  positive)


In [75]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']

계수가 0이 없을때까지 돌린 결과 위의 변수들이 영향력이 컸음을 알 수 있었으며 결과는 아래와 같다

계수: [ 0.00046581  0.00160215 -0.00243765 -0.00078459  0.00021311]

변수: ['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']  

##### #중구

In [86]:
rawdata_junggu = rawdata[rawdata['sigungu']==11140]
rawdata_junggu

,year,quarter,sigungu,dong,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
0,2015,1,11140,11140540,11152009251,657,376,281,35,55,...,294,251,112,3,0,17,85,101,53,4
1,2015,1,11140,11140550,99170342338,964,353,611,0,55,...,42,36,14,0,0,5,27,22,6,0
2,2015,1,11140,11140580,9889692667,2881,1575,1306,123,664,...,704,625,551,33,0,107,508,485,496,67
3,2015,1,11140,11140605,26217489487,6,0,6,0,0,...,143,131,89,7,0,59,183,152,62,3
4,2015,1,11140,11140615,8372678394,1750,1085,665,34,219,...,772,582,476,29,0,116,340,365,471,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2019,2,11140,11140635,6356292769,697,329,368,93,132,...,234,203,267,117,184,357,368,246,203,95
313,2019,2,11140,11140645,1597285285,4109,1938,2171,455,703,...,1210,1056,1324,690,1370,2014,1771,1147,1144,595
314,2019,2,11140,11140650,8769325536,3440,1717,1723,355,546,...,451,429,556,275,422,691,738,478,415,194
315,2019,2,11140,11140665,2521616514,3458,1678,1780,427,515,...,1133,1050,1438,758,1047,1744,1612,1077,1147,692


rawdata1_junggu : 중구의 데이터만 모아서 year, quarter 모두 dummy 변수로 만든 경우

In [87]:
rawdata1_junggu = pd.get_dummies(rawdata_junggu, columns=['year','quarter','sigungu','dong'], prefix=['year','quarter','sigungu','dong'])

In [88]:
variable = ['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_10_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '여성연령대_60_이상_직장_인구_수',
 '집객시설_수',
 '관공서_수',
 '은행_수',
 '종합병원_수',
 '일반_병원_수',
 '약국_수',
 '유치원_수',
 '초등학교_수',
 '중학교_수',
 '고등학교_수',
 '대학교_수',
 '백화점_수',
 '슈퍼마켓_수',
 '극장_수',
 '숙박_시설_수',
 '공항_수',
 '철도_역_수',
 '버스_터미널_수',
 '지하철_역_수',
 '버스_정거장_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대_6_유동인구_수',
 '월요일_유동인구_수',
 '화요일_유동인구_수',
 '수요일_유동인구_수',
 '목요일_유동인구_수',
 '금요일_유동인구_수',
 '토요일_유동인구_수',
 '일요일_유동인구_수',
 '남성연령대_10_월요일시간대_1_유동인구_수',
 '남성연령대_10_월요일시간대_2_유동인구_수',
 '남성연령대_10_월요일시간대_3_유동인구_수',
 '남성연령대_10_월요일시간대_4_유동인구_수',
 '남성연령대_10_월요일시간대_5_유동인구_수',
 '남성연령대_10_월요일시간대_6_유동인구_수',
 '남성연령대_10_화요일시간대_1_유동인구_수',
 '남성연령대_10_화요일시간대_2_유동인구_수',
 '남성연령대_10_화요일시간대_3_유동인구_수',
 '남성연령대_10_화요일시간대_4_유동인구_수',
 '남성연령대_10_화요일시간대_5_유동인구_수',
 '남성연령대_10_화요일시간대_6_유동인구_수',
 '남성연령대_10_수요일시간대_1_유동인구_수',
 '남성연령대_10_수요일시간대_2_유동인구_수',
 '남성연령대_10_수요일시간대_3_유동인구_수',
 '남성연령대_10_수요일시간대_4_유동인구_수',
 '남성연령대_10_수요일시간대_5_유동인구_수',
 '남성연령대_10_수요일시간대_6_유동인구_수',
 '남성연령대_10_목요일시간대_1_유동인구_수',
 '남성연령대_10_목요일시간대_2_유동인구_수',
 '남성연령대_10_목요일시간대_3_유동인구_수',
 '남성연령대_10_목요일시간대_4_유동인구_수',
 '남성연령대_10_목요일시간대_5_유동인구_수',
 '남성연령대_10_목요일시간대_6_유동인구_수',
 '남성연령대_10_금요일시간대_1_유동인구_수',
 '남성연령대_10_금요일시간대_2_유동인구_수',
 '남성연령대_10_금요일시간대_3_유동인구_수',
 '남성연령대_10_금요일시간대_4_유동인구_수',
 '남성연령대_10_금요일시간대_5_유동인구_수',
 '남성연령대_10_금요일시간대_6_유동인구_수',
 '남성연령대_10_토요일시간대_1_유동인구_수',
 '남성연령대_10_토요일시간대_2_유동인구_수',
 '남성연령대_10_토요일시간대_3_유동인구_수',
 '남성연령대_10_토요일시간대_4_유동인구_수',
 '남성연령대_10_토요일시간대_5_유동인구_수',
 '남성연령대_10_토요일시간대_6_유동인구_수',
 '남성연령대_10_일요일시간대_1_유동인구_수',
 '남성연령대_10_일요일시간대_2_유동인구_수',
 '남성연령대_10_일요일시간대_3_유동인구_수',
 '남성연령대_10_일요일시간대_4_유동인구_수',
 '남성연령대_10_일요일시간대_5_유동인구_수',
 '남성연령대_10_일요일시간대_6_유동인구_수',
 '남성연령대_20_월요일시간대_1_유동인구_수',
 '남성연령대_20_월요일시간대_2_유동인구_수',
 '남성연령대_20_월요일시간대_3_유동인구_수',
 '남성연령대_20_월요일시간대_4_유동인구_수',
 '남성연령대_20_월요일시간대_5_유동인구_수',
 '남성연령대_20_월요일시간대_6_유동인구_수',
 '남성연령대_20_화요일시간대_1_유동인구_수',
 '남성연령대_20_화요일시간대_2_유동인구_수',
 '남성연령대_20_화요일시간대_3_유동인구_수',
 '남성연령대_20_화요일시간대_4_유동인구_수',
 '남성연령대_20_화요일시간대_5_유동인구_수',
 '남성연령대_20_화요일시간대_6_유동인구_수',
 '남성연령대_20_수요일시간대_1_유동인구_수',
 '남성연령대_20_수요일시간대_2_유동인구_수',
 '남성연령대_20_수요일시간대_3_유동인구_수',
 '남성연령대_20_수요일시간대_4_유동인구_수',
 '남성연령대_20_수요일시간대_5_유동인구_수',
 '남성연령대_20_수요일시간대_6_유동인구_수',
 '남성연령대_20_목요일시간대_1_유동인구_수',
 '남성연령대_20_목요일시간대_2_유동인구_수',
 '남성연령대_20_목요일시간대_3_유동인구_수',
 '남성연령대_20_목요일시간대_4_유동인구_수',
 '남성연령대_20_목요일시간대_5_유동인구_수',
 '남성연령대_20_목요일시간대_6_유동인구_수',
 '남성연령대_20_금요일시간대_1_유동인구_수',
 '남성연령대_20_금요일시간대_2_유동인구_수',
 '남성연령대_20_금요일시간대_3_유동인구_수',
 '남성연령대_20_금요일시간대_4_유동인구_수',
 '남성연령대_20_금요일시간대_5_유동인구_수',
 '남성연령대_20_금요일시간대_6_유동인구_수',
 '남성연령대_20_토요일시간대_1_유동인구_수',
 '남성연령대_20_토요일시간대_2_유동인구_수',
 '남성연령대_20_토요일시간대_3_유동인구_수',
 '남성연령대_20_토요일시간대_4_유동인구_수',
 '남성연령대_20_토요일시간대_5_유동인구_수',
 '남성연령대_20_토요일시간대_6_유동인구_수',
 '남성연령대_20_일요일시간대_1_유동인구_수',
 '남성연령대_20_일요일시간대_2_유동인구_수',
 '남성연령대_20_일요일시간대_3_유동인구_수',
 '남성연령대_20_일요일시간대_4_유동인구_수',
 '남성연령대_20_일요일시간대_5_유동인구_수',
 '남성연령대_20_일요일시간대_6_유동인구_수',
 '남성연령대_30_월요일시간대_1_유동인구_수',
 '남성연령대_30_월요일시간대_2_유동인구_수',
 '남성연령대_30_월요일시간대_3_유동인구_수',
 '남성연령대_30_월요일시간대_4_유동인구_수',
 '남성연령대_30_월요일시간대_5_유동인구_수',
 '남성연령대_30_월요일시간대_6_유동인구_수',
 '남성연령대_30_화요일시간대_1_유동인구_수',
 '남성연령대_30_화요일시간대_2_유동인구_수',
 '남성연령대_30_화요일시간대_3_유동인구_수',
 '남성연령대_30_화요일시간대_4_유동인구_수',
 '남성연령대_30_화요일시간대_5_유동인구_수',
 '남성연령대_30_화요일시간대_6_유동인구_수',
 '남성연령대_30_수요일시간대_1_유동인구_수',
 '남성연령대_30_수요일시간대_2_유동인구_수',
 '남성연령대_30_수요일시간대_3_유동인구_수',
 '남성연령대_30_수요일시간대_4_유동인구_수',
 '남성연령대_30_수요일시간대_5_유동인구_수',
 '남성연령대_30_수요일시간대_6_유동인구_수',
 '남성연령대_30_목요일시간대_1_유동인구_수',
 '남성연령대_30_목요일시간대_2_유동인구_수',
 '남성연령대_30_목요일시간대_3_유동인구_수',
 '남성연령대_30_목요일시간대_4_유동인구_수',
 '남성연령대_30_목요일시간대_5_유동인구_수',
 '남성연령대_30_목요일시간대_6_유동인구_수',
 '남성연령대_30_금요일시간대_1_유동인구_수',
 '남성연령대_30_금요일시간대_2_유동인구_수',
 '남성연령대_30_금요일시간대_3_유동인구_수',
 '남성연령대_30_금요일시간대_4_유동인구_수',
 '남성연령대_30_금요일시간대_5_유동인구_수',
 '남성연령대_30_금요일시간대_6_유동인구_수',
 '남성연령대_30_토요일시간대_1_유동인구_수',
 '남성연령대_30_토요일시간대_2_유동인구_수',
 '남성연령대_30_토요일시간대_3_유동인구_수',
 '남성연령대_30_토요일시간대_4_유동인구_수',
 '남성연령대_30_토요일시간대_5_유동인구_수',
 '남성연령대_30_토요일시간대_6_유동인구_수',
 '남성연령대_30_일요일시간대_1_유동인구_수',
 '남성연령대_30_일요일시간대_2_유동인구_수',
 '남성연령대_30_일요일시간대_3_유동인구_수',
 '남성연령대_30_일요일시간대_4_유동인구_수',
 '남성연령대_30_일요일시간대_5_유동인구_수',
 '남성연령대_30_일요일시간대_6_유동인구_수',
 '남성연령대_40_월요일시간대_1_유동인구_수',
 '남성연령대_40_월요일시간대_2_유동인구_수',
 '남성연령대_40_월요일시간대_3_유동인구_수',
 '남성연령대_40_월요일시간대_4_유동인구_수',
 '남성연령대_40_월요일시간대_5_유동인구_수',
 '남성연령대_40_월요일시간대_6_유동인구_수',
 '남성연령대_40_화요일시간대_1_유동인구_수',
 '남성연령대_40_화요일시간대_2_유동인구_수',
 '남성연령대_40_화요일시간대_3_유동인구_수',
 '남성연령대_40_화요일시간대_4_유동인구_수',
 '남성연령대_40_화요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_6_유동인구_수',
 '남성연령대_40_수요일시간대_1_유동인구_수',
 '남성연령대_40_수요일시간대_2_유동인구_수',
 '남성연령대_40_수요일시간대_3_유동인구_수',
 '남성연령대_40_수요일시간대_4_유동인구_수',
 '남성연령대_40_수요일시간대_5_유동인구_수',
 '남성연령대_40_수요일시간대_6_유동인구_수',
 '남성연령대_40_목요일시간대_1_유동인구_수',
 '남성연령대_40_목요일시간대_2_유동인구_수',
 '남성연령대_40_목요일시간대_3_유동인구_수',
 '남성연령대_40_목요일시간대_4_유동인구_수',
 '남성연령대_40_목요일시간대_5_유동인구_수',
 '남성연령대_40_목요일시간대_6_유동인구_수',
 '남성연령대_40_금요일시간대_1_유동인구_수',
 '남성연령대_40_금요일시간대_2_유동인구_수',
 '남성연령대_40_금요일시간대_3_유동인구_수',
 '남성연령대_40_금요일시간대_4_유동인구_수',
 '남성연령대_40_금요일시간대_5_유동인구_수',
 '남성연령대_40_금요일시간대_6_유동인구_수',
 '남성연령대_40_토요일시간대_1_유동인구_수',
 '남성연령대_40_토요일시간대_2_유동인구_수',
 '남성연령대_40_토요일시간대_3_유동인구_수',
 '남성연령대_40_토요일시간대_4_유동인구_수',
 '남성연령대_40_토요일시간대_5_유동인구_수',
 '남성연령대_40_토요일시간대_6_유동인구_수',
 '남성연령대_40_일요일시간대_1_유동인구_수',
 '남성연령대_40_일요일시간대_2_유동인구_수',
 '남성연령대_40_일요일시간대_3_유동인구_수',
 '남성연령대_40_일요일시간대_4_유동인구_수',
 '남성연령대_40_일요일시간대_5_유동인구_수',
 '남성연령대_40_일요일시간대_6_유동인구_수',
 '남성연령대_50_월요일시간대_1_유동인구_수',
 '남성연령대_50_월요일시간대_2_유동인구_수',
 '남성연령대_50_월요일시간대_3_유동인구_수',
 '남성연령대_50_월요일시간대_4_유동인구_수',
 '남성연령대_50_월요일시간대_5_유동인구_수',
 '남성연령대_50_월요일시간대_6_유동인구_수',
 '남성연령대_50_화요일시간대_1_유동인구_수',
 '남성연령대_50_화요일시간대_2_유동인구_수',
 '남성연령대_50_화요일시간대_3_유동인구_수',
 '남성연령대_50_화요일시간대_4_유동인구_수',
 '남성연령대_50_화요일시간대_5_유동인구_수',
 '남성연령대_50_화요일시간대_6_유동인구_수',
 '남성연령대_50_수요일시간대_1_유동인구_수',
 '남성연령대_50_수요일시간대_2_유동인구_수',
 '남성연령대_50_수요일시간대_3_유동인구_수',
 '남성연령대_50_수요일시간대_4_유동인구_수',
 '남성연령대_50_수요일시간대_5_유동인구_수',
 '남성연령대_50_수요일시간대_6_유동인구_수',
 '남성연령대_50_목요일시간대_1_유동인구_수',
 '남성연령대_50_목요일시간대_2_유동인구_수',
 '남성연령대_50_목요일시간대_3_유동인구_수',
 '남성연령대_50_목요일시간대_4_유동인구_수',
 '남성연령대_50_목요일시간대_5_유동인구_수',
 '남성연령대_50_목요일시간대_6_유동인구_수',
 '남성연령대_50_금요일시간대_1_유동인구_수',
 '남성연령대_50_금요일시간대_2_유동인구_수',
 '남성연령대_50_금요일시간대_3_유동인구_수',
 '남성연령대_50_금요일시간대_4_유동인구_수',
 '남성연령대_50_금요일시간대_5_유동인구_수',
 '남성연령대_50_금요일시간대_6_유동인구_수',
 '남성연령대_50_토요일시간대_1_유동인구_수',
 '남성연령대_50_토요일시간대_2_유동인구_수',
 '남성연령대_50_토요일시간대_3_유동인구_수',
 '남성연령대_50_토요일시간대_4_유동인구_수',
 '남성연령대_50_토요일시간대_5_유동인구_수',
 '남성연령대_50_토요일시간대_6_유동인구_수',
 '남성연령대_50_일요일시간대_1_유동인구_수',
 '남성연령대_50_일요일시간대_2_유동인구_수',
 '남성연령대_50_일요일시간대_3_유동인구_수',
 '남성연령대_50_일요일시간대_4_유동인구_수',
 '남성연령대_50_일요일시간대_5_유동인구_수',
 '남성연령대_50_일요일시간대_6_유동인구_수',
 '남성연령대_60_이상_월요일시간대_1_유동인구_수',
 '남성연령대_60_이상_월요일시간대_2_유동인구_수',
 '남성연령대_60_이상_월요일시간대_3_유동인구_수',
 '남성연령대_60_이상_월요일시간대_4_유동인구_수',
 '남성연령대_60_이상_월요일시간대_5_유동인구_수',
 '남성연령대_60_이상_월요일시간대_6_유동인구_수',
 '남성연령대_60_이상_화요일시간대_1_유동인구_수',
 '남성연령대_60_이상_화요일시간대_2_유동인구_수',
 '남성연령대_60_이상_화요일시간대_3_유동인구_수',
 '남성연령대_60_이상_화요일시간대_4_유동인구_수',
 '남성연령대_60_이상_화요일시간대_5_유동인구_수',
 '남성연령대_60_이상_화요일시간대_6_유동인구_수',
 '남성연령대_60_이상_수요일시간대_1_유동인구_수',
 '남성연령대_60_이상_수요일시간대_2_유동인구_수',
 '남성연령대_60_이상_수요일시간대_3_유동인구_수',
 '남성연령대_60_이상_수요일시간대_4_유동인구_수',
 '남성연령대_60_이상_수요일시간대_5_유동인구_수',
 '남성연령대_60_이상_수요일시간대_6_유동인구_수',
 '남성연령대_60_이상_목요일시간대_1_유동인구_수',
 '남성연령대_60_이상_목요일시간대_2_유동인구_수',
 '남성연령대_60_이상_목요일시간대_3_유동인구_수',
 '남성연령대_60_이상_목요일시간대_4_유동인구_수',
 '남성연령대_60_이상_목요일시간대_5_유동인구_수',
 '남성연령대_60_이상_목요일시간대_6_유동인구_수',
 '남성연령대_60_이상_금요일시간대_1_유동인구_수',
 '남성연령대_60_이상_금요일시간대_2_유동인구_수',
 '남성연령대_60_이상_금요일시간대_3_유동인구_수',
 '남성연령대_60_이상_금요일시간대_4_유동인구_수',
 '남성연령대_60_이상_금요일시간대_5_유동인구_수',
 '남성연령대_60_이상_금요일시간대_6_유동인구_수',
 '남성연령대_60_이상_토요일시간대_1_유동인구_수',
 '남성연령대_60_이상_토요일시간대_2_유동인구_수',
 '남성연령대_60_이상_토요일시간대_3_유동인구_수',
 '남성연령대_60_이상_토요일시간대_4_유동인구_수',
 '남성연령대_60_이상_토요일시간대_5_유동인구_수',
 '남성연령대_60_이상_토요일시간대_6_유동인구_수',
 '남성연령대_60_이상_일요일시간대_1_유동인구_수',
 '남성연령대_60_이상_일요일시간대_2_유동인구_수',
 '남성연령대_60_이상_일요일시간대_3_유동인구_수',
 '남성연령대_60_이상_일요일시간대_4_유동인구_수',
 '남성연령대_60_이상_일요일시간대_5_유동인구_수',
 '남성연령대_60_이상_일요일시간대_6_유동인구_수',
 '여성연령대_10_월요일시간대_1_유동인구_수',
 '여성연령대_10_월요일시간대_2_유동인구_수',
 '여성연령대_10_월요일시간대_3_유동인구_수',
 '여성연령대_10_월요일시간대_4_유동인구_수',
 '여성연령대_10_월요일시간대_5_유동인구_수',
 '여성연령대_10_월요일시간대_6_유동인구_수',
 '여성연령대_10_화요일시간대_1_유동인구_수',
 '여성연령대_10_화요일시간대_2_유동인구_수',
 '여성연령대_10_화요일시간대_3_유동인구_수',
 '여성연령대_10_화요일시간대_4_유동인구_수',
 '여성연령대_10_화요일시간대_5_유동인구_수',
 '여성연령대_10_화요일시간대_6_유동인구_수',
 '여성연령대_10_수요일시간대_1_유동인구_수',
 '여성연령대_10_수요일시간대_2_유동인구_수',
 '여성연령대_10_수요일시간대_3_유동인구_수',
 '여성연령대_10_수요일시간대_4_유동인구_수',
 '여성연령대_10_수요일시간대_5_유동인구_수',
 '여성연령대_10_수요일시간대_6_유동인구_수',
 '여성연령대_10_목요일시간대_1_유동인구_수',
 '여성연령대_10_목요일시간대_2_유동인구_수',
 '여성연령대_10_목요일시간대_3_유동인구_수',
 '여성연령대_10_목요일시간대_4_유동인구_수',
 '여성연령대_10_목요일시간대_5_유동인구_수',
 '여성연령대_10_목요일시간대_6_유동인구_수',
 '여성연령대_10_금요일시간대_1_유동인구_수',
 '여성연령대_10_금요일시간대_2_유동인구_수',
 '여성연령대_10_금요일시간대_3_유동인구_수',
 '여성연령대_10_금요일시간대_4_유동인구_수',
 '여성연령대_10_금요일시간대_5_유동인구_수',
 '여성연령대_10_금요일시간대_6_유동인구_수',
 '여성연령대_10_토요일시간대_1_유동인구_수',
 '여성연령대_10_토요일시간대_2_유동인구_수',
 '여성연령대_10_토요일시간대_3_유동인구_수',
 '여성연령대_10_토요일시간대_4_유동인구_수',
 '여성연령대_10_토요일시간대_5_유동인구_수',
 '여성연령대_10_토요일시간대_6_유동인구_수',
 '여성연령대_10_일요일시간대_1_유동인구_수',
 '여성연령대_10_일요일시간대_2_유동인구_수',
 '여성연령대_10_일요일시간대_3_유동인구_수',
 '여성연령대_10_일요일시간대_4_유동인구_수',
 '여성연령대_10_일요일시간대_5_유동인구_수',
 '여성연령대_10_일요일시간대_6_유동인구_수',
 '여성연령대_20_월요일시간대_1_유동인구_수',
 '여성연령대_20_월요일시간대_2_유동인구_수',
 '여성연령대_20_월요일시간대_3_유동인구_수',
 '여성연령대_20_월요일시간대_4_유동인구_수',
 '여성연령대_20_월요일시간대_5_유동인구_수',
 '여성연령대_20_월요일시간대_6_유동인구_수',
 '여성연령대_20_화요일시간대_1_유동인구_수',
 '여성연령대_20_화요일시간대_2_유동인구_수',
 '여성연령대_20_화요일시간대_3_유동인구_수',
 '여성연령대_20_화요일시간대_4_유동인구_수',
 '여성연령대_20_화요일시간대_5_유동인구_수',
 '여성연령대_20_화요일시간대_6_유동인구_수',
 '여성연령대_20_수요일시간대_1_유동인구_수',
 '여성연령대_20_수요일시간대_2_유동인구_수',
 '여성연령대_20_수요일시간대_3_유동인구_수',
 '여성연령대_20_수요일시간대_4_유동인구_수',
 '여성연령대_20_수요일시간대_5_유동인구_수',
 '여성연령대_20_수요일시간대_6_유동인구_수',
 '여성연령대_20_목요일시간대_1_유동인구_수',
 '여성연령대_20_목요일시간대_2_유동인구_수',
 '여성연령대_20_목요일시간대_3_유동인구_수',
 '여성연령대_20_목요일시간대_4_유동인구_수',
 '여성연령대_20_목요일시간대_5_유동인구_수',
 '여성연령대_20_목요일시간대_6_유동인구_수',
 '여성연령대_20_금요일시간대_1_유동인구_수',
 '여성연령대_20_금요일시간대_2_유동인구_수',
 '여성연령대_20_금요일시간대_3_유동인구_수',
 '여성연령대_20_금요일시간대_4_유동인구_수',
 '여성연령대_20_금요일시간대_5_유동인구_수',
 '여성연령대_20_금요일시간대_6_유동인구_수',
 '여성연령대_20_토요일시간대_1_유동인구_수',
 '여성연령대_20_토요일시간대_2_유동인구_수',
 '여성연령대_20_토요일시간대_3_유동인구_수',
 '여성연령대_20_토요일시간대_4_유동인구_수',
 '여성연령대_20_토요일시간대_5_유동인구_수',
 '여성연령대_20_토요일시간대_6_유동인구_수',
 '여성연령대_20_일요일시간대_1_유동인구_수',
 '여성연령대_20_일요일시간대_2_유동인구_수',
 '여성연령대_20_일요일시간대_3_유동인구_수',
 '여성연령대_20_일요일시간대_4_유동인구_수',
 '여성연령대_20_일요일시간대_5_유동인구_수',
 '여성연령대_20_일요일시간대_6_유동인구_수',
 '여성연령대_30_월요일시간대_1_유동인구_수',
 '여성연령대_30_월요일시간대_2_유동인구_수',
 '여성연령대_30_월요일시간대_3_유동인구_수',
 '여성연령대_30_월요일시간대_4_유동인구_수',
 '여성연령대_30_월요일시간대_5_유동인구_수',
 '여성연령대_30_월요일시간대_6_유동인구_수',
 '여성연령대_30_화요일시간대_1_유동인구_수',
 '여성연령대_30_화요일시간대_2_유동인구_수',
 '여성연령대_30_화요일시간대_3_유동인구_수',
 '여성연령대_30_화요일시간대_4_유동인구_수',
 '여성연령대_30_화요일시간대_5_유동인구_수',
 '여성연령대_30_화요일시간대_6_유동인구_수',
 '여성연령대_30_수요일시간대_1_유동인구_수',
 '여성연령대_30_수요일시간대_2_유동인구_수',
 '여성연령대_30_수요일시간대_3_유동인구_수',
 '여성연령대_30_수요일시간대_4_유동인구_수',
 '여성연령대_30_수요일시간대_5_유동인구_수',
 '여성연령대_30_수요일시간대_6_유동인구_수',
 '여성연령대_30_목요일시간대_1_유동인구_수',
 '여성연령대_30_목요일시간대_2_유동인구_수',
 '여성연령대_30_목요일시간대_3_유동인구_수',
 '여성연령대_30_목요일시간대_4_유동인구_수',
 '여성연령대_30_목요일시간대_5_유동인구_수',
 '여성연령대_30_목요일시간대_6_유동인구_수',
 '여성연령대_30_금요일시간대_1_유동인구_수',
 '여성연령대_30_금요일시간대_2_유동인구_수',
 '여성연령대_30_금요일시간대_3_유동인구_수',
 '여성연령대_30_금요일시간대_4_유동인구_수',
 '여성연령대_30_금요일시간대_5_유동인구_수',
 '여성연령대_30_금요일시간대_6_유동인구_수',
 '여성연령대_30_토요일시간대_1_유동인구_수',
 '여성연령대_30_토요일시간대_2_유동인구_수',
 '여성연령대_30_토요일시간대_3_유동인구_수',
 '여성연령대_30_토요일시간대_4_유동인구_수',
 '여성연령대_30_토요일시간대_5_유동인구_수',
 '여성연령대_30_토요일시간대_6_유동인구_수',
 '여성연령대_30_일요일시간대_1_유동인구_수',
 '여성연령대_30_일요일시간대_2_유동인구_수',
 '여성연령대_30_일요일시간대_3_유동인구_수',
 '여성연령대_30_일요일시간대_4_유동인구_수',
 '여성연령대_30_일요일시간대_5_유동인구_수',
 '여성연령대_30_일요일시간대_6_유동인구_수',
 '여성연령대_40_월요일시간대_1_유동인구_수',
 '여성연령대_40_월요일시간대_2_유동인구_수',
 '여성연령대_40_월요일시간대_3_유동인구_수',
 '여성연령대_40_월요일시간대_4_유동인구_수',
 '여성연령대_40_월요일시간대_5_유동인구_수',
 '여성연령대_40_월요일시간대_6_유동인구_수',
 '여성연령대_40_화요일시간대_1_유동인구_수',
 '여성연령대_40_화요일시간대_2_유동인구_수',
 '여성연령대_40_화요일시간대_3_유동인구_수',
 '여성연령대_40_화요일시간대_4_유동인구_수',
 '여성연령대_40_화요일시간대_5_유동인구_수',
 '여성연령대_40_화요일시간대_6_유동인구_수',
 '여성연령대_40_수요일시간대_1_유동인구_수',
 '여성연령대_40_수요일시간대_2_유동인구_수',
 '여성연령대_40_수요일시간대_3_유동인구_수',
 '여성연령대_40_수요일시간대_4_유동인구_수',
 '여성연령대_40_수요일시간대_5_유동인구_수',
 '여성연령대_40_수요일시간대_6_유동인구_수',
 '여성연령대_40_목요일시간대_1_유동인구_수',
 '여성연령대_40_목요일시간대_2_유동인구_수',
 '여성연령대_40_목요일시간대_3_유동인구_수',
 '여성연령대_40_목요일시간대_4_유동인구_수',
 '여성연령대_40_목요일시간대_5_유동인구_수',
 '여성연령대_40_목요일시간대_6_유동인구_수',
 '여성연령대_40_금요일시간대_1_유동인구_수',
 '여성연령대_40_금요일시간대_2_유동인구_수',
 '여성연령대_40_금요일시간대_3_유동인구_수',
 '여성연령대_40_금요일시간대_4_유동인구_수',
 '여성연령대_40_금요일시간대_5_유동인구_수',
 '여성연령대_40_금요일시간대_6_유동인구_수',
 '여성연령대_40_토요일시간대_1_유동인구_수',
 '여성연령대_40_토요일시간대_2_유동인구_수',
 '여성연령대_40_토요일시간대_3_유동인구_수',
 '여성연령대_40_토요일시간대_4_유동인구_수',
 '여성연령대_40_토요일시간대_5_유동인구_수',
 '여성연령대_40_토요일시간대_6_유동인구_수',
 '여성연령대_40_일요일시간대_1_유동인구_수',
 '여성연령대_40_일요일시간대_2_유동인구_수',
 '여성연령대_40_일요일시간대_3_유동인구_수',
 '여성연령대_40_일요일시간대_4_유동인구_수',
 '여성연령대_40_일요일시간대_5_유동인구_수',
 '여성연령대_40_일요일시간대_6_유동인구_수',
 '여성연령대_50_월요일시간대_1_유동인구_수',
 '여성연령대_50_월요일시간대_2_유동인구_수',
 '여성연령대_50_월요일시간대_3_유동인구_수',
 '여성연령대_50_월요일시간대_4_유동인구_수',
 '여성연령대_50_월요일시간대_5_유동인구_수',
 '여성연령대_50_월요일시간대_6_유동인구_수',
 '여성연령대_50_화요일시간대_1_유동인구_수',
 '여성연령대_50_화요일시간대_2_유동인구_수',
 '여성연령대_50_화요일시간대_3_유동인구_수',
 '여성연령대_50_화요일시간대_4_유동인구_수',
 '여성연령대_50_화요일시간대_5_유동인구_수',
 '여성연령대_50_화요일시간대_6_유동인구_수',
 '여성연령대_50_수요일시간대_1_유동인구_수',
 '여성연령대_50_수요일시간대_2_유동인구_수',
 '여성연령대_50_수요일시간대_3_유동인구_수',
 '여성연령대_50_수요일시간대_4_유동인구_수',
 '여성연령대_50_수요일시간대_5_유동인구_수',
 '여성연령대_50_수요일시간대_6_유동인구_수',
 '여성연령대_50_목요일시간대_1_유동인구_수',
 '여성연령대_50_목요일시간대_2_유동인구_수',
 '여성연령대_50_목요일시간대_3_유동인구_수',
 '여성연령대_50_목요일시간대_4_유동인구_수',
 '여성연령대_50_목요일시간대_5_유동인구_수',
 '여성연령대_50_목요일시간대_6_유동인구_수',
 '여성연령대_50_금요일시간대_1_유동인구_수',
 '여성연령대_50_금요일시간대_2_유동인구_수',
 '여성연령대_50_금요일시간대_3_유동인구_수',
 '여성연령대_50_금요일시간대_4_유동인구_수',
 '여성연령대_50_금요일시간대_5_유동인구_수',
 '여성연령대_50_금요일시간대_6_유동인구_수',
 '여성연령대_50_토요일시간대_1_유동인구_수',
 '여성연령대_50_토요일시간대_2_유동인구_수',
 '여성연령대_50_토요일시간대_3_유동인구_수',
 '여성연령대_50_토요일시간대_4_유동인구_수',
 '여성연령대_50_토요일시간대_5_유동인구_수',
 '여성연령대_50_토요일시간대_6_유동인구_수',
 '여성연령대_50_일요일시간대_1_유동인구_수',
 '여성연령대_50_일요일시간대_2_유동인구_수',
 '여성연령대_50_일요일시간대_3_유동인구_수',
 '여성연령대_50_일요일시간대_4_유동인구_수',
 '여성연령대_50_일요일시간대_5_유동인구_수',
 '여성연령대_50_일요일시간대_6_유동인구_수',
 '여성연령대_60_이상_월요일시간대_1_유동인구_수',
 '여성연령대_60_이상_월요일시간대_2_유동인구_수',
 '여성연령대_60_이상_월요일시간대_3_유동인구_수',
 '여성연령대_60_이상_월요일시간대_4_유동인구_수',
 '여성연령대_60_이상_월요일시간대_5_유동인구_수',
 '여성연령대_60_이상_월요일시간대_6_유동인구_수',
 '여성연령대_60_이상_화요일시간대_1_유동인구_수',
 '여성연령대_60_이상_화요일시간대_2_유동인구_수',
 '여성연령대_60_이상_화요일시간대_3_유동인구_수',
 '여성연령대_60_이상_화요일시간대_4_유동인구_수',
 '여성연령대_60_이상_화요일시간대_5_유동인구_수',
 '여성연령대_60_이상_화요일시간대_6_유동인구_수',
 '여성연령대_60_이상_수요일시간대_1_유동인구_수',
 '여성연령대_60_이상_수요일시간대_2_유동인구_수',
 '여성연령대_60_이상_수요일시간대_3_유동인구_수',
 '여성연령대_60_이상_수요일시간대_4_유동인구_수',
 '여성연령대_60_이상_수요일시간대_5_유동인구_수',
 '여성연령대_60_이상_수요일시간대_6_유동인구_수',
 '여성연령대_60_이상_목요일시간대_1_유동인구_수',
 '여성연령대_60_이상_목요일시간대_2_유동인구_수',
 '여성연령대_60_이상_목요일시간대_3_유동인구_수',
 '여성연령대_60_이상_목요일시간대_4_유동인구_수',
 '여성연령대_60_이상_목요일시간대_5_유동인구_수',
 '여성연령대_60_이상_목요일시간대_6_유동인구_수',
 '여성연령대_60_이상_금요일시간대_1_유동인구_수',
 '여성연령대_60_이상_금요일시간대_2_유동인구_수',
 '여성연령대_60_이상_금요일시간대_3_유동인구_수',
 '여성연령대_60_이상_금요일시간대_4_유동인구_수',
 '여성연령대_60_이상_금요일시간대_5_유동인구_수',
 '여성연령대_60_이상_금요일시간대_6_유동인구_수',
 '여성연령대_60_이상_토요일시간대_1_유동인구_수',
 '여성연령대_60_이상_토요일시간대_2_유동인구_수',
 '여성연령대_60_이상_토요일시간대_3_유동인구_수',
 '여성연령대_60_이상_토요일시간대_4_유동인구_수',
 '여성연령대_60_이상_토요일시간대_5_유동인구_수',
 '여성연령대_60_이상_토요일시간대_6_유동인구_수',
 '여성연령대_60_이상_일요일시간대_1_유동인구_수',
 '여성연령대_60_이상_일요일시간대_2_유동인구_수',
 '여성연령대_60_이상_일요일시간대_3_유동인구_수',
 '여성연령대_60_이상_일요일시간대_4_유동인구_수',
 '여성연령대_60_이상_일요일시간대_5_유동인구_수',
 '여성연령대_60_이상_일요일시간대_6_유동인구_수',
 'year_2015',
 'year_2016',
 'year_2017',
 'year_2018',
 'year_2019',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4',
 'sigungu_11140',
 'dong_11140540',
 'dong_11140550',
 'dong_11140580',
 'dong_11140605',
 'dong_11140615',
 'dong_11140625',
 'dong_11140635',
 'dong_11140645',
 'dong_11140650',
 'dong_11140665',
 'dong_11140670']


In [89]:
rawdata1_junggu["Logamt"] = np.log1p(rawdata1_junggu["amt"])
rawdata1_junggu.head()

,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,dong_11140580,dong_11140605,dong_11140615,dong_11140625,dong_11140635,dong_11140645,dong_11140650,dong_11140665,dong_11140670,Logamt
0,11152009251,657,376,281,35,55,80,119,194,174,...,0,0,0,0,0,0,0,0,0,23.134886
1,99170342338,964,353,611,0,55,171,244,317,177,...,0,0,0,0,0,0,0,0,0,25.320105
2,9889692667,2881,1575,1306,123,664,615,600,611,268,...,1,0,0,0,0,0,0,0,0,23.014759
3,26217489487,6,0,6,0,0,0,0,6,0,...,0,1,0,0,0,0,0,0,0,23.989693
4,8372678394,1750,1085,665,34,219,224,610,606,57,...,0,0,1,0,0,0,0,0,0,22.848240


In [90]:
rawdata1_junggu = rawdata1_junggu.drop(['amt'],axis=1)
rawdata1_junggu.head()

,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,남성연령대 10 상주인구 수,...,dong_11140580,dong_11140605,dong_11140615,dong_11140625,dong_11140635,dong_11140645,dong_11140650,dong_11140665,dong_11140670,Logamt
0,657,376,281,35,55,80,119,194,174,16,...,0,0,0,0,0,0,0,0,0,23.134886
1,964,353,611,0,55,171,244,317,177,0,...,0,0,0,0,0,0,0,0,0,25.320105
2,2881,1575,1306,123,664,615,600,611,268,79,...,1,0,0,0,0,0,0,0,0,23.014759
3,6,0,6,0,0,0,0,6,0,0,...,0,1,0,0,0,0,0,0,0,23.989693
4,1750,1085,665,34,219,224,610,606,57,14,...,0,0,1,0,0,0,0,0,0,22.848240


첫번째 lasso regression

In [98]:
#lasso 회귀
data_frame = rawdata1_junggu
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = variable
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.637624672329251,  0.7212332523242558,  0.8027135657815256, [-1.35548032e-04 -0.00000000e+00 -0.00000000e+00 -1.69669475e-04
 -0.00000000e+00 -1.56525620e-04  7.37457715e-04  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.27320283e-08 -6.65329997e-11  7.44860987e-11 -5.77587965e-11
 -1.85794861e-10  5.38270751e-11 -2.41119348e-10 -6.73529885e-10
  4.25010088e-09 -6.18063668e-10  1.99724892e-09  3.80872191e-06
 -2.60364757e-04  1.76868047e-05 -0.00000000e+00  3.91571149e-04
  0.00000000e+00  0.00000000e+00  4.27874875e-04 -6.49666614e-04
  3.48170594e-04 -2.79201465e-04 -1.69325702e-04  0.00000000e+00
  0.00000000e+00 -8.13951537e-10 -5.84592030e-05 -3.49277107e-05
  8.19597682e-05  1.43560352e-04 -1.17648566e-04 -3.00390363e-06
  7.97376846e-05 -1.35406704e-04  1.39955731e-04  0.00000000e+00
  2.82695280e-04 -3.0

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.46636693830255, tolerance: 0.01753036052775323
  positive)


In [102]:
#변수 출력
zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)
jjin_zero_index = np.where(variables == 0)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대

while문을 이용해 위의 lasso regression을 회귀식에서 계수가 0인 변수가 없을때까지 돌렸다

In [104]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata1_junggu
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(variable[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.435341249974204, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.638397813942002, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.493285183102556, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not conver

중요 변수

In [106]:
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수']

중요 변수들의 계수

In [107]:
variables

array([-0.00493897, -0.00359462,  0.00194974,  0.00715803,  0.00540519,
        0.00269105,  0.01062378,  0.00317626])

결과

계수: [-0.00493897, -0.00359462,  0.00194974,  0.00715803,  0.00540519,0.00269105,  0.01062378,  0.00317626]

변수: ['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', 
     '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수']

##### #노원구

In [109]:
rawdata_nowon = rawdata[rawdata['sigungu']==11350]
rawdata_nowon

,year,quarter,sigungu,dong,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
11,2015,1,11350,11350560,6106374871,818,437,381,81,137,...,383,284,387,65,0,171,275,295,412,61
12,2015,1,11350,11350595,10682998200,5769,2868,2901,876,1051,...,583,456,560,66,0,223,469,468,627,50
13,2015,1,11350,11350611,708022569,1826,879,947,381,311,...,46,33,32,2,0,14,32,36,37,2
14,2015,1,11350,11350624,7511669918,1604,797,807,333,197,...,642,497,583,75,0,317,537,507,612,72
15,2015,1,11350,11350630,3505036944,1827,906,921,337,277,...,261,204,247,27,0,121,222,229,268,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2019,2,11350,11350611,1354416123,1523,759,764,258,330,...,159,161,194,94,203,264,219,216,230,98
320,2019,2,11350,11350624,7500392817,1815,926,889,327,251,...,1033,894,1470,813,1157,934,972,872,1371,691
321,2019,2,11350,11350630,4787971115,2506,1237,1269,315,413,...,719,695,823,255,283,398,643,623,737,221
322,2019,2,11350,11350640,26976275176,7366,3723,3643,923,1173,...,2651,2660,3298,1637,2725,3163,2959,2439,2898,1461


In [110]:
rawdata1_nowon = pd.get_dummies(rawdata_nowon, columns=['year','quarter','sigungu','dong'], prefix=['year','quarter','sigungu','dong'])

In [115]:
rawdata1_nowon.columns

Index(['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수',
       '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수', '연령대 60 이상 상주인구 수',
       '남성연령대 10 상주인구 수',
       ...
       'quarter_4', 'sigungu_11350', 'dong_11350560', 'dong_11350595',
       'dong_11350611', 'dong_11350624', 'dong_11350630', 'dong_11350640',
       'dong_11350670', 'Logamt'],
      dtype='object', length=631)

In [116]:
variable = ['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_10_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '여성연령대_60_이상_직장_인구_수',
 '집객시설_수',
 '관공서_수',
 '은행_수',
 '종합병원_수',
 '일반_병원_수',
 '약국_수',
 '유치원_수',
 '초등학교_수',
 '중학교_수',
 '고등학교_수',
 '대학교_수',
 '백화점_수',
 '슈퍼마켓_수',
 '극장_수',
 '숙박_시설_수',
 '공항_수',
 '철도_역_수',
 '버스_터미널_수',
 '지하철_역_수',
 '버스_정거장_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대_6_유동인구_수',
 '월요일_유동인구_수',
 '화요일_유동인구_수',
 '수요일_유동인구_수',
 '목요일_유동인구_수',
 '금요일_유동인구_수',
 '토요일_유동인구_수',
 '일요일_유동인구_수',
 '남성연령대_10_월요일시간대_1_유동인구_수',
 '남성연령대_10_월요일시간대_2_유동인구_수',
 '남성연령대_10_월요일시간대_3_유동인구_수',
 '남성연령대_10_월요일시간대_4_유동인구_수',
 '남성연령대_10_월요일시간대_5_유동인구_수',
 '남성연령대_10_월요일시간대_6_유동인구_수',
 '남성연령대_10_화요일시간대_1_유동인구_수',
 '남성연령대_10_화요일시간대_2_유동인구_수',
 '남성연령대_10_화요일시간대_3_유동인구_수',
 '남성연령대_10_화요일시간대_4_유동인구_수',
 '남성연령대_10_화요일시간대_5_유동인구_수',
 '남성연령대_10_화요일시간대_6_유동인구_수',
 '남성연령대_10_수요일시간대_1_유동인구_수',
 '남성연령대_10_수요일시간대_2_유동인구_수',
 '남성연령대_10_수요일시간대_3_유동인구_수',
 '남성연령대_10_수요일시간대_4_유동인구_수',
 '남성연령대_10_수요일시간대_5_유동인구_수',
 '남성연령대_10_수요일시간대_6_유동인구_수',
 '남성연령대_10_목요일시간대_1_유동인구_수',
 '남성연령대_10_목요일시간대_2_유동인구_수',
 '남성연령대_10_목요일시간대_3_유동인구_수',
 '남성연령대_10_목요일시간대_4_유동인구_수',
 '남성연령대_10_목요일시간대_5_유동인구_수',
 '남성연령대_10_목요일시간대_6_유동인구_수',
 '남성연령대_10_금요일시간대_1_유동인구_수',
 '남성연령대_10_금요일시간대_2_유동인구_수',
 '남성연령대_10_금요일시간대_3_유동인구_수',
 '남성연령대_10_금요일시간대_4_유동인구_수',
 '남성연령대_10_금요일시간대_5_유동인구_수',
 '남성연령대_10_금요일시간대_6_유동인구_수',
 '남성연령대_10_토요일시간대_1_유동인구_수',
 '남성연령대_10_토요일시간대_2_유동인구_수',
 '남성연령대_10_토요일시간대_3_유동인구_수',
 '남성연령대_10_토요일시간대_4_유동인구_수',
 '남성연령대_10_토요일시간대_5_유동인구_수',
 '남성연령대_10_토요일시간대_6_유동인구_수',
 '남성연령대_10_일요일시간대_1_유동인구_수',
 '남성연령대_10_일요일시간대_2_유동인구_수',
 '남성연령대_10_일요일시간대_3_유동인구_수',
 '남성연령대_10_일요일시간대_4_유동인구_수',
 '남성연령대_10_일요일시간대_5_유동인구_수',
 '남성연령대_10_일요일시간대_6_유동인구_수',
 '남성연령대_20_월요일시간대_1_유동인구_수',
 '남성연령대_20_월요일시간대_2_유동인구_수',
 '남성연령대_20_월요일시간대_3_유동인구_수',
 '남성연령대_20_월요일시간대_4_유동인구_수',
 '남성연령대_20_월요일시간대_5_유동인구_수',
 '남성연령대_20_월요일시간대_6_유동인구_수',
 '남성연령대_20_화요일시간대_1_유동인구_수',
 '남성연령대_20_화요일시간대_2_유동인구_수',
 '남성연령대_20_화요일시간대_3_유동인구_수',
 '남성연령대_20_화요일시간대_4_유동인구_수',
 '남성연령대_20_화요일시간대_5_유동인구_수',
 '남성연령대_20_화요일시간대_6_유동인구_수',
 '남성연령대_20_수요일시간대_1_유동인구_수',
 '남성연령대_20_수요일시간대_2_유동인구_수',
 '남성연령대_20_수요일시간대_3_유동인구_수',
 '남성연령대_20_수요일시간대_4_유동인구_수',
 '남성연령대_20_수요일시간대_5_유동인구_수',
 '남성연령대_20_수요일시간대_6_유동인구_수',
 '남성연령대_20_목요일시간대_1_유동인구_수',
 '남성연령대_20_목요일시간대_2_유동인구_수',
 '남성연령대_20_목요일시간대_3_유동인구_수',
 '남성연령대_20_목요일시간대_4_유동인구_수',
 '남성연령대_20_목요일시간대_5_유동인구_수',
 '남성연령대_20_목요일시간대_6_유동인구_수',
 '남성연령대_20_금요일시간대_1_유동인구_수',
 '남성연령대_20_금요일시간대_2_유동인구_수',
 '남성연령대_20_금요일시간대_3_유동인구_수',
 '남성연령대_20_금요일시간대_4_유동인구_수',
 '남성연령대_20_금요일시간대_5_유동인구_수',
 '남성연령대_20_금요일시간대_6_유동인구_수',
 '남성연령대_20_토요일시간대_1_유동인구_수',
 '남성연령대_20_토요일시간대_2_유동인구_수',
 '남성연령대_20_토요일시간대_3_유동인구_수',
 '남성연령대_20_토요일시간대_4_유동인구_수',
 '남성연령대_20_토요일시간대_5_유동인구_수',
 '남성연령대_20_토요일시간대_6_유동인구_수',
 '남성연령대_20_일요일시간대_1_유동인구_수',
 '남성연령대_20_일요일시간대_2_유동인구_수',
 '남성연령대_20_일요일시간대_3_유동인구_수',
 '남성연령대_20_일요일시간대_4_유동인구_수',
 '남성연령대_20_일요일시간대_5_유동인구_수',
 '남성연령대_20_일요일시간대_6_유동인구_수',
 '남성연령대_30_월요일시간대_1_유동인구_수',
 '남성연령대_30_월요일시간대_2_유동인구_수',
 '남성연령대_30_월요일시간대_3_유동인구_수',
 '남성연령대_30_월요일시간대_4_유동인구_수',
 '남성연령대_30_월요일시간대_5_유동인구_수',
 '남성연령대_30_월요일시간대_6_유동인구_수',
 '남성연령대_30_화요일시간대_1_유동인구_수',
 '남성연령대_30_화요일시간대_2_유동인구_수',
 '남성연령대_30_화요일시간대_3_유동인구_수',
 '남성연령대_30_화요일시간대_4_유동인구_수',
 '남성연령대_30_화요일시간대_5_유동인구_수',
 '남성연령대_30_화요일시간대_6_유동인구_수',
 '남성연령대_30_수요일시간대_1_유동인구_수',
 '남성연령대_30_수요일시간대_2_유동인구_수',
 '남성연령대_30_수요일시간대_3_유동인구_수',
 '남성연령대_30_수요일시간대_4_유동인구_수',
 '남성연령대_30_수요일시간대_5_유동인구_수',
 '남성연령대_30_수요일시간대_6_유동인구_수',
 '남성연령대_30_목요일시간대_1_유동인구_수',
 '남성연령대_30_목요일시간대_2_유동인구_수',
 '남성연령대_30_목요일시간대_3_유동인구_수',
 '남성연령대_30_목요일시간대_4_유동인구_수',
 '남성연령대_30_목요일시간대_5_유동인구_수',
 '남성연령대_30_목요일시간대_6_유동인구_수',
 '남성연령대_30_금요일시간대_1_유동인구_수',
 '남성연령대_30_금요일시간대_2_유동인구_수',
 '남성연령대_30_금요일시간대_3_유동인구_수',
 '남성연령대_30_금요일시간대_4_유동인구_수',
 '남성연령대_30_금요일시간대_5_유동인구_수',
 '남성연령대_30_금요일시간대_6_유동인구_수',
 '남성연령대_30_토요일시간대_1_유동인구_수',
 '남성연령대_30_토요일시간대_2_유동인구_수',
 '남성연령대_30_토요일시간대_3_유동인구_수',
 '남성연령대_30_토요일시간대_4_유동인구_수',
 '남성연령대_30_토요일시간대_5_유동인구_수',
 '남성연령대_30_토요일시간대_6_유동인구_수',
 '남성연령대_30_일요일시간대_1_유동인구_수',
 '남성연령대_30_일요일시간대_2_유동인구_수',
 '남성연령대_30_일요일시간대_3_유동인구_수',
 '남성연령대_30_일요일시간대_4_유동인구_수',
 '남성연령대_30_일요일시간대_5_유동인구_수',
 '남성연령대_30_일요일시간대_6_유동인구_수',
 '남성연령대_40_월요일시간대_1_유동인구_수',
 '남성연령대_40_월요일시간대_2_유동인구_수',
 '남성연령대_40_월요일시간대_3_유동인구_수',
 '남성연령대_40_월요일시간대_4_유동인구_수',
 '남성연령대_40_월요일시간대_5_유동인구_수',
 '남성연령대_40_월요일시간대_6_유동인구_수',
 '남성연령대_40_화요일시간대_1_유동인구_수',
 '남성연령대_40_화요일시간대_2_유동인구_수',
 '남성연령대_40_화요일시간대_3_유동인구_수',
 '남성연령대_40_화요일시간대_4_유동인구_수',
 '남성연령대_40_화요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_6_유동인구_수',
 '남성연령대_40_수요일시간대_1_유동인구_수',
 '남성연령대_40_수요일시간대_2_유동인구_수',
 '남성연령대_40_수요일시간대_3_유동인구_수',
 '남성연령대_40_수요일시간대_4_유동인구_수',
 '남성연령대_40_수요일시간대_5_유동인구_수',
 '남성연령대_40_수요일시간대_6_유동인구_수',
 '남성연령대_40_목요일시간대_1_유동인구_수',
 '남성연령대_40_목요일시간대_2_유동인구_수',
 '남성연령대_40_목요일시간대_3_유동인구_수',
 '남성연령대_40_목요일시간대_4_유동인구_수',
 '남성연령대_40_목요일시간대_5_유동인구_수',
 '남성연령대_40_목요일시간대_6_유동인구_수',
 '남성연령대_40_금요일시간대_1_유동인구_수',
 '남성연령대_40_금요일시간대_2_유동인구_수',
 '남성연령대_40_금요일시간대_3_유동인구_수',
 '남성연령대_40_금요일시간대_4_유동인구_수',
 '남성연령대_40_금요일시간대_5_유동인구_수',
 '남성연령대_40_금요일시간대_6_유동인구_수',
 '남성연령대_40_토요일시간대_1_유동인구_수',
 '남성연령대_40_토요일시간대_2_유동인구_수',
 '남성연령대_40_토요일시간대_3_유동인구_수',
 '남성연령대_40_토요일시간대_4_유동인구_수',
 '남성연령대_40_토요일시간대_5_유동인구_수',
 '남성연령대_40_토요일시간대_6_유동인구_수',
 '남성연령대_40_일요일시간대_1_유동인구_수',
 '남성연령대_40_일요일시간대_2_유동인구_수',
 '남성연령대_40_일요일시간대_3_유동인구_수',
 '남성연령대_40_일요일시간대_4_유동인구_수',
 '남성연령대_40_일요일시간대_5_유동인구_수',
 '남성연령대_40_일요일시간대_6_유동인구_수',
 '남성연령대_50_월요일시간대_1_유동인구_수',
 '남성연령대_50_월요일시간대_2_유동인구_수',
 '남성연령대_50_월요일시간대_3_유동인구_수',
 '남성연령대_50_월요일시간대_4_유동인구_수',
 '남성연령대_50_월요일시간대_5_유동인구_수',
 '남성연령대_50_월요일시간대_6_유동인구_수',
 '남성연령대_50_화요일시간대_1_유동인구_수',
 '남성연령대_50_화요일시간대_2_유동인구_수',
 '남성연령대_50_화요일시간대_3_유동인구_수',
 '남성연령대_50_화요일시간대_4_유동인구_수',
 '남성연령대_50_화요일시간대_5_유동인구_수',
 '남성연령대_50_화요일시간대_6_유동인구_수',
 '남성연령대_50_수요일시간대_1_유동인구_수',
 '남성연령대_50_수요일시간대_2_유동인구_수',
 '남성연령대_50_수요일시간대_3_유동인구_수',
 '남성연령대_50_수요일시간대_4_유동인구_수',
 '남성연령대_50_수요일시간대_5_유동인구_수',
 '남성연령대_50_수요일시간대_6_유동인구_수',
 '남성연령대_50_목요일시간대_1_유동인구_수',
 '남성연령대_50_목요일시간대_2_유동인구_수',
 '남성연령대_50_목요일시간대_3_유동인구_수',
 '남성연령대_50_목요일시간대_4_유동인구_수',
 '남성연령대_50_목요일시간대_5_유동인구_수',
 '남성연령대_50_목요일시간대_6_유동인구_수',
 '남성연령대_50_금요일시간대_1_유동인구_수',
 '남성연령대_50_금요일시간대_2_유동인구_수',
 '남성연령대_50_금요일시간대_3_유동인구_수',
 '남성연령대_50_금요일시간대_4_유동인구_수',
 '남성연령대_50_금요일시간대_5_유동인구_수',
 '남성연령대_50_금요일시간대_6_유동인구_수',
 '남성연령대_50_토요일시간대_1_유동인구_수',
 '남성연령대_50_토요일시간대_2_유동인구_수',
 '남성연령대_50_토요일시간대_3_유동인구_수',
 '남성연령대_50_토요일시간대_4_유동인구_수',
 '남성연령대_50_토요일시간대_5_유동인구_수',
 '남성연령대_50_토요일시간대_6_유동인구_수',
 '남성연령대_50_일요일시간대_1_유동인구_수',
 '남성연령대_50_일요일시간대_2_유동인구_수',
 '남성연령대_50_일요일시간대_3_유동인구_수',
 '남성연령대_50_일요일시간대_4_유동인구_수',
 '남성연령대_50_일요일시간대_5_유동인구_수',
 '남성연령대_50_일요일시간대_6_유동인구_수',
 '남성연령대_60_이상_월요일시간대_1_유동인구_수',
 '남성연령대_60_이상_월요일시간대_2_유동인구_수',
 '남성연령대_60_이상_월요일시간대_3_유동인구_수',
 '남성연령대_60_이상_월요일시간대_4_유동인구_수',
 '남성연령대_60_이상_월요일시간대_5_유동인구_수',
 '남성연령대_60_이상_월요일시간대_6_유동인구_수',
 '남성연령대_60_이상_화요일시간대_1_유동인구_수',
 '남성연령대_60_이상_화요일시간대_2_유동인구_수',
 '남성연령대_60_이상_화요일시간대_3_유동인구_수',
 '남성연령대_60_이상_화요일시간대_4_유동인구_수',
 '남성연령대_60_이상_화요일시간대_5_유동인구_수',
 '남성연령대_60_이상_화요일시간대_6_유동인구_수',
 '남성연령대_60_이상_수요일시간대_1_유동인구_수',
 '남성연령대_60_이상_수요일시간대_2_유동인구_수',
 '남성연령대_60_이상_수요일시간대_3_유동인구_수',
 '남성연령대_60_이상_수요일시간대_4_유동인구_수',
 '남성연령대_60_이상_수요일시간대_5_유동인구_수',
 '남성연령대_60_이상_수요일시간대_6_유동인구_수',
 '남성연령대_60_이상_목요일시간대_1_유동인구_수',
 '남성연령대_60_이상_목요일시간대_2_유동인구_수',
 '남성연령대_60_이상_목요일시간대_3_유동인구_수',
 '남성연령대_60_이상_목요일시간대_4_유동인구_수',
 '남성연령대_60_이상_목요일시간대_5_유동인구_수',
 '남성연령대_60_이상_목요일시간대_6_유동인구_수',
 '남성연령대_60_이상_금요일시간대_1_유동인구_수',
 '남성연령대_60_이상_금요일시간대_2_유동인구_수',
 '남성연령대_60_이상_금요일시간대_3_유동인구_수',
 '남성연령대_60_이상_금요일시간대_4_유동인구_수',
 '남성연령대_60_이상_금요일시간대_5_유동인구_수',
 '남성연령대_60_이상_금요일시간대_6_유동인구_수',
 '남성연령대_60_이상_토요일시간대_1_유동인구_수',
 '남성연령대_60_이상_토요일시간대_2_유동인구_수',
 '남성연령대_60_이상_토요일시간대_3_유동인구_수',
 '남성연령대_60_이상_토요일시간대_4_유동인구_수',
 '남성연령대_60_이상_토요일시간대_5_유동인구_수',
 '남성연령대_60_이상_토요일시간대_6_유동인구_수',
 '남성연령대_60_이상_일요일시간대_1_유동인구_수',
 '남성연령대_60_이상_일요일시간대_2_유동인구_수',
 '남성연령대_60_이상_일요일시간대_3_유동인구_수',
 '남성연령대_60_이상_일요일시간대_4_유동인구_수',
 '남성연령대_60_이상_일요일시간대_5_유동인구_수',
 '남성연령대_60_이상_일요일시간대_6_유동인구_수',
 '여성연령대_10_월요일시간대_1_유동인구_수',
 '여성연령대_10_월요일시간대_2_유동인구_수',
 '여성연령대_10_월요일시간대_3_유동인구_수',
 '여성연령대_10_월요일시간대_4_유동인구_수',
 '여성연령대_10_월요일시간대_5_유동인구_수',
 '여성연령대_10_월요일시간대_6_유동인구_수',
 '여성연령대_10_화요일시간대_1_유동인구_수',
 '여성연령대_10_화요일시간대_2_유동인구_수',
 '여성연령대_10_화요일시간대_3_유동인구_수',
 '여성연령대_10_화요일시간대_4_유동인구_수',
 '여성연령대_10_화요일시간대_5_유동인구_수',
 '여성연령대_10_화요일시간대_6_유동인구_수',
 '여성연령대_10_수요일시간대_1_유동인구_수',
 '여성연령대_10_수요일시간대_2_유동인구_수',
 '여성연령대_10_수요일시간대_3_유동인구_수',
 '여성연령대_10_수요일시간대_4_유동인구_수',
 '여성연령대_10_수요일시간대_5_유동인구_수',
 '여성연령대_10_수요일시간대_6_유동인구_수',
 '여성연령대_10_목요일시간대_1_유동인구_수',
 '여성연령대_10_목요일시간대_2_유동인구_수',
 '여성연령대_10_목요일시간대_3_유동인구_수',
 '여성연령대_10_목요일시간대_4_유동인구_수',
 '여성연령대_10_목요일시간대_5_유동인구_수',
 '여성연령대_10_목요일시간대_6_유동인구_수',
 '여성연령대_10_금요일시간대_1_유동인구_수',
 '여성연령대_10_금요일시간대_2_유동인구_수',
 '여성연령대_10_금요일시간대_3_유동인구_수',
 '여성연령대_10_금요일시간대_4_유동인구_수',
 '여성연령대_10_금요일시간대_5_유동인구_수',
 '여성연령대_10_금요일시간대_6_유동인구_수',
 '여성연령대_10_토요일시간대_1_유동인구_수',
 '여성연령대_10_토요일시간대_2_유동인구_수',
 '여성연령대_10_토요일시간대_3_유동인구_수',
 '여성연령대_10_토요일시간대_4_유동인구_수',
 '여성연령대_10_토요일시간대_5_유동인구_수',
 '여성연령대_10_토요일시간대_6_유동인구_수',
 '여성연령대_10_일요일시간대_1_유동인구_수',
 '여성연령대_10_일요일시간대_2_유동인구_수',
 '여성연령대_10_일요일시간대_3_유동인구_수',
 '여성연령대_10_일요일시간대_4_유동인구_수',
 '여성연령대_10_일요일시간대_5_유동인구_수',
 '여성연령대_10_일요일시간대_6_유동인구_수',
 '여성연령대_20_월요일시간대_1_유동인구_수',
 '여성연령대_20_월요일시간대_2_유동인구_수',
 '여성연령대_20_월요일시간대_3_유동인구_수',
 '여성연령대_20_월요일시간대_4_유동인구_수',
 '여성연령대_20_월요일시간대_5_유동인구_수',
 '여성연령대_20_월요일시간대_6_유동인구_수',
 '여성연령대_20_화요일시간대_1_유동인구_수',
 '여성연령대_20_화요일시간대_2_유동인구_수',
 '여성연령대_20_화요일시간대_3_유동인구_수',
 '여성연령대_20_화요일시간대_4_유동인구_수',
 '여성연령대_20_화요일시간대_5_유동인구_수',
 '여성연령대_20_화요일시간대_6_유동인구_수',
 '여성연령대_20_수요일시간대_1_유동인구_수',
 '여성연령대_20_수요일시간대_2_유동인구_수',
 '여성연령대_20_수요일시간대_3_유동인구_수',
 '여성연령대_20_수요일시간대_4_유동인구_수',
 '여성연령대_20_수요일시간대_5_유동인구_수',
 '여성연령대_20_수요일시간대_6_유동인구_수',
 '여성연령대_20_목요일시간대_1_유동인구_수',
 '여성연령대_20_목요일시간대_2_유동인구_수',
 '여성연령대_20_목요일시간대_3_유동인구_수',
 '여성연령대_20_목요일시간대_4_유동인구_수',
 '여성연령대_20_목요일시간대_5_유동인구_수',
 '여성연령대_20_목요일시간대_6_유동인구_수',
 '여성연령대_20_금요일시간대_1_유동인구_수',
 '여성연령대_20_금요일시간대_2_유동인구_수',
 '여성연령대_20_금요일시간대_3_유동인구_수',
 '여성연령대_20_금요일시간대_4_유동인구_수',
 '여성연령대_20_금요일시간대_5_유동인구_수',
 '여성연령대_20_금요일시간대_6_유동인구_수',
 '여성연령대_20_토요일시간대_1_유동인구_수',
 '여성연령대_20_토요일시간대_2_유동인구_수',
 '여성연령대_20_토요일시간대_3_유동인구_수',
 '여성연령대_20_토요일시간대_4_유동인구_수',
 '여성연령대_20_토요일시간대_5_유동인구_수',
 '여성연령대_20_토요일시간대_6_유동인구_수',
 '여성연령대_20_일요일시간대_1_유동인구_수',
 '여성연령대_20_일요일시간대_2_유동인구_수',
 '여성연령대_20_일요일시간대_3_유동인구_수',
 '여성연령대_20_일요일시간대_4_유동인구_수',
 '여성연령대_20_일요일시간대_5_유동인구_수',
 '여성연령대_20_일요일시간대_6_유동인구_수',
 '여성연령대_30_월요일시간대_1_유동인구_수',
 '여성연령대_30_월요일시간대_2_유동인구_수',
 '여성연령대_30_월요일시간대_3_유동인구_수',
 '여성연령대_30_월요일시간대_4_유동인구_수',
 '여성연령대_30_월요일시간대_5_유동인구_수',
 '여성연령대_30_월요일시간대_6_유동인구_수',
 '여성연령대_30_화요일시간대_1_유동인구_수',
 '여성연령대_30_화요일시간대_2_유동인구_수',
 '여성연령대_30_화요일시간대_3_유동인구_수',
 '여성연령대_30_화요일시간대_4_유동인구_수',
 '여성연령대_30_화요일시간대_5_유동인구_수',
 '여성연령대_30_화요일시간대_6_유동인구_수',
 '여성연령대_30_수요일시간대_1_유동인구_수',
 '여성연령대_30_수요일시간대_2_유동인구_수',
 '여성연령대_30_수요일시간대_3_유동인구_수',
 '여성연령대_30_수요일시간대_4_유동인구_수',
 '여성연령대_30_수요일시간대_5_유동인구_수',
 '여성연령대_30_수요일시간대_6_유동인구_수',
 '여성연령대_30_목요일시간대_1_유동인구_수',
 '여성연령대_30_목요일시간대_2_유동인구_수',
 '여성연령대_30_목요일시간대_3_유동인구_수',
 '여성연령대_30_목요일시간대_4_유동인구_수',
 '여성연령대_30_목요일시간대_5_유동인구_수',
 '여성연령대_30_목요일시간대_6_유동인구_수',
 '여성연령대_30_금요일시간대_1_유동인구_수',
 '여성연령대_30_금요일시간대_2_유동인구_수',
 '여성연령대_30_금요일시간대_3_유동인구_수',
 '여성연령대_30_금요일시간대_4_유동인구_수',
 '여성연령대_30_금요일시간대_5_유동인구_수',
 '여성연령대_30_금요일시간대_6_유동인구_수',
 '여성연령대_30_토요일시간대_1_유동인구_수',
 '여성연령대_30_토요일시간대_2_유동인구_수',
 '여성연령대_30_토요일시간대_3_유동인구_수',
 '여성연령대_30_토요일시간대_4_유동인구_수',
 '여성연령대_30_토요일시간대_5_유동인구_수',
 '여성연령대_30_토요일시간대_6_유동인구_수',
 '여성연령대_30_일요일시간대_1_유동인구_수',
 '여성연령대_30_일요일시간대_2_유동인구_수',
 '여성연령대_30_일요일시간대_3_유동인구_수',
 '여성연령대_30_일요일시간대_4_유동인구_수',
 '여성연령대_30_일요일시간대_5_유동인구_수',
 '여성연령대_30_일요일시간대_6_유동인구_수',
 '여성연령대_40_월요일시간대_1_유동인구_수',
 '여성연령대_40_월요일시간대_2_유동인구_수',
 '여성연령대_40_월요일시간대_3_유동인구_수',
 '여성연령대_40_월요일시간대_4_유동인구_수',
 '여성연령대_40_월요일시간대_5_유동인구_수',
 '여성연령대_40_월요일시간대_6_유동인구_수',
 '여성연령대_40_화요일시간대_1_유동인구_수',
 '여성연령대_40_화요일시간대_2_유동인구_수',
 '여성연령대_40_화요일시간대_3_유동인구_수',
 '여성연령대_40_화요일시간대_4_유동인구_수',
 '여성연령대_40_화요일시간대_5_유동인구_수',
 '여성연령대_40_화요일시간대_6_유동인구_수',
 '여성연령대_40_수요일시간대_1_유동인구_수',
 '여성연령대_40_수요일시간대_2_유동인구_수',
 '여성연령대_40_수요일시간대_3_유동인구_수',
 '여성연령대_40_수요일시간대_4_유동인구_수',
 '여성연령대_40_수요일시간대_5_유동인구_수',
 '여성연령대_40_수요일시간대_6_유동인구_수',
 '여성연령대_40_목요일시간대_1_유동인구_수',
 '여성연령대_40_목요일시간대_2_유동인구_수',
 '여성연령대_40_목요일시간대_3_유동인구_수',
 '여성연령대_40_목요일시간대_4_유동인구_수',
 '여성연령대_40_목요일시간대_5_유동인구_수',
 '여성연령대_40_목요일시간대_6_유동인구_수',
 '여성연령대_40_금요일시간대_1_유동인구_수',
 '여성연령대_40_금요일시간대_2_유동인구_수',
 '여성연령대_40_금요일시간대_3_유동인구_수',
 '여성연령대_40_금요일시간대_4_유동인구_수',
 '여성연령대_40_금요일시간대_5_유동인구_수',
 '여성연령대_40_금요일시간대_6_유동인구_수',
 '여성연령대_40_토요일시간대_1_유동인구_수',
 '여성연령대_40_토요일시간대_2_유동인구_수',
 '여성연령대_40_토요일시간대_3_유동인구_수',
 '여성연령대_40_토요일시간대_4_유동인구_수',
 '여성연령대_40_토요일시간대_5_유동인구_수',
 '여성연령대_40_토요일시간대_6_유동인구_수',
 '여성연령대_40_일요일시간대_1_유동인구_수',
 '여성연령대_40_일요일시간대_2_유동인구_수',
 '여성연령대_40_일요일시간대_3_유동인구_수',
 '여성연령대_40_일요일시간대_4_유동인구_수',
 '여성연령대_40_일요일시간대_5_유동인구_수',
 '여성연령대_40_일요일시간대_6_유동인구_수',
 '여성연령대_50_월요일시간대_1_유동인구_수',
 '여성연령대_50_월요일시간대_2_유동인구_수',
 '여성연령대_50_월요일시간대_3_유동인구_수',
 '여성연령대_50_월요일시간대_4_유동인구_수',
 '여성연령대_50_월요일시간대_5_유동인구_수',
 '여성연령대_50_월요일시간대_6_유동인구_수',
 '여성연령대_50_화요일시간대_1_유동인구_수',
 '여성연령대_50_화요일시간대_2_유동인구_수',
 '여성연령대_50_화요일시간대_3_유동인구_수',
 '여성연령대_50_화요일시간대_4_유동인구_수',
 '여성연령대_50_화요일시간대_5_유동인구_수',
 '여성연령대_50_화요일시간대_6_유동인구_수',
 '여성연령대_50_수요일시간대_1_유동인구_수',
 '여성연령대_50_수요일시간대_2_유동인구_수',
 '여성연령대_50_수요일시간대_3_유동인구_수',
 '여성연령대_50_수요일시간대_4_유동인구_수',
 '여성연령대_50_수요일시간대_5_유동인구_수',
 '여성연령대_50_수요일시간대_6_유동인구_수',
 '여성연령대_50_목요일시간대_1_유동인구_수',
 '여성연령대_50_목요일시간대_2_유동인구_수',
 '여성연령대_50_목요일시간대_3_유동인구_수',
 '여성연령대_50_목요일시간대_4_유동인구_수',
 '여성연령대_50_목요일시간대_5_유동인구_수',
 '여성연령대_50_목요일시간대_6_유동인구_수',
 '여성연령대_50_금요일시간대_1_유동인구_수',
 '여성연령대_50_금요일시간대_2_유동인구_수',
 '여성연령대_50_금요일시간대_3_유동인구_수',
 '여성연령대_50_금요일시간대_4_유동인구_수',
 '여성연령대_50_금요일시간대_5_유동인구_수',
 '여성연령대_50_금요일시간대_6_유동인구_수',
 '여성연령대_50_토요일시간대_1_유동인구_수',
 '여성연령대_50_토요일시간대_2_유동인구_수',
 '여성연령대_50_토요일시간대_3_유동인구_수',
 '여성연령대_50_토요일시간대_4_유동인구_수',
 '여성연령대_50_토요일시간대_5_유동인구_수',
 '여성연령대_50_토요일시간대_6_유동인구_수',
 '여성연령대_50_일요일시간대_1_유동인구_수',
 '여성연령대_50_일요일시간대_2_유동인구_수',
 '여성연령대_50_일요일시간대_3_유동인구_수',
 '여성연령대_50_일요일시간대_4_유동인구_수',
 '여성연령대_50_일요일시간대_5_유동인구_수',
 '여성연령대_50_일요일시간대_6_유동인구_수',
 '여성연령대_60_이상_월요일시간대_1_유동인구_수',
 '여성연령대_60_이상_월요일시간대_2_유동인구_수',
 '여성연령대_60_이상_월요일시간대_3_유동인구_수',
 '여성연령대_60_이상_월요일시간대_4_유동인구_수',
 '여성연령대_60_이상_월요일시간대_5_유동인구_수',
 '여성연령대_60_이상_월요일시간대_6_유동인구_수',
 '여성연령대_60_이상_화요일시간대_1_유동인구_수',
 '여성연령대_60_이상_화요일시간대_2_유동인구_수',
 '여성연령대_60_이상_화요일시간대_3_유동인구_수',
 '여성연령대_60_이상_화요일시간대_4_유동인구_수',
 '여성연령대_60_이상_화요일시간대_5_유동인구_수',
 '여성연령대_60_이상_화요일시간대_6_유동인구_수',
 '여성연령대_60_이상_수요일시간대_1_유동인구_수',
 '여성연령대_60_이상_수요일시간대_2_유동인구_수',
 '여성연령대_60_이상_수요일시간대_3_유동인구_수',
 '여성연령대_60_이상_수요일시간대_4_유동인구_수',
 '여성연령대_60_이상_수요일시간대_5_유동인구_수',
 '여성연령대_60_이상_수요일시간대_6_유동인구_수',
 '여성연령대_60_이상_목요일시간대_1_유동인구_수',
 '여성연령대_60_이상_목요일시간대_2_유동인구_수',
 '여성연령대_60_이상_목요일시간대_3_유동인구_수',
 '여성연령대_60_이상_목요일시간대_4_유동인구_수',
 '여성연령대_60_이상_목요일시간대_5_유동인구_수',
 '여성연령대_60_이상_목요일시간대_6_유동인구_수',
 '여성연령대_60_이상_금요일시간대_1_유동인구_수',
 '여성연령대_60_이상_금요일시간대_2_유동인구_수',
 '여성연령대_60_이상_금요일시간대_3_유동인구_수',
 '여성연령대_60_이상_금요일시간대_4_유동인구_수',
 '여성연령대_60_이상_금요일시간대_5_유동인구_수',
 '여성연령대_60_이상_금요일시간대_6_유동인구_수',
 '여성연령대_60_이상_토요일시간대_1_유동인구_수',
 '여성연령대_60_이상_토요일시간대_2_유동인구_수',
 '여성연령대_60_이상_토요일시간대_3_유동인구_수',
 '여성연령대_60_이상_토요일시간대_4_유동인구_수',
 '여성연령대_60_이상_토요일시간대_5_유동인구_수',
 '여성연령대_60_이상_토요일시간대_6_유동인구_수',
 '여성연령대_60_이상_일요일시간대_1_유동인구_수',
 '여성연령대_60_이상_일요일시간대_2_유동인구_수',
 '여성연령대_60_이상_일요일시간대_3_유동인구_수',
 '여성연령대_60_이상_일요일시간대_4_유동인구_수',
 '여성연령대_60_이상_일요일시간대_5_유동인구_수',
 '여성연령대_60_이상_일요일시간대_6_유동인구_수',
 'year_2015',
 'year_2016',
 'year_2017',
 'year_2018',
 'year_2019',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4',
 'sigungu_11350', 'dong_11350560', 'dong_11350595','dong_11350611', 'dong_11350624', 'dong_11350630', 'dong_11350640','dong_11350670']


In [112]:
rawdata1_nowon["Logamt"] = np.log1p(rawdata1_nowon["amt"])
rawdata1_nowon.head()

,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,quarter_4,sigungu_11350,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
11,6106374871,818,437,381,81,137,188,160,171,81,...,0,1,1,0,0,0,0,0,0,22.532599
12,10682998200,5769,2868,2901,876,1051,1106,957,1015,764,...,0,1,0,1,0,0,0,0,0,23.091919
13,708022569,1826,879,947,381,311,223,382,352,177,...,0,1,0,0,1,0,0,0,0,20.377987
14,7511669918,1604,797,807,333,197,229,369,254,222,...,0,1,0,0,0,1,0,0,0,22.739724
15,3505036944,1827,906,921,337,277,283,348,349,233,...,0,1,0,0,0,0,1,0,0,21.977467


In [113]:
rawdata1_nowon = rawdata1_nowon.drop(['amt'],axis=1)
rawdata1_nowon.head()

,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,남성연령대 10 상주인구 수,...,quarter_4,sigungu_11350,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
11,818,437,381,81,137,188,160,171,81,47,...,0,1,1,0,0,0,0,0,0,22.532599
12,5769,2868,2901,876,1051,1106,957,1015,764,473,...,0,1,0,1,0,0,0,0,0,23.091919
13,1826,879,947,381,311,223,382,352,177,212,...,0,1,0,0,1,0,0,0,0,20.377987
14,1604,797,807,333,197,229,369,254,222,189,...,0,1,0,0,0,1,0,0,0,22.739724
15,1827,906,921,337,277,283,348,349,233,178,...,0,1,0,0,0,0,1,0,0,21.977467


첫번째 lasso regression

In [117]:
#lasso 회귀
data_frame = rawdata1_nowon
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = variable
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.6019179294043653,  0.6388789861452432,  0.6719233862673892, [-2.83092169e-05 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -1.76914482e-07 -1.58204296e-11 -9.30587099e-11  2.77092116e-10
  2.18009429e-10 -3.70610683e-10  4.38476508e-10 -1.66822727e-09
  8.12643495e-10 -1.19717203e-11  5.51213523e-10  0.00000000e+00
 -1.59441489e-04  7.55820260e-05  5.55085311e-04 -0.00000000e+00
  0.00000000e+00 -2.94930099e-04  1.36223277e-04  0.00000000e+00
  2.76975118e-05 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.79709584e-08 -4.04142665e-05  0.00000000e+00
 -7.11996890e-05  0.00000000e+00 -1.02892395e-03 -1.02036512e-03
  4.16328856e-06  3.52100232e-04  5.62738351e-04  0.00000000e+00
 -0.00000000e+00 -0.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.250746993309174, tolerance: 0.011453186703177914
  positive)


In [118]:
#변수 출력
zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)
jjin_zero_index = np.where(variables == 0)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(variable[index])
    
impovar

['총 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대_6_유동인구_수',
 '월요일_유동인구_수',
 '화요일_유동인구_수',
 '수요일_유동인구_수',
 '목요일_유동인구_수',
 '금요일_유동인구_수',
 '토요일_유동인구_수',
 '일요일_유동인구_수',
 '남성연령대_10_토요일시간대_5_유동인구_수',
 '남성연령대_20_토요일시간대_1_유동인구_수',
 '남성연령대_20_일요일시간대_2_유동인구_수',
 '남성연령대_40_월요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_6_유

회귀식의 계수가 0인 변수들이 없을때까지 lasso regression 돌리기

In [119]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata1_nowon
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(variable[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2565756573646902, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.8254246785410093, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.0546135481882257, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not 

In [120]:
len(jjin_zero_index[0])

0

중요 변수

In [121]:
impovar

['총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수']

변수들의 계수

In [122]:
variables

array([ 9.54410210e-05,  3.89973634e-03, -8.26271477e-03, -9.21530204e-04,
        5.00341513e-04,  4.93070152e-03,  3.81030077e-03,  6.20964084e-03])

결과

계수: [ 9.54410210e-05,  3.89973634e-03, -8.26271477e-03, -9.21530204e-04, 5.00341513e-04,  4.93070152e-03,  
     3.81030077e-03,  6.20964084e-03]
    
변수: ['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', '연령대 30 상주인구 수',
 '연령대 40 상주인구 수', '연령대 50 상주인구 수']

### Lasso Regression (1) 정리

#### 1. 전체
계수: [ 0.00046581  0.00160215 -0.00243765 -0.00078459  0.00021311]

변수: ['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수'] 


#### 2. 중구
계수: [-0.00493897, -0.00359462,  0.00194974,  0.00715803,  0.00540519,0.00269105,  0.01062378,  0.00317626]

변수: ['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', 
     '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수']
     
     
#### 3. 노원구
계수: [ 9.54410210e-05,  3.89973634e-03, -8.26271477e-03, -9.21530204e-04, 5.00341513e-04,  4.93070152e-03,  
     3.81030077e-03,  6.20964084e-03]
     
변수: ['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', '연령대 30 상주인구 수',
 '연령대 40 상주인구 수', '연령대 50 상주인구 수']

### Lasso Regression(1) 의 변수들을 이용한 Regression

위에서 정해진 변수들을 이용해 다양한 방법의 regression을 시행했다

Linear regression,
Ridge regression,
Lasso regression,
Elastic Net,
Gradient Boosting,
Support Vector,
XGBoost

In [128]:
def predict_columns(data_frame, x_cols, y_cols):
    train=data_frame[data_frame['year_2019'] != 1]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [LinearRegression(),Ridge(), Lasso(), ElasticNet(), GradientBoostingRegressor(), SVR(), XGBRegressor()]
    reg_model_names=['LinearRegression','Ridge', "Lasso", "ElasticNet", "GradientBoositng", " SupportVector", "XGBoost"]
    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        print('{} : {},  {},  {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse))

전체데이터

In [134]:
#전체데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
# 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4',
predict_columns(rawdata1,['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수'],'Logamt')

LinearRegression : 0.8697750965123084,  -20.255862061081498,  0.9759445844424742
Ridge : 0.8697750965123084,  -20.280201354442383,  0.9759636980375371
Lasso : 0.8975528742900861,  -25.424927348838757,  0.9817927094361175
ElasticNet : 0.8975528742900861,  -23.50152792517141,  0.9801613996986538
GradientBoositng : 0.44851042890088194,  0.9130960579715283,  0.2674391336588923
 SupportVector : 0.8410850132499733,  -2.5333677343128285,  0.854125067638971


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.27035647029026677, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11901433395058802, tolerance: 0.030091418941377436
  positive)


XGBoost : 0.38667351114946263,  0.9395903307404219,  0.2362259376858241


중구

In [136]:
predict_columns(rawdata1_junggu, ['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수'],'Logamt')

LinearRegression : 0.9453240486988517,  -0.35608262266269763,  0.8100363344514193
Ridge : 0.9453240486988517,  -0.3570942704298934,  0.8099935059727555
Lasso : 0.8901097003923102,  -0.7804720117993684,  0.8213356243452182
ElasticNet : 0.9453240486988517,  -0.5940556758354918,  0.8152308237333857
GradientBoositng : 0.3606514640253007,  0.9494131417033138,  0.20861139421940808
 SupportVector : 0.8901097003923102,  -1.216431512178111,  0.7819167404755283


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0044935406114917, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.875724212476172, tolerance: 0.01753036052775323
  positive)


XGBoost : 0.3381280594477895,  0.9471126807930891,  0.22916346808476024


노원구

In [138]:
predict_columns(rawdata1_nowon, ['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'quarter_1', 'quarter_2', 'quarter_3',
                                 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수',
                                 '연령대 20 상주인구 수', '연령대 30 상주인구 수', '연령대 40 상주인구 수', '연령대 50 상주인구 수'],'Logamt')

LinearRegression : 0.7678514039712836,  0.1987663739360479,  0.683419411163352
Ridge : 0.7678514039712836,  0.238536305746324,  0.6700516331572409
Lasso : 0.6249942611141408,  0.05985427054372139,  0.6198419575115524
ElasticNet : 0.6249942611141408,  0.12040072254939727,  0.6144949289397605
GradientBoositng : 0.45271600391439193,  0.9618003767776601,  0.19878934537524137
 SupportVector : 0.7640462191691582,  -1.2464394840806592,  0.8094206639959118


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22.689429768591964, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23.667957973129774, tolerance: 0.011453186703177914
  positive)


XGBoost : 0.45271600391439193,  0.968701429632332,  0.17842750309060723


### 2) Lasso Regression - year변수는 숫자로, quarter 변수는 object

#### #전지역

In [318]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

rawdata = pd.read_excel(path+'finaldata.xlsx', encoding = 'euc_kr')
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)

In [319]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [320]:
#MAE
def mae(y,yhat):
    return np.mean(abs(y - yhat))

In [321]:
rawdata2 = rawdata.copy()
rawdata2[['quarter','sigungu','dong']] = rawdata2[['quarter','sigungu','dong']].astype('object')
rawdata2.dtypes

year                            int64
quarter                        object
sigungu                        object
dong                           object
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

In [322]:
rawdata2 = pd.get_dummies(rawdata2, columns=['quarter','sigungu','dong'], prefix=['quarter','sigungu','dong'])

In [323]:
import numpy as np
rawdata2["Logamt"] = np.log1p(rawdata2["amt"])
rawdata2 = rawdata2.drop(['amt'],axis=1)
rawdata2.head()

,year,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,dong_11140665,dong_11140670,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
0,2015,657,376,281,35,55,80,119,194,174,...,0,0,0,0,0,0,0,0,0,23.134886
1,2015,964,353,611,0,55,171,244,317,177,...,0,0,0,0,0,0,0,0,0,25.320105
2,2015,2881,1575,1306,123,664,615,600,611,268,...,0,0,0,0,0,0,0,0,0,23.014759
3,2015,6,0,6,0,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,23.989693
4,2015,1750,1085,665,34,219,224,610,606,57,...,0,0,0,0,0,0,0,0,0,22.848240


In [324]:
columnname = rawdata2.columns
columnname = pd.DataFrame(columnname)
columnname.shape

(639, 1)

칼럼을 모두 가져오는 column_array

In [325]:
column_array = []
for i in range(639):
    column_array = np.append(column_array, columnname.iloc[[i]])

In [326]:
column_array = column_array.tolist()
len(column_array)

639

In [327]:
data_frame = rawdata2
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year'] != 2019)]
test=data_frame[data_frame['year'] == 2019]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7537845704249974,  0.6598234634488311,  0.9136377031306749, [-0.00000000e+00 -1.40972049e-04  0.00000000e+00 -1.03380073e-05
 -1.46043069e-03 -8.48412964e-04  1.38222514e-03  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  1.54860510e-03  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -1.27499560e-07  2.11003898e-11  9.71387089e-11
 -6.34077869e-11  1.03439904e-09 -1.67175332e-09  2.34117438e-10
  1.20878722e-09  7.45448824e-09 -9.73692853e-10 -7.38814884e-11
  0.00000000e+00 -0.00000000e+00  1.90529700e-05 -1.99330405e-04
 -0.00000000e+00  0.00000000e+00 -1.38810983e-04  4.24316188e-05
  1.00998541e-04  2.18230479e-04 -5.09598313e-04 -3.00541979e-05
 -4.51081722e-04 -0.00000000e+00 -2.65929069e-10 -6.11439005e-05
 -6.57281831e-05  1.55990059e-04  3.35560336e-04 -2.11609469e-04
 -9.85426206e-06  1.06138199e-04 -1.19714931e-04  3.10542971e-04
  0.00000000e+00  1.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.198818391012406, tolerance: 0.030091418941377436
  positive)


In [329]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)
jjin_zero_index = np.where(variables == 0)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])
    
impovar

['총 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '여성연령대_60_이상_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_

In [330]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata2
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year'] != 2019)]
    test=data_frame[data_frame['year'] == 2019]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.060179537014385, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.601928210872412, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23.000844468682402, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not 

계수

In [331]:
variables

array([-0.00170474,  0.00335228])

중요 변수

In [332]:
impovar

['year', '총 상주인구 수']

#### #중구

In [333]:
rawdata_junggu = rawdata[rawdata['sigungu']==11140]
rawdata_junggu

,year,quarter,sigungu,dong,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
0,2015,1,11140,11140540,11152009251,657,376,281,35,55,...,294,251,112,3,0,17,85,101,53,4
1,2015,1,11140,11140550,99170342338,964,353,611,0,55,...,42,36,14,0,0,5,27,22,6,0
2,2015,1,11140,11140580,9889692667,2881,1575,1306,123,664,...,704,625,551,33,0,107,508,485,496,67
3,2015,1,11140,11140605,26217489487,6,0,6,0,0,...,143,131,89,7,0,59,183,152,62,3
4,2015,1,11140,11140615,8372678394,1750,1085,665,34,219,...,772,582,476,29,0,116,340,365,471,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2019,2,11140,11140635,6356292769,697,329,368,93,132,...,234,203,267,117,184,357,368,246,203,95
313,2019,2,11140,11140645,1597285285,4109,1938,2171,455,703,...,1210,1056,1324,690,1370,2014,1771,1147,1144,595
314,2019,2,11140,11140650,8769325536,3440,1717,1723,355,546,...,451,429,556,275,422,691,738,478,415,194
315,2019,2,11140,11140665,2521616514,3458,1678,1780,427,515,...,1133,1050,1438,758,1047,1744,1612,1077,1147,692


In [334]:
rawdata2_junggu = pd.get_dummies(rawdata_junggu, columns=['quarter','sigungu','dong'], prefix=['quarter','sigungu','dong'])

In [335]:
import numpy as np
rawdata2_junggu["Logamt"] = np.log1p(rawdata2_junggu["amt"])
rawdata2_junggu = rawdata2_junggu.drop(['amt'],axis=1)
rawdata2_junggu.head()

,year,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,dong_11140580,dong_11140605,dong_11140615,dong_11140625,dong_11140635,dong_11140645,dong_11140650,dong_11140665,dong_11140670,Logamt
0,2015,657,376,281,35,55,80,119,194,174,...,0,0,0,0,0,0,0,0,0,23.134886
1,2015,964,353,611,0,55,171,244,317,177,...,0,0,0,0,0,0,0,0,0,25.320105
2,2015,2881,1575,1306,123,664,615,600,611,268,...,1,0,0,0,0,0,0,0,0,23.014759
3,2015,6,0,6,0,0,0,0,6,0,...,0,1,0,0,0,0,0,0,0,23.989693
4,2015,1750,1085,665,34,219,224,610,606,57,...,0,0,1,0,0,0,0,0,0,22.848240


In [336]:
columnname = rawdata2_junggu.columns
columnname = pd.DataFrame(columnname)
columnname.shape[0]

631

칼럼을 모두 가져오는 column_array

In [337]:
column_array = []
for i in range(columnname.shape[0]):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()
column_array

['year',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_

In [338]:
data_frame = rawdata2_junggu
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year'] != 2019)]
test=data_frame[data_frame['year'] == 2019]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.637624672329251,  0.7212332523242557,  0.8027135657815259, [-0.00000000e+00 -1.35548032e-04 -0.00000000e+00 -0.00000000e+00
 -1.69669475e-04 -0.00000000e+00 -1.56525620e-04  7.37457715e-04
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  7.27320283e-08 -6.65329997e-11  7.44860987e-11
 -5.77587965e-11 -1.85794861e-10  5.38270751e-11 -2.41119348e-10
 -6.73529885e-10  4.25010088e-09 -6.18063668e-10  1.99724892e-09
  3.80872191e-06 -2.60364757e-04  1.76868047e-05 -0.00000000e+00
  3.91571149e-04  0.00000000e+00  0.00000000e+00  4.27874875e-04
 -6.49666614e-04  3.48170594e-04 -2.79201465e-04 -1.69325702e-04
  0.00000000e+00  0.00000000e+00 -8.13951537e-10 -5.84592030e-05
 -3.49277107e-05  8.19597682e-05  1.43560352e-04 -1.17648566e-04
 -3.00390363e-06  7.97376846e-05 -1.35406704e-04  1.39955731e-04
  0.00000000e+00  2.8

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.466366938302549, tolerance: 0.01753036052775323
  positive)


In [339]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)
jjin_zero_index = np.where(variables == 0)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])
    
impovar

['총 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대

In [340]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata2_junggu
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year'] != 2019)]
    test=data_frame[data_frame['year'] == 2019]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.435341249974204, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.324678923236177, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.036056292279627, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not conver

계수

In [341]:
variables

array([-0.0009219 ,  0.00143855])

중요 변수

In [342]:
impovar

['year', '총 상주인구 수']

#### #노원구

In [343]:
rawdata_nowon = rawdata[rawdata['sigungu']==11350]
rawdata_nowon

,year,quarter,sigungu,dong,amt,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,...,여성연령대_60_이상_토요일시간대_3_유동인구_수,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수
11,2015,1,11350,11350560,6106374871,818,437,381,81,137,...,383,284,387,65,0,171,275,295,412,61
12,2015,1,11350,11350595,10682998200,5769,2868,2901,876,1051,...,583,456,560,66,0,223,469,468,627,50
13,2015,1,11350,11350611,708022569,1826,879,947,381,311,...,46,33,32,2,0,14,32,36,37,2
14,2015,1,11350,11350624,7511669918,1604,797,807,333,197,...,642,497,583,75,0,317,537,507,612,72
15,2015,1,11350,11350630,3505036944,1827,906,921,337,277,...,261,204,247,27,0,121,222,229,268,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2019,2,11350,11350611,1354416123,1523,759,764,258,330,...,159,161,194,94,203,264,219,216,230,98
320,2019,2,11350,11350624,7500392817,1815,926,889,327,251,...,1033,894,1470,813,1157,934,972,872,1371,691
321,2019,2,11350,11350630,4787971115,2506,1237,1269,315,413,...,719,695,823,255,283,398,643,623,737,221
322,2019,2,11350,11350640,26976275176,7366,3723,3643,923,1173,...,2651,2660,3298,1637,2725,3163,2959,2439,2898,1461


In [344]:
rawdata2_nowon= pd.get_dummies(rawdata_nowon, columns=['quarter','sigungu','dong'], prefix=['quarter','sigungu','dong'])

In [345]:
import numpy as np
rawdata2_nowon["Logamt"] = np.log1p(rawdata2_nowon["amt"])
rawdata2_nowon = rawdata2_nowon.drop(['amt'],axis=1)
rawdata2_nowon.head()

,year,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,quarter_4,sigungu_11350,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
11,2015,818,437,381,81,137,188,160,171,81,...,0,1,1,0,0,0,0,0,0,22.532599
12,2015,5769,2868,2901,876,1051,1106,957,1015,764,...,0,1,0,1,0,0,0,0,0,23.091919
13,2015,1826,879,947,381,311,223,382,352,177,...,0,1,0,0,1,0,0,0,0,20.377987
14,2015,1604,797,807,333,197,229,369,254,222,...,0,1,0,0,0,1,0,0,0,22.739724
15,2015,1827,906,921,337,277,283,348,349,233,...,0,1,0,0,0,0,1,0,0,21.977467


In [346]:
columnname = rawdata2_nowon.columns
columnname = pd.DataFrame(columnname)
columnname.shape[0]

627

In [347]:
column_array = []
for i in range(columnname.shape[0]):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()
column_array

['year',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_

In [348]:
data_frame = rawdata2_nowon
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year'] != 2019)]
test=data_frame[data_frame['year'] == 2019]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.6019179294043653,  0.6388789861452431,  0.6719233862673895, [-0.00000000e+00 -2.83092169e-05 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -1.76914482e-07 -1.58204296e-11 -9.30587099e-11
  2.77092116e-10  2.18009429e-10 -3.70610683e-10  4.38476508e-10
 -1.66822727e-09  8.12643495e-10 -1.19717203e-11  5.51213523e-10
  0.00000000e+00 -1.59441489e-04  7.55820260e-05  5.55085311e-04
 -0.00000000e+00  0.00000000e+00 -2.94930099e-04  1.36223277e-04
  0.00000000e+00  2.76975118e-05 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.79709584e-08 -4.04142665e-05
  0.00000000e+00 -7.11996890e-05  0.00000000e+00 -1.02892395e-03
 -1.02036512e-03  4.16328856e-06  3.52100232e-04  5.62738351e-04
  0.00000000e+00 -0.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.250746993309174, tolerance: 0.011453186703177914
  positive)


In [349]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
zero_index = list(zero_index)
jjin_zero_index = np.where(variables == 0)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])
    
impovar

['총 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_유동인구_수',
 '시간대_4_유동인구_수',
 '시간대_5_유동인구_수',
 '시간대_6_유동인구_수',
 '월요일_유동인구_수',
 '화요일_유동인구_수',
 '수요일_유동인구_수',
 '목요일_유동인구_수',
 '금요일_유동인구_수',
 '토요일_유동인구_수',
 '일요일_유동인구_수',
 '남성연령대_10_토요일시간대_5_유동인구_수',
 '남성연령대_20_토요일시간대_1_유동인구_수',
 '남성연령대_20_일요일시간대_2_유동인구_수',
 '남성연령대_40_월요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_5_유동인구_수',
 '남성연령대_40_화요일시간대_6_유

In [350]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata2_nowon
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year'] != 2019)]
    test=data_frame[data_frame['year'] == 2019]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2565756573646902, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6174039281997592, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.626264449390397, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not c

계수

In [351]:
variables

array([ 0.00045472,  0.00396188, -0.00812093,  0.00056575,  0.00461872,
        0.00572772,  0.00079508])

변수

In [352]:
impovar

['year',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수']

### Lasso Regression (2) 정리

#### 1. 전체
계수: [-0.00170474,  0.00335228]

변수: ['year', '총 상주인구 수'] 


#### 2. 중구
계수: [-0.0009219 ,  0.00143855]

변수: ['year', '총 상주인구 수']
     
     
#### 3. 노원구
계수: [ 0.00045472,  0.00396188, -0.00812093,  0.00056575,  0.00461872, 0.00572772,  0.00079508]
     
변수: ['year', '총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수','연령대 10 상주인구 수','연령대 20 상주인구 수', '연령대 30 상주인구 수']

### Lasso Regression(2)의 변수들을 이용한 regression

In [353]:
def predict_columns(data_frame, x_cols, y_cols):
    train=data_frame[data_frame['year'] != 2019]
    test=data_frame[data_frame['year'] == 2019]
    reg_models = [LinearRegression(),Ridge(), Lasso(), ElasticNet(), GradientBoostingRegressor(), SVR(), XGBRegressor()]
    reg_model_names=['LinearRegression','Ridge', "Lasso", "ElasticNet", "GradientBoositng", " SupportVector", "XGBoost"]
    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        print('{} : {},  {},  {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse))

In [355]:
#전체데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
predict_columns(rawdata2,['year', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수'],'Logamt')

LinearRegression : 0.9598927016910935,  -94.68640737223774,  1.0517811378417248
Ridge : 0.9598927016910935,  -95.02966403897274,  1.051754233433702
Lasso : 1.0743848212595104,  -115.3965297257124,  1.0374553018647408
ElasticNet : 1.0743848212595104,  -114.89157107836809,  1.0374484151347703
GradientBoositng : 0.5758646153492257,  0.7274332039962721,  0.41332527424438337
 SupportVector : 0.8410850132499733,  -2.697995729097732,  0.8856040552660961
XGBoost : 0.5176683934234186,  0.9209454496803228,  0.26650654280366354


In [356]:
#중구데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
predict_columns(rawdata2_junggu,['year', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수'],'Logamt')

LinearRegression : 0.8947304512396833,  -2.6843470948765837,  0.953603019233551
Ridge : 0.8947304512396833,  -2.6861844077329136,  0.9534811957391267
Lasso : 0.9810187913014011,  -2.710759238622231,  0.9293703627321898
ElasticNet : 0.9810187913014011,  -2.707185231607725,  0.9293600371508077
GradientBoositng : 0.5900548407715526,  0.7964053759892727,  0.37500062113201804
 SupportVector : 0.8901097003923102,  -1.3120254166539183,  0.8047518056554329
XGBoost : 0.40679665276857224,  0.8407175477045259,  0.33789872748470756


In [357]:
#노원구데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
predict_columns(rawdata2_nowon,['year', '총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수','연령대 10 상주인구 수','연령대 20 상주인구 수', '연령대 30 상주인구 수'],'Logamt')

LinearRegression : 0.741712111764193,  -0.05147831655494528,  0.7324563624475391
Ridge : 0.741712111764193,  -0.048691734250806196,  0.7318802123040127
Lasso : 0.6249942611141408,  -0.2955705558695876,  0.6883918759604348
ElasticNet : 0.6249942611141408,  -0.28318669333564084,  0.6848815543943685
GradientBoositng : 0.39153064953780614,  0.9508210904419644,  0.22191104086232674
 SupportVector : 0.7640462191691582,  -1.1306136167665253,  0.7816205152271747


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23.026176478399098, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.408858713623157, tolerance: 0.011453186703177914
  positive)


XGBoost : 0.39153064953780614,  0.9470376877726523,  0.2304701513605181


### 3) Lasso Regression - year만 object인 경우

#### #전지역 

In [358]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

rawdata = pd.read_excel(path+'finaldata.xlsx', encoding = 'euc_kr')
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)

In [359]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [360]:
#MAE
def mae(y,yhat):
    return np.mean(abs(y - yhat))

In [361]:
rawdata3 = rawdata.copy()
rawdata3[['year','sigungu','dong']] = rawdata3[['year','sigungu','dong']].astype('object')
rawdata3.dtypes

year                           object
quarter                         int64
sigungu                        object
dong                           object
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

In [362]:
rawdata3 = pd.get_dummies(rawdata3, columns=['year','sigungu','dong'], prefix=['year','sigungu','dong'])

In [363]:
import numpy as np
rawdata3["Logamt"] = np.log1p(rawdata3["amt"])
rawdata3 = rawdata3.drop(['amt'],axis=1)
rawdata3.head()

,quarter,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,dong_11140665,dong_11140670,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
0,1,657,376,281,35,55,80,119,194,174,...,0,0,0,0,0,0,0,0,0,23.134886
1,1,964,353,611,0,55,171,244,317,177,...,0,0,0,0,0,0,0,0,0,25.320105
2,1,2881,1575,1306,123,664,615,600,611,268,...,0,0,0,0,0,0,0,0,0,23.014759
3,1,6,0,6,0,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,23.989693
4,1,1750,1085,665,34,219,224,610,606,57,...,0,0,0,0,0,0,0,0,0,22.848240


In [364]:
columnname = rawdata3.columns
columnname = pd.DataFrame(columnname)
columnname.shape[0]

640

In [365]:
column_array = []
for i in range(columnname.shape[0]):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()
column_array

['quarter',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연

In [366]:
data_frame = rawdata3
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7537845704249974,  0.6598234634488311,  0.9136377031306749, [ 0.00000000e+00 -1.40972049e-04  0.00000000e+00 -1.03380073e-05
 -1.46043069e-03 -8.48412964e-04  1.38222514e-03  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  1.54860510e-03  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -1.27499560e-07  2.11003898e-11  9.71387089e-11
 -6.34077869e-11  1.03439904e-09 -1.67175332e-09  2.34117438e-10
  1.20878722e-09  7.45448824e-09 -9.73692853e-10 -7.38814884e-11
  0.00000000e+00 -0.00000000e+00  1.90529700e-05 -1.99330405e-04
 -0.00000000e+00  0.00000000e+00 -1.38810983e-04  4.24316188e-05
  1.00998541e-04  2.18230479e-04 -5.09598313e-04 -3.00541979e-05
 -4.51081722e-04 -0.00000000e+00 -2.65929069e-10 -6.11439005e-05
 -6.57281831e-05  1.55990059e-04  3.35560336e-04 -2.11609469e-04
 -9.85426206e-06  1.06138199e-04 -1.19714931e-04  3.10542971e-04
  0.00000000e+00  1.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.198818391012406, tolerance: 0.030091418941377436
  positive)


In [367]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
jjin_zero_index = np.where(abs(variables) == 0)
zero_index = list(zero_index)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])
    
impovar

['총 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '여성연령대_60_이상_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_

In [370]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata3
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39.41852714544927, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39.65459222300166, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 71.54923093510212, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not con

계수

In [371]:
variables

array([ 0.00067039,  0.00155308, -0.00261123, -0.00280639,  0.00050741,
        0.00034721, -0.0009334 , -0.00042385,  0.00687425, -0.00443119])

중요 변수

In [372]:
impovar

['quarter',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수']

#### #중구

In [373]:
rawdata3_junggu = rawdata_junggu.copy()
rawdata3_junggu[['year','sigungu','dong']] = rawdata3_junggu[['year','sigungu','dong']].astype('object')
rawdata3_junggu.dtypes

year                           object
quarter                         int64
sigungu                        object
dong                           object
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

In [374]:
rawdata3_junggu = pd.get_dummies(rawdata3_junggu, columns=['year','sigungu','dong'], prefix=['year','sigungu','dong'])

In [375]:
import numpy as np
rawdata3_junggu["Logamt"] = np.log1p(rawdata3_junggu["amt"])
rawdata3_junggu = rawdata3_junggu.drop(['amt'],axis=1)
rawdata3_junggu.head()

,quarter,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,dong_11140580,dong_11140605,dong_11140615,dong_11140625,dong_11140635,dong_11140645,dong_11140650,dong_11140665,dong_11140670,Logamt
0,1,657,376,281,35,55,80,119,194,174,...,0,0,0,0,0,0,0,0,0,23.134886
1,1,964,353,611,0,55,171,244,317,177,...,0,0,0,0,0,0,0,0,0,25.320105
2,1,2881,1575,1306,123,664,615,600,611,268,...,1,0,0,0,0,0,0,0,0,23.014759
3,1,6,0,6,0,0,0,0,6,0,...,0,1,0,0,0,0,0,0,0,23.989693
4,1,1750,1085,665,34,219,224,610,606,57,...,0,0,1,0,0,0,0,0,0,22.848240


In [376]:
columnname = rawdata3_junggu.columns
columnname = pd.DataFrame(columnname)
columnname.shape


column_array = []
for i in range(columnname.shape[0]):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()
column_array

['quarter',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연

In [377]:
data_frame = rawdata3_junggu
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.637624672329251,  0.7212332523242557,  0.8027135657815259, [ 0.00000000e+00 -1.35548032e-04 -0.00000000e+00 -0.00000000e+00
 -1.69669475e-04 -0.00000000e+00 -1.56525620e-04  7.37457715e-04
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  7.27320283e-08 -6.65329997e-11  7.44860987e-11
 -5.77587965e-11 -1.85794861e-10  5.38270751e-11 -2.41119348e-10
 -6.73529885e-10  4.25010088e-09 -6.18063668e-10  1.99724892e-09
  3.80872191e-06 -2.60364757e-04  1.76868047e-05 -0.00000000e+00
  3.91571149e-04  0.00000000e+00  0.00000000e+00  4.27874875e-04
 -6.49666614e-04  3.48170594e-04 -2.79201465e-04 -1.69325702e-04
  0.00000000e+00  0.00000000e+00 -8.13951537e-10 -5.84592030e-05
 -3.49277107e-05  8.19597682e-05  1.43560352e-04 -1.17648566e-04
 -3.00390363e-06  7.97376846e-05 -1.35406704e-04  1.39955731e-04
  0.00000000e+00  2.8

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.466366938302549, tolerance: 0.01753036052775323
  positive)


In [378]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
jjin_zero_index = np.where(abs(variables) == 0)
zero_index = list(zero_index)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])

In [379]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata3_junggu
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.435341249974204, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.324678923236177, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.036056292279627, tolerance: 0.01753036052775323
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not conver

계수

In [380]:
variables

array([-0.0009219 ,  0.00143855])

중요 변수

In [381]:
impovar

['quarter', '총 상주인구 수']

#### #노원구

In [390]:
rawdata3_nowon = rawdata_nowon.copy()
rawdata3_nowon[['year','sigungu','dong']] = rawdata3_nowon[['year','sigungu','dong']].astype('object')
rawdata3_nowon.dtypes

year                           object
quarter                         int64
sigungu                        object
dong                           object
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

In [391]:
#MAE
def mae(y,yhat):
    return np.mean(abs(y - yhat))

In [392]:
rawdata3_nowon = pd.get_dummies(rawdata3_nowon, columns=['year','sigungu','dong'], prefix=['year','sigungu','dong'])

In [393]:
import numpy as np
rawdata3_nowon["Logamt"] = np.log1p(rawdata3_nowon["amt"])
rawdata3_nowon = rawdata3_nowon.drop(['amt'],axis=1)
rawdata3_nowon.head()

,quarter,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,...,year_2019,sigungu_11350,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
11,1,818,437,381,81,137,188,160,171,81,...,0,1,1,0,0,0,0,0,0,22.532599
12,1,5769,2868,2901,876,1051,1106,957,1015,764,...,0,1,0,1,0,0,0,0,0,23.091919
13,1,1826,879,947,381,311,223,382,352,177,...,0,1,0,0,1,0,0,0,0,20.377987
14,1,1604,797,807,333,197,229,369,254,222,...,0,1,0,0,0,1,0,0,0,22.739724
15,1,1827,906,921,337,277,283,348,349,233,...,0,1,0,0,0,0,1,0,0,21.977467


In [394]:
columnname = rawdata3_nowon.columns
columnname = pd.DataFrame(columnname)
columnname.shape

column_array = []
for i in range(columnname.shape[0]):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()
column_array


['quarter',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연

In [395]:
data_frame = rawdata3_nowon
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year_2019'] != 1)]
test=data_frame[data_frame['year_2019'] == 1]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.6019179294043653,  0.6388789861452431,  0.6719233862673895, [ 0.00000000e+00 -2.83092169e-05 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -1.76914482e-07 -1.58204296e-11 -9.30587099e-11
  2.77092116e-10  2.18009429e-10 -3.70610683e-10  4.38476508e-10
 -1.66822727e-09  8.12643495e-10 -1.19717203e-11  5.51213523e-10
  0.00000000e+00 -1.59441489e-04  7.55820260e-05  5.55085311e-04
 -0.00000000e+00  0.00000000e+00 -2.94930099e-04  1.36223277e-04
  0.00000000e+00  2.76975118e-05 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.79709584e-08 -4.04142665e-05
  0.00000000e+00 -7.11996890e-05  0.00000000e+00 -1.02892395e-03
 -1.02036512e-03  4.16328856e-06  3.52100232e-04  5.62738351e-04
  0.00000000e+00 -0.

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.250746993309174, tolerance: 0.011453186703177914
  positive)


In [396]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
jjin_zero_index = np.where(abs(variables) == 0)
zero_index = list(zero_index)
jjin_zero_index = list(jjin_zero_index)

impovar = []
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])

In [398]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata3_nowon
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year_2019'] != 1)]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2565756573646902, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.6174039281997592, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.626264449390397, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not c

계수

In [399]:
variables

array([ 0.00045472,  0.00396188, -0.00812093,  0.00056575,  0.00461872,
        0.00572772,  0.00079508])

중요 변수

In [ ]:
impovar

### lasso regression (3) 정리

#### 1. 전체
계수: [ 0.00067039,  0.00155308, -0.00261123, -0.00280639,  0.00050741, 0.00034721, -0.0009334 , -0.00042385,  0.00687425, -0.00443119]

변수: ['quarter','총 상주인구 수','남성 상주인구 수','여성 상주인구 수','연령대 10 상주인구 수','연령대 20 상주인구 수','연령대 30 상주인구 수','연령대 40 상주인구 수','연령대 50 상주인구 수','연령대 60 이상 상주인구 수']



#### 2. 중구
계수: [-0.0009219 ,  0.00143855]

변수: ['quarter', '총 상주인구 수']

     
     
#### 3. 노원구
계수: [ 0.00045472,  0.00396188, -0.00812093,  0.00056575,  0.00461872, 0.00572772,  0.00079508]
     
변수: ['quarter','총 상주인구 수','남성 상주인구 수','여성 상주인구 수','연령대 10 상주인구 수','연령대 20 상주인구 수','연령대 30 상주인구 수']
 

### Lasso Regression(3)의 변수들을 이용한 regression

In [402]:
def predict_columns(data_frame, x_cols, y_cols):
    train=data_frame[data_frame['year_2019'] != 1]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [LinearRegression(),Ridge(), Lasso(), ElasticNet(), GradientBoostingRegressor(), SVR(), XGBRegressor()]
    reg_model_names=['LinearRegression','Ridge', "Lasso", "ElasticNet", "GradientBoositng", " SupportVector", "XGBoost"]
    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        print('{} : {},  {},  {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse))

In [403]:
#전체데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
predict_columns(rawdata3,['year_2019','year_2018', 'year_2017','year_2016','year_2015','quarter','총 상주인구 수','남성 상주인구 수','여성 상주인구 수','연령대 10 상주인구 수','연령대 20 상주인구 수','연령대 30 상주인구 수','연령대 40 상주인구 수','연령대 50 상주인구 수','연령대 60 이상 상주인구 수'],'Logamt')

LinearRegression : 0.8142195409567529,  -12.981427275259172,  0.9374412193185337
Ridge : 0.8142195409567529,  -12.992888281273085,  0.9375334279294986
Lasso : 0.7864417631789751,  -21.4232636790219,  0.9638222273941315
ElasticNet : 0.7864417631789751,  -18.712390475454693,  0.9570251086926163
GradientBoositng : 0.4002589771000702,  0.9476820948564201,  0.2222028464515473
 SupportVector : 0.8410850132499733,  -3.0896491485600492,  0.8474960074255103


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 67.32215993830899, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 97.04696334647875, tolerance: 0.030091418941377436
  positive)


XGBoost : 0.39645434064883617,  0.9577565829623946,  0.21153275653399664


In [404]:
#중구데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
predict_columns(rawdata3_junggu,['year_2019','year_2018', 'year_2017','year_2016','year_2015','quarter', '총 상주인구 수'],'Logamt')

LinearRegression : 0.9810187913014011,  -2.518842917022728,  0.9304702131080155
Ridge : 0.9810187913014011,  -2.5224458699249612,  0.9304630845204631
Lasso : 0.9810187913014011,  -2.710759238622231,  0.9293703627321898
ElasticNet : 0.9810187913014011,  -2.707185231607725,  0.9293600371508077
GradientBoositng : 0.6297109137164836,  0.7382633784401769,  0.4009455190610845
 SupportVector : 0.8901097003923102,  -1.2857793750297342,  0.8034900464413965
XGBoost : 0.5388018228073927,  0.8716643698404054,  0.296335442350547


In [405]:
#노원구데이터로 Lasso regression에서 구한 변수들과 year, quarter 변수 포함해서 회귀
predict_columns(rawdata3_nowon,['year_2019','year_2018', 'year_2017','year_2016','year_2015','quarter','총 상주인구 수','남성 상주인구 수','여성 상주인구 수','연령대 10 상주인구 수','연령대 20 상주인구 수','연령대 30 상주인구 수'],'Logamt')

LinearRegression : 0.6249942611141408,  -0.031153920465750273,  0.7143638041235213
Ridge : 0.6249942611141408,  -0.018393552443740413,  0.7127102929869338
Lasso : 0.6249942611141408,  -0.2955705558695825,  0.6883918759604343
ElasticNet : 0.6249942611141408,  -0.2831866933356384,  0.6848815543943682
GradientBoositng : 0.45271600391439193,  0.9607676321463998,  0.20045247269170177
 SupportVector : 0.7640462191691582,  -1.2656394938975528,  0.8047175837778922


C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23.026176478399098, tolerance: 0.011453186703177914
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.408858713623157, tolerance: 0.011453186703177914
  positive)


XGBoost : 0.39153064953780614,  0.96161430851011,  0.1904119000348615


### 4) Lasso Regression - year, quarter 모두 숫자

In [406]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

rawdata = pd.read_excel(path+'finaldata.xlsx', encoding = 'euc_kr')
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)

In [407]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [408]:
#MAE
def mae(y,yhat):
    return np.mean(abs(y - yhat))

In [409]:
rawdata4 = rawdata.copy()
rawdata4.dtypes

year                           int64
quarter                        int64
sigungu                        int64
dong                           int64
amt                            int64
                               ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수    int64
여성연령대_60_이상_일요일시간대_3_유동인구_수    int64
여성연령대_60_이상_일요일시간대_4_유동인구_수    int64
여성연령대_60_이상_일요일시간대_5_유동인구_수    int64
여성연령대_60_이상_일요일시간대_6_유동인구_수    int64
Length: 618, dtype: object

In [410]:
import numpy as np
rawdata4["Logamt"] = np.log1p(rawdata4["amt"])
rawdata4 = rawdata4.drop(['amt'],axis=1)
rawdata4.head()

,year,quarter,sigungu,dong,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,...,여성연령대_60_이상_토요일시간대_4_유동인구_수,여성연령대_60_이상_토요일시간대_5_유동인구_수,여성연령대_60_이상_토요일시간대_6_유동인구_수,여성연령대_60_이상_일요일시간대_1_유동인구_수,여성연령대_60_이상_일요일시간대_2_유동인구_수,여성연령대_60_이상_일요일시간대_3_유동인구_수,여성연령대_60_이상_일요일시간대_4_유동인구_수,여성연령대_60_이상_일요일시간대_5_유동인구_수,여성연령대_60_이상_일요일시간대_6_유동인구_수,Logamt
0,2015,1,11140,11140540,657,376,281,35,55,80,...,251,112,3,0,17,85,101,53,4,23.134886
1,2015,1,11140,11140550,964,353,611,0,55,171,...,36,14,0,0,5,27,22,6,0,25.320105
2,2015,1,11140,11140580,2881,1575,1306,123,664,615,...,625,551,33,0,107,508,485,496,67,23.014759
3,2015,1,11140,11140605,6,0,6,0,0,0,...,131,89,7,0,59,183,152,62,3,23.989693
4,2015,1,11140,11140615,1750,1085,665,34,219,224,...,582,476,29,0,116,340,365,471,178,22.848240


In [411]:
columnname = rawdata4.columns
columnname = pd.DataFrame(columnname)
columnname.shape

column_array = []
for i in range(columnname.shape[0]):
    column_array = np.append(column_array, columnname.iloc[[i]])
    
column_array = column_array.tolist()
column_array #column name list

['year',
 'quarter',
 'sigungu',
 'dong',
 '총 상주인구 수',
 '남성 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 20 상주인구 수',
 '연령대 30 상주인구 수',
 '연령대 40 상주인구 수',
 '연령대 50 상주인구 수',
 '연령대 60 이상 상주인구 수',
 '남성연령대 10 상주인구 수',
 '남성연령대 20 상주인구 수',
 '남성연령대 30 상주인구 수',
 '남성연령대 40 상주인구 수',
 '남성연령대 50 상주인구 수',
 '남성연령대 60 이상 상주인구 수',
 '여성연령대 10 상주인구 수',
 '여성연령대 20 상주인구 수',
 '여성연령대 30 상주인구 수',
 '여성연령대 40 상주인구 수',
 '여성연령대 50 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_면적_99_제곱미터_세대_수',
 '아파트_면적_132_제곱미터_세대_수',
 '아파트_면적_165_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_가격_6_억_이상_세대_수',
 '아파트_평균_면적',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장

In [412]:
data_frame = rawdata4
#x_cols: 설명변수, y_cols: 반응변수
x_cols  = column_array[0:len(column_array)-1]
y_cols  = 'Logamt' 
train=data_frame[(data_frame['year'] != 2019)]
test=data_frame[data_frame['year'] == 2019]
reg_models = [Lasso()]
reg_model_names=["Lasso"]

for i in range(len(reg_models)):
    reg = reg_models[i].fit(train[x_cols],train[y_cols])
    pred = reg_models[i].predict(test[x_cols])
    r2 = r2_score(pred,test[y_cols])
    rmse = mean_squared_error(pred, test[y_cols])**0.5
    variables = reg.coef_
    print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))

Lasso : 0.7663972148327077,  0.7304766949969105,  0.773028055807735, [-0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.22732943e-06
 -7.82468445e-05  0.00000000e+00 -8.01488144e-05 -1.34374111e-03
 -0.00000000e+00  8.97703562e-04  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  1.92680452e-08  3.59546936e-12  7.61073158e-11 -7.16990940e-11
  3.17244903e-10 -8.30865187e-10  2.87395146e-11  4.23903469e-10
  4.92968147e-09 -2.67017952e-10 -4.37775228e-10  7.03799459e-04
 -1.42205542e-04  1.38417494e-04  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -1.81330582e-04  1.40021891e-04  2.16068622e-04
  2.79655748e-05 -6.84011717e-04 -1.13272728e-04 -0.00000000e+00
  0.00000000e+00 -2.09523976e-09 -5.76768793e-05 -4.96831457e-05
  1.17036615e-04  3.79370499e-04 -1.71002964e-04 -5.85896874e-06
  9.36014577e-05 -1.2

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.897513494014568, tolerance: 0.030091418941377436
  positive)


In [413]:
zero_index = np.where(abs(variables) != 0)  #계수가 0이 아닌 변수들의 index 모음
jjin_zero_index = np.where(abs(variables) == 0)
zero_index = list(zero_index)
jjin_zero_index = list(jjin_zero_index)

impovar = [] #important variables
for i in range(len(zero_index[0])):
    index = zero_index[0][i]
    impovar.append(column_array[index])

impovar

['dong',
 '총 상주인구 수',
 '여성 상주인구 수',
 '연령대 10 상주인구 수',
 '연령대 30 상주인구 수',
 '월_평균_소득_금액',
 '지출_총금액',
 '식료품_지출_총금액',
 '의류_신발_지출_총금액',
 '생활용품_지출_총금액',
 '의료비_지출_총금액',
 '교통_지출_총금액',
 '여가_지출_총금액',
 '문화_지출_총금액',
 '교육_지출_총금액',
 '유흥_지출_총금액',
 '아파트_단지_수',
 '아파트_면적_66_제곱미터_미만_세대_수',
 '아파트_면적_66_제곱미터_세대_수',
 '아파트_가격_1_억_미만_세대_수',
 '아파트_가격_1_억_세대_수',
 '아파트_가격_2_억_세대_수',
 '아파트_가격_3_억_세대_수',
 '아파트_가격_4_억_세대_수',
 '아파트_가격_5_억_세대_수',
 '아파트_평균_시가',
 '총_직장_인구_수',
 '남성_직장_인구_수',
 '여성_직장_인구_수',
 '연령대_10_직장_인구_수',
 '연령대_20_직장_인구_수',
 '연령대_30_직장_인구_수',
 '연령대_40_직장_인구_수',
 '연령대_50_직장_인구_수',
 '연령대_60_이상_직장_인구_수',
 '남성연령대_20_직장_인구_수',
 '남성연령대_30_직장_인구_수',
 '남성연령대_40_직장_인구_수',
 '남성연령대_50_직장_인구_수',
 '남성연령대_60_이상_직장_인구_수',
 '여성연령대_10_직장_인구_수',
 '여성연령대_20_직장_인구_수',
 '여성연령대_30_직장_인구_수',
 '여성연령대_40_직장_인구_수',
 '여성연령대_50_직장_인구_수',
 '총_유동인구_수',
 '남성_유동인구_수',
 '여성_유동인구_수',
 '연령대_10_유동인구_수',
 '연령대_20_유동인구_수',
 '연령대_30_유동인구_수',
 '연령대_40_유동인구_수',
 '연령대_50_유동인구_수',
 '연령대_60_이상_유동인구_수',
 '시간대_1_유동인구_수',
 '시간대_2_유동인구_수',
 '시간대_3_

In [414]:
while len(jjin_zero_index[0]) != 0:

    #lasso 회귀
    data_frame = rawdata4
    #x_cols: 설명변수, y_cols: 반응변수
    x_cols  = impovar
    y_cols  = 'Logamt' 
    train=data_frame[(data_frame['year'] != 2019)]
    test=data_frame[data_frame['year'] == 2019]
    reg_models = [Lasso()]
    reg_model_names=["Lasso"]

    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        variables = reg.coef_
        #print('{} : {},  {},  {}, {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse,variables))


    #변수 출력
    zero_index = np.where(variables != 0)  #계수가 0이 아닌 변수들의 index 모음
    jjin_zero_index = np.where(variables == 0)
    zero_index = list(zero_index)
    jjin_zero_index = list(jjin_zero_index)

    impovar = []
    for i in range(len(zero_index[0])):
        index = zero_index[0][i]
        impovar.append(column_array[index])
    
    if len(jjin_zero_index[0]) == 0:
        break

C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.82249881392733, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.889346539501474, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21.343908724300285, tolerance: 0.030091418941377436
  positive)
C:\Users\Yoonhee\Anaconda3\envs\yoonhee\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not c

ValueError: at least one array or dtype is required

0이 아닌 계수가 존재하지 않는다

In [415]:
variables

array([0.])

중요 변수 또한 존재하지 않는다

In [416]:
impovar

[]

# 2. Final Regression

### year, quarter 모두 object인 데이터 이용해서 하는게 좋음, XGBoost와 Gradient Boost 모두 적합해보기

* Gradient Boosting

In [23]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

rawdata = pd.read_excel(path+'finaldata.xlsx', encoding = 'euc_kr')
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)

데이터에 존재하는 년도 print

In [25]:
rawdata['year'].unique()

array([2015, 2016, 2017, 2018, 2019], dtype=int64)

가장 좋은 모델이 year과 quarter 모두 object인 모델이었기때문에 데이터타입을 바꿔준다

In [26]:
rawdata[['year','quarter']] = rawdata[['year','quarter']].astype('object')
rawdata.dtypes

year                           object
quarter                        object
sigungu                         int64
dong                            int64
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

In [27]:
data = pd.get_dummies(rawdata, columns=['year','quarter','sigungu','dong'], prefix=['year','quarter','sigungu','dong'])

amt 대신 logamt로 계산해서 저장

In [28]:
import numpy as np
data["Logamt"] = np.log1p(data["amt"])
data = data.drop(['amt'],axis=1)
data.head()

,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,남성연령대 10 상주인구 수,...,dong_11140665,dong_11140670,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
0,657,376,281,35,55,80,119,194,174,16,...,0,0,0,0,0,0,0,0,0,23.134886
1,964,353,611,0,55,171,244,317,177,0,...,0,0,0,0,0,0,0,0,0,25.320105
2,2881,1575,1306,123,664,615,600,611,268,79,...,0,0,0,0,0,0,0,0,0,23.014759
3,6,0,6,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,23.989693
4,1750,1085,665,34,219,224,610,606,57,14,...,0,0,0,0,0,0,0,0,0,22.848240


Gradient Boosting model에 적합

위에서 Lasso Regression(1)의 결과를 보았을때 중요 변수들과 관련된 데이터를 불러와 Gradient Boosting을 시행한다

In [765]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\raw데이터\\'

resident = pd.read_csv(path+'상주인구.csv', encoding = 'euc_kr') #상주인구
area=pd.read_csv(path+'상권영역.csv',encoding='euc_kr') #상권영역

path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'
amtdata = pd.read_csv(path+'train_test.csv', encoding = 'euc_kr')

In [767]:
resident=resident[['기준_년_코드','기준_분기_코드','상권_구분_코드','상권 코드','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']]

데이터를 merge시켜준다

In [769]:
testdata = pd.merge(resident,area, left_on = '상권 코드',right_on = '상권_코드', how ='left')
testdata = testdata.drop(['기준_년월_코드','상권_구분_코드_x','상권_구분_코드_명'],axis=1)
# testdata.columns
testdata = testdata.rename(columns = {'상권_구분_코드_y':'상권_구분_코드'})
testdata = testdata[['기준_년_코드','기준_분기_코드','시군구_코드','행정동_코드','상권_코드','상권_코드_명','상권_구분_코드','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수',
       '연령대 10 상주인구 수', '연령대 20 상주인구 수']]
testdata.head()

,기준_년_코드,기준_분기_코드,시군구_코드,행정동_코드,상권_코드,상권_코드_명,상권_구분_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수
0,2020,1,11680,11680580,1001496,강남 마이스 관광특구,U,24,16,8,6,2
1,2020,1,11710,11710566,1001495,잠실 관광특구,U,8221,3708,4513,822,1679
2,2020,1,11110,11110615,1001494,종로?청계 관광특구,U,1924,1274,650,49,195
3,2020,1,11140,11140590,1001493,동대문패션타운 관광특구,U,2427,1146,1281,169,505
4,2020,1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,U,1104,401,703,15,51


In [770]:
amtdata = amtdata.drop(['Unnamed: 0'],axis=1)

In [771]:
amtdata.tail()

,기준_년_코드,기준_분기_코드,시군구_코드,행정동_코드,당월_매출_금액
698,2020,1,11350,11350625,9.723491e+08
699,2020,1,11350,11350630,3.636089e+09
700,2020,1,11350,11350640,2.735657e+10
701,2020,1,11350,11350670,2.432913e+09
702,2020,1,11350,11350695,9.286891e+09


In [772]:
finaltestdata = pd.merge(testdata, amtdata, left_on = ['기준_년_코드','기준_분기_코드','시군구_코드','행정동_코드'], right_on = ['기준_년_코드','기준_분기_코드','시군구_코드','행정동_코드'], how = 'left')
finaltestdata.head(10)

,기준_년_코드,기준_분기_코드,시군구_코드,행정동_코드,상권_코드,상권_코드_명,상권_구분_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,당월_매출_금액
0,2020,1,11680,11680580,1001496,강남 마이스 관광특구,U,24,16,8,6,2,NaN
1,2020,1,11710,11710566,1001495,잠실 관광특구,U,8221,3708,4513,822,1679,NaN
2,2020,1,11110,11110615,1001494,종로?청계 관광특구,U,1924,1274,650,49,195,NaN
3,2020,1,11140,11140590,1001493,동대문패션타운 관광특구,U,2427,1146,1281,169,505,4.088451e+10
4,2020,1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,U,1104,401,703,15,51,1.748050e+11
5,2020,1,11170,11170650,1001491,이태원 관광특구,U,5176,2462,2714,366,699,NaN
6,2020,1,11740,11740685,1001490,길동복조리시장,R,3097,1540,1557,440,373,NaN
7,2020,1,11740,11740660,1001489,둔촌역전통시장,R,595,304,291,81,102,NaN
8,2020,1,11740,11740650,1001488,성내골목시장,R,1023,527,496,115,140,NaN
9,2020,1,11740,11740610,1001487,천호시장,R,90,52,38,1,7,NaN


In [773]:
finaltestdata1 = finaltestdata[finaltestdata['시군구_코드']==11140]
finaltestdata2 = finaltestdata[finaltestdata['시군구_코드']==11350]

In [774]:
finaltestdata = finaltestdata1.append(finaltestdata2)
finaltestdata.head()

,기준_년_코드,기준_분기_코드,시군구_코드,행정동_코드,상권_코드,상권_코드_명,상권_구분_코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,당월_매출_금액
3,2020,1,11140,11140590,1001493,동대문패션타운 관광특구,U,2427,1146,1281,169,505,4.088451e+10
4,2020,1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,U,1104,401,703,15,51,1.748050e+11
204,2020,1,11140,11140680,1001285,합동(새벽)시장,R,90,71,19,4,6,5.734713e+08
205,2020,1,11140,11140670,1001284,서울중앙시장,R,994,535,459,68,125,5.273552e+09
206,2020,1,11140,11140650,1001283,백학시장,R,818,427,391,77,99,7.978687e+09


In [775]:
finaltestdata['시군구_코드'].unique()

array([11140, 11350], dtype=int64)

In [776]:
finaltestdata.columns

Index(['기준_년_코드', '기준_분기_코드', '시군구_코드', '행정동_코드', '상권_코드', '상권_코드_명',
       '상권_구분_코드', '총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수',
       '연령대 20 상주인구 수', '당월_매출_금액'],
      dtype='object')

In [777]:
finaltestdata["Logamt"] = np.log1p(finaltestdata["당월_매출_금액"])

In [778]:
finaltestdata = finaltestdata.rename(columns = {'기준_년_코드':'year','기준_분기_코드':'quarter','시군구_코드':'sigungu','행정동_코드':'dong','당월_매출_금액':'amt' })
finaltestdata.columns

Index(['year', 'quarter', 'sigungu', 'dong', '상권_코드', '상권_코드_명', '상권_구분_코드',
       '총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수',
       'amt', 'Logamt'],
      dtype='object')

In [779]:
finaltestdata = pd.get_dummies(finaltestdata,columns = ['year','quarter'],prefix=['year','quarter'])

In [780]:
finaltestdata.columns

Index(['sigungu', 'dong', '상권_코드', '상권_코드_명', '상권_구분_코드', '총 상주인구 수',
       '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', 'amt',
       'Logamt', 'year_2014', 'year_2015', 'year_2016', 'year_2017',
       'year_2018', 'year_2019', 'year_2020', 'quarter_1', 'quarter_2',
       'quarter_3', 'quarter_4'],
      dtype='object')

In [781]:
finaltestdata = finaltestdata.dropna(axis=0)

In [782]:
finaltestdata.columns

Index(['sigungu', 'dong', '상권_코드', '상권_코드_명', '상권_구분_코드', '총 상주인구 수',
       '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수', 'amt',
       'Logamt', 'year_2014', 'year_2015', 'year_2016', 'year_2017',
       'year_2018', 'year_2019', 'year_2020', 'quarter_1', 'quarter_2',
       'quarter_3', 'quarter_4'],
      dtype='object')

Gradient Boosting 모델에 적합

In [783]:
x_train = finaltestdata[finaltestdata['year_2020'] != 1][['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019','year_2020', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']]
y_train = finaltestdata[finaltestdata['year_2020'] != 1]['Logamt']
reg1 = GradientBoostingRegressor()
reg1.fit(x_train, y_train)
print(reg1.score(x_train, y_train))

0.7276904175280521


예측된 Logamt 값

In [784]:
x_test = finaltestdata[finaltestdata['year_2020'] == 1][['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']]
y_predict = reg1.predict(x_test)
print(y_predict)

[23.32720215 26.19813428 21.33026288 22.92919654 23.21576124 22.89320343
 24.20394848 23.98734218 24.50063909 24.66501426 24.57561431 24.6123467
 23.6454572  24.68159056 24.64158648 24.34053598 24.6104522  23.07090058
 24.58423172 24.01044351 24.63096264 24.20394848 23.21576124 25.10794269
 23.65512094 25.46186808 23.37009027 24.52297999 23.11806266 23.20758139
 24.25313531 22.90134219 23.6269716  22.77588313 22.97872287 23.9699973
 23.21576124 22.43075715 22.67572937 23.06248817 24.11391783 25.54794831
 23.06438908 22.42753377 22.90975709 22.72514196 21.92436769 23.85793477
 22.74289471 22.22521491 23.03287874 22.77069067 22.93563286 22.53698143
 23.01166919 23.96112713 23.68170297 23.46786538 22.76205184 22.63562631
 22.32847652 22.56117298 22.4204076  22.3601321  23.61594496 23.41548799
 22.18827891 23.28705276 23.09590006 22.10818806]


In [785]:
y_predict = pd.DataFrame(y_predict)
type(y_predict)

pandas.core.frame.DataFrame

In [787]:
finaltestdata2020 = finaltestdata[finaltestdata['year_2020']==1]
finaltestdata2020 = finaltestdata2020[['sigungu','dong','상권_코드','상권_코드_명','Logamt']]
finaltestdata2020.columns

Index(['sigungu', 'dong', '상권_코드', '상권_코드_명', 'Logamt'], dtype='object')

In [790]:
finaltestdata2020.head()

,sigungu,dong,상권_코드,상권_코드_명,Logamt
3,11140,11140590,1001493,동대문패션타운 관광특구,24.434017
4,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,25.886937
204,11140,11140680,1001285,합동(새벽)시장,20.167218
205,11140,11140670,1001284,서울중앙시장,22.385970
206,11140,11140650,1001283,백학시장,22.800040


In [791]:
y_predict = y_predict.rename(columns = {0:'predict_logamt'})
y_predict.head()

,predict_logamt
0,23.327202
1,26.198134
2,21.330263
3,22.929197
4,23.215761


In [792]:
finaltestdata2020 = finaltestdata2020.reset_index(drop=True)
finaltestdata2020

,sigungu,dong,상권_코드,상권_코드_명,Logamt
0,11140,11140590,1001493,동대문패션타운 관광특구,24.434017
1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,25.886937
2,11140,11140680,1001285,합동(새벽)시장,20.167218
3,11140,11140670,1001284,서울중앙시장,22.385970
4,11140,11140650,1001283,백학시장,22.800040
...,...,...,...,...,...
65,11350,11350640,1000382,상계로12길,24.032222
66,11350,11350630,1000381,동일로242길,22.014174
67,11350,11350595,1000380,동일로192길,23.251698
68,11350,11350595,1000379,동일로173가길,23.251698


### Gradient Boosting 결과

In [793]:
result = pd.concat([finaltestdata2020,y_predict], axis=1)
result.head()

,sigungu,dong,상권_코드,상권_코드_명,Logamt,predict_logamt
0,11140,11140590,1001493,동대문패션타운 관광특구,24.434017,23.327202
1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,25.886937,26.198134
2,11140,11140680,1001285,합동(새벽)시장,20.167218,21.330263
3,11140,11140670,1001284,서울중앙시장,22.385970,22.929197
4,11140,11140650,1001283,백학시장,22.800040,23.215761


Log amt로 적합을 했기 때문에 exponential을 취해 amt로 다시 저장해준다

In [794]:
result["amt"] = np.exp(result["Logamt"])

In [795]:
result["predict_amt"] = np.exp(result["predict_logamt"])

In [796]:
result = result[['sigungu','dong','Logamt','predict_logamt','amt','predict_amt']]
result.head()

,sigungu,dong,Logamt,predict_logamt,amt,predict_amt
0,11140,11140590,24.434017,23.327202,4.088451e+10,1.351684e+10
1,11140,11140520,25.886937,26.198134,1.748050e+11,2.386191e+11
2,11140,11140680,20.167218,21.330263,5.734713e+08,1.834913e+09
3,11140,11140670,22.385970,22.929197,5.273552e+09,9.078697e+09
4,11140,11140650,22.800040,23.215761,7.978687e+09,1.209141e+10


In [797]:
result=result.groupby(['sigungu','dong']).sum()
result.head()

Logamt  predict_logamt           amt   predict_amt
sigungu dong                                                            
11140   11140520  129.434684      125.613480  8.740249e+11  5.159035e+11
        11140540   68.805753       70.488190  2.740149e+10  4.891499e+10
        11140550  124.329335      123.012424  3.148320e+11  2.819614e+11
        11140570   47.809303       47.025211  4.816019e+10  3.287157e+10
        11140580  114.285605      114.362691  4.223685e+10  4.416441e+10

ratio = ((예상된 amt)-(실제 amt))/(예상된 amt) * 100  : 어느 지역을 우선적으로 선정할지를 정하기 위해 ratio라는 새로운 변수를 만들어 예측한 매출에 비해 실제 매출이 얼만큼 나왔는지에 대한 퍼센트를 표현한다

In [798]:
result['ratio'] = (result['predict_amt']-result['amt'])/result['predict_amt']*100

In [799]:
result = result.sort_values('ratio',ascending = False)

In [800]:
result.head()

Logamt  predict_logamt           amt   predict_amt  \
sigungu dong                                                              
11350   11350625  20.695225       22.762052  9.723491e+08  7.681278e+09   
11140   11140635  67.158732       70.210728  1.582499e+10  5.327015e+10   
11350   11350611  20.898184       22.108188  1.191149e+09  3.994515e+09   
11140   11140680  20.167218       21.330263  5.734713e+08  1.834913e+09   
        11140665  42.830601       44.968110  3.995552e+09  1.202596e+10   

                      ratio  
sigungu dong                 
11350   11350625  87.341312  
11140   11140635  70.292946  
11350   11350611  70.180390  
11140   11140680  68.746676  
        11140665  66.775613

In [801]:
result.to_excel('gb_amtratio.xlsx')

* XGBoost

In [686]:
import pandas as pd
path = 'C:\\Users\\Yoonhee\\Desktop\\공모전\\빅콘\\'

rawdata = pd.read_excel(path+'finaldata.xlsx', encoding = 'euc_kr')
rawdata = rawdata.drop(['Unnamed: 0'],axis = 1)

In [687]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [688]:
#MAE
def mae(y,yhat):
    return np.mean(abs(y - yhat))

In [689]:
rawdata[['year','quarter']] = rawdata[['year','quarter']].astype('object')
rawdata.dtypes

year                           object
quarter                        object
sigungu                         int64
dong                            int64
amt                             int64
                                ...  
여성연령대_60_이상_일요일시간대_2_유동인구_수     int64
여성연령대_60_이상_일요일시간대_3_유동인구_수     int64
여성연령대_60_이상_일요일시간대_4_유동인구_수     int64
여성연령대_60_이상_일요일시간대_5_유동인구_수     int64
여성연령대_60_이상_일요일시간대_6_유동인구_수     int64
Length: 618, dtype: object

In [690]:
data = pd.get_dummies(rawdata, columns=['year','quarter','sigungu','dong'], prefix=['year','quarter','sigungu','dong'])

In [691]:
import numpy as np
data["Logamt"] = np.log1p(data["amt"])
data = data.drop(['amt'],axis=1)
data.head()

,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,남성연령대 10 상주인구 수,...,dong_11140665,dong_11140670,dong_11350560,dong_11350595,dong_11350611,dong_11350624,dong_11350630,dong_11350640,dong_11350670,Logamt
0,657,376,281,35,55,80,119,194,174,16,...,0,0,0,0,0,0,0,0,0,23.134886
1,964,353,611,0,55,171,244,317,177,0,...,0,0,0,0,0,0,0,0,0,25.320105
2,2881,1575,1306,123,664,615,600,611,268,79,...,0,0,0,0,0,0,0,0,0,23.014759
3,6,0,6,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,23.989693
4,1750,1085,665,34,219,224,610,606,57,14,...,0,0,0,0,0,0,0,0,0,22.848240


In [692]:
def predict_columns(data_frame, x_cols, y_cols):
    train=data_frame[data_frame['year_2019'] != 1]
    test=data_frame[data_frame['year_2019'] == 1]
    reg_models = [GradientBoostingRegressor(),XGBRegressor()]
    reg_model_names=["GradientBoosting","XGBoost"]
    for i in range(len(reg_models)):
        reg = reg_models[i].fit(train[x_cols],train[y_cols])
        pred = reg_models[i].predict(test[x_cols])
        r2 = r2_score(pred,test[y_cols])
        rmse = mean_squared_error(pred, test[y_cols])**0.5
        print('{} : {},  {},  {}'.format(reg_model_names[i], mae(test[y_cols], pred.astype(int)),r2,rmse))

In [693]:
predict_columns(data,['총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수'],'Logamt')

GradientBoosting : 0.379939270490867,  0.9332164488441482,  0.2418929944688228
XGBoost : 0.3726600361690528,  0.9524824162040172,  0.2259931669631528


In [741]:
finaltestdata2020.head()

,sigungu,dong,상권_코드,상권_코드_명,Logamt
0,11140,11140590,1001493,동대문패션타운 관광특구,24.434017
1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,25.886937
2,11140,11140680,1001285,합동(새벽)시장,20.167218
3,11140,11140670,1001284,서울중앙시장,22.385970
4,11140,11140650,1001283,백학시장,22.800040


XGB regression에 데이터 적합

In [749]:
x_train = finaltestdata[finaltestdata['year_2020'] != 1][['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019','year_2020', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']]
y_train = finaltestdata[finaltestdata['year_2020'] != 1]['Logamt']
reg2 = XGBRegressor()
reg2.fit(x_train, y_train)
print(reg2.score(x_train, y_train))

0.944488077208038


점수가 위의 Gradient Boosting에 비해 월등히 좋아 결론적으로 XGBoost 모델을 선택하게 되었다

예측한 logamt

In [743]:
x_test = finaltestdata[finaltestdata['year_2020'] == 1][['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4','총 상주인구 수', '남성 상주인구 수', '여성 상주인구 수', '연령대 10 상주인구 수', '연령대 20 상주인구 수']]
y_predict = reg2.predict(x_test)
print(y_predict)

[24.341238 25.991917 20.33211  22.61191  23.041515 22.615759 24.583767
 23.531631 24.513292 24.879608 24.512499 24.796543 23.379848 24.594831
 24.830406 24.85797  24.476824 22.9473   24.705923 24.57797  25.102205
 24.583767 22.963608 26.02384  23.73523  26.243076 23.4296   24.576284
 22.880789 22.992432 23.89025  22.943384 23.442102 23.063494 23.203022
 24.814201 23.212215 22.119154 22.808716 23.05576  24.445635 25.319302
 22.639841 22.950695 22.641247 22.55033  21.672785 23.474161 21.936857
 21.917433 22.999191 22.638546 22.611029 22.385695 23.150322 24.00306
 23.322598 23.239895 21.698868 21.852774 21.95568  23.119047 22.830647
 23.213335 24.205864 23.794498 22.129257 23.359882 23.283026 21.142946]


In [744]:
y_predict = pd.DataFrame(y_predict)

In [745]:
finaltestdata2020 = finaltestdata[finaltestdata['year_2020']==1]
finaltestdata2020 = finaltestdata2020[['sigungu','dong','상권_코드','상권_코드_명','Logamt']]
finaltestdata2020.columns

Index(['sigungu', 'dong', '상권_코드', '상권_코드_명', 'Logamt'], dtype='object')

In [746]:
y_predict = y_predict.rename(columns = {0:'predict_logamt'})
y_predict.head()

,predict_logamt
0,24.341238
1,25.991917
2,20.332109
3,22.611910
4,23.041515


In [747]:
finaltestdata2020 = finaltestdata2020.reset_index(drop=True)
finaltestdata2020

,sigungu,dong,상권_코드,상권_코드_명,Logamt
0,11140,11140590,1001493,동대문패션타운 관광특구,24.434017
1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,25.886937
2,11140,11140680,1001285,합동(새벽)시장,20.167218
3,11140,11140670,1001284,서울중앙시장,22.385970
4,11140,11140650,1001283,백학시장,22.800040
...,...,...,...,...,...
65,11350,11350640,1000382,상계로12길,24.032222
66,11350,11350630,1000381,동일로242길,22.014174
67,11350,11350595,1000380,동일로192길,23.251698
68,11350,11350595,1000379,동일로173가길,23.251698


In [748]:
result = pd.concat([finaltestdata2020,y_predict], axis=1)
result.head()

,sigungu,dong,상권_코드,상권_코드_명,Logamt,predict_logamt
0,11140,11140590,1001493,동대문패션타운 관광특구,24.434017,24.341238
1,11140,11140520,1001492,명동 남대문 북창동 다동 무교동 관광특구,25.886937,25.991917
2,11140,11140680,1001285,합동(새벽)시장,20.167218,20.332109
3,11140,11140670,1001284,서울중앙시장,22.385970,22.611910
4,11140,11140650,1001283,백학시장,22.800040,23.041515


In [750]:
result["amt"] = np.exp(result["Logamt"])
result["predict_amt"] = np.exp(result["predict_logamt"])
result = result[['sigungu','dong','Logamt','predict_logamt','amt','predict_amt']]
result.head()

,sigungu,dong,Logamt,predict_logamt,amt,predict_amt
0,11140,11140590,24.434017,24.341238,4.088451e+10,3.726193e+10
1,11140,11140520,25.886937,25.991917,1.748050e+11,1.941538e+11
2,11140,11140680,20.167218,20.332109,5.734713e+08,6.762744e+08
3,11140,11140670,22.385970,22.611910,5.273552e+09,6.610399e+09
4,11140,11140650,22.800040,23.041515,7.978687e+09,1.015788e+10


In [751]:
result=result.groupby(['sigungu','dong']).sum()
result.head()


Logamt  predict_logamt           amt   predict_amt
sigungu dong                                                            
11140   11140520  129.434684      127.673004  8.740249e+11  7.524553e+11
        11140540   68.805753       70.034164  2.740149e+10  4.145880e+10
        11140550  124.329335      124.669960  3.148320e+11  3.482461e+11
        11140570   47.809303       47.164829  4.816019e+10  3.530156e+10
        11140580  114.285605      115.153763  4.223685e+10  5.055123e+10

In [808]:
result['ratio'] = (result['predict_amt']-result['amt'])/result['predict_amt']*100
result = result.sort_values('ratio')
result.to_excel('xgboost_amtratio.xlsx')

In [809]:
result.head()

,,Logamt,predict_logamt,amt,predict_amt,ratio
sigungu,dong,,,,,
11350,11350640,96.128890,93.352692,1.094263e+11,6.342424e+10,-72.530669
11140,11140520,129.434684,125.613480,8.740249e+11,5.159035e+11,-69.416363
11350,11350624,23.047809,22.561173,1.022200e+10,6.283374e+09,-62.683366
11140,11140570,47.809303,47.025211,4.816019e+10,3.287157e+10,-46.510161
11350,11350560,22.623790,22.420408,6.689402e+09,5.458324e+09,-22.554143


In [810]:
result = result.sort_values('ratio',ascending=True)
result.head()

,,Logamt,predict_logamt,amt,predict_amt,ratio
sigungu,dong,,,,,
11350,11350640,96.128890,93.352692,1.094263e+11,6.342424e+10,-72.530669
11140,11140520,129.434684,125.613480,8.740249e+11,5.159035e+11,-69.416363
11350,11350624,23.047809,22.561173,1.022200e+10,6.283374e+09,-62.683366
11140,11140570,47.809303,47.025211,4.816019e+10,3.287157e+10,-46.510161
11350,11350560,22.623790,22.420408,6.689402e+09,5.458324e+09,-22.554143
